In [1]:
import matplotlib.pyplot as plt
%matplotlib tk
import matplotlib.animation as animation
import time
#libraries we need:
import mysql.connector as my_sql
import matplotlib.animation as animation 
import matplotlib.pyplot as plt 
import numpy as np 
global data
import scipy.signal as signal
from scipy.signal import find_peaks
import math


In [17]:
#need another code to upload these heartrates into the database
from collections import Counter
import functools

arr = []
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)
#maybe init
my_db = my_sql.connect(
        host="localhost",
        user="Arduino",
        passwd="ArduinoGetStarted.com",
        database="vital_signs")
my_cursor = my_db.cursor()
yacc_table = "acceleration" #get accel
HR_table = "test" #upload HR
bed_status = 0 #initialize for get_bed_status

#parameters for calculate heart rate:
smoothrate = 3
bandpass = [0.81,2]
samplingfreq = 10
peakdis = 3
peakprom = 0.5

def count_duplicates(lst):
    counter = Counter(lst)
    for tup, count in counter.items():
        if count > 1:
            print(f"{tup}: {count}")
    if not any(count > 1 for count in counter.values()):
        print("No Duplicates")
    return count, tup

def get_bed_status(table): #select acceleration data from desired table - WORKS
    sql = "SELECT status FROM "+table+" ORDER BY id DESC LIMIT 3"
    my_cursor.execute(sql)
    bed_array = my_cursor.fetchall()
    print(bed_array)
    reps, tup = count_duplicates(bed_array)
    tup = functools.reduce(lambda sub, ele: sub * 10 + ele, tup)
    #print(tup)
    #print(reps)
    #reps=1 #rn for testing!!
    if tup == 0:
        if reps ==1: #found 3 in a row with no one there
            bed_status = 0
        else:
            bed_status = 1
    else:
        bed_status = 1
    print(bed_status)
    return bed_status

def check_db_size(table, max, delete): #works to maintain the desired database sizes
    my_cursor.execute("SELECT COUNT(*) FROM "+table)
    data = my_cursor.fetchall()
    count = data[0][0]
    if count>max: #or can do it count>100
        sql = "DELETE FROM "+table+" LIMIT "+str(delete)
        my_cursor.execute(sql) #can't use delete top w/ mysql - this will delete from row and limit it to the top 50 rows
        my_db.commit()
        print("deleted because had")
        print(count)
    #end of function 4 - maybe return size

def select_accel(table): #select acceleration data from desired table - WORKS
    sql1 = "SELECT * FROM "+table
    my_cursor.execute(sql1)
    data = my_cursor.fetchall() #TEST AGAIN ONCE MYSQL ON I had bed_status here instead ie named wrong hence maybe why data never got properly returned
    check_db_size(table, 150, 100) #to range from 50 to 150 always?
    return data

def minute_data(data):
    arr = [] #forgot to reclear this each time, maybe the reason we got multiple data things?
    xar = [] #not working bc never passed id - data is just the array of accel values
    for i in range(len(data)-3,len(data)): #get last 3 rows of data
    #for i in range(0,len(data)):
        yar = []
        xar.append(data[i][0]) #id
        print(data[i][0],":\n")
        yar.append(data[i][5]) #array; row 4 = breathing rate
        yar = (data[i][5].split(","))
        for i in range(0,len(yar)-1):
            yar[i] = float(yar[i]) #convert string to float
        yar.pop() #removes the extra value
        #print(yar)
        arr.append(yar)
    graphData = arr #rename
    return (graphData) #checked and returns 3 values in the array

def CalculateSingleHR(smoothrate,bandpassArr, samplingfreq, peakdis, peakprom, data, numberofperiods = 1):
    
    if isinstance(data, np.ndarray):
        print("Good to continue")    
    else:
        data = np.array(data)

    length = np.array(data).size
    plt.plot(range(0,length), data, color = 'red')
    plt.show()
    
    newArr = np.ones(length)
    
    sos = signal.butter(5, bandpassArr, 'band', fs=samplingfreq, output='sos')
    #sos = signal.butter(10, [20, 200], 'band', fs=450, output='sos')
    filterdata = signal.sosfilt(sos, data)
    filterdata = filterdata[35:]
    length = np.array(filterdata).size
    plt.plot(filterdata, color = "blue")
    plt.show()

    for j in range(0, length-smoothrate):
        RMS = 0
        for k in range(0,smoothrate):
            RMS += (filterdata[j+k]) ** 2
        RMS = math.sqrt(RMS/(smoothrate))
        newArr[j] = RMS
    plt.plot(range(0,length-smoothrate), newArr[:length-smoothrate])
    plt.show()
    HR = 0

    peaks, _ = find_peaks(newArr, distance=peakdis, prominence = peakprom)
    #peaks, _ = find_peaks(newArr, distance=3, prominence = 1.5)


    HR = HR + len(peaks)*numberofperiods #6 for 10s
    print(HR)
    
    plt.plot(range(0, length- smoothrate),newArr[:length-smoothrate])
    plt.plot(peaks, newArr[peaks], "x")
    plt.show()
    predicted = HR
        
    return predicted
    
def insert_HR(HR, bed_status, table):
    sql = "INSERT INTO "+table+" (heart_rate, bed_status) VALUES (%s,%s)" #need 2 values or it won't work for me
    val = (HR, bed_status)
    my_cursor.execute(sql, val)
    my_db.commit() #need this after every execute!!
    print("1 record inserted, ID:", my_cursor.lastrowid) #proof that it works
    #end 3 - don't need a return
    check_db_size(table, 100, 50) #want to keep HR database b/w 50 to 100 entries for now

def check_repeats(table, old_id): #to not run plot/upload to HR if nothing has been updated in accel table
    sql = "SELECT id FROM "+table+" ORDER BY id DESC LIMIT 3"
    my_cursor.execute(sql)
    new_id = my_cursor.fetchall()
    if old_id ==new_id: #found 3 in a row with no one there
        id_status = 1
    else:
        id_status = 0
    return id_status
         
def animate(i):
    graphData = 0;
    #check_repeats(table, old_id)
    #function 4: bed status
    bed_status = get_bed_status("acceleration") #works
    if bed_status == 0:
        HR = 0
    else:
        #function 0: get data from desired table and set to variable called data
        data = select_accel("acceleration") 
        if(len(data) > 300):
            print("Huge Data")
        #function 1: get last 3 rows and add all values as floats to an array
        graphData = minute_data(data)
        #print(graphData) #display all yacc values from that 1min total increment
        HR = 0 #initialize
        for i in range(len(graphData)-3,len(graphData)): #checked and len graphData == 3
            PredictedHR = CalculateSingleHR(smoothrate, bandpass, samplingfreq, peakdis, peakprom, graphData[i])
            HR = HR + PredictedHR #should get 3 heart rate values
            # #plt.plot(range(40*i, 40+40*i),tempArr)
            # #plt.plot(peaks+40*i, tempArr[peaks], "x")
            # #plt.plot(peaks, graphData[peaks], "x")
        #HR = PredictedHR
        print(HR)
        graphData = 0

    #both bed_status meet here = either calculated HR or HR=0
    #function 3 = insert HR into database table with HR
    insert_HR(HR, bed_status, "test")

    #
    #plt.show() #too many plots rn
    #fig = plt.figure()
    #ax1 = fig.add_subplot(1,1,1)

    #function 5: graph real-time update by extracting all rows from HR graph
    my_cursor.execute("SELECT * FROM test")
    data = my_cursor.fetchall()
    xar = []
    yar = []
    for row in data:
        xar.append(row[0]) #timestamp - id actually omg it's all zero
            #print(row)
        yar.append(row[3]) #heartrate; row 4 = breathing rate
    #sql = "INSERT INTO test (heart_rate, bed_status) VALUES (%s,%s)" #need 2 values or it won't work for me
    #val = (HR, 1)
    #my_cursor.execute(sql, val)
    #my_db.commit()
    ax1.clear()
    ax1.plot(xar,yar)
    #end of 5 - return graph idk

    #save var somewhere
    sql = "SELECT id FROM acceleration ORDER BY id DESC LIMIT 3"
    my_cursor.execute(sql)
    old_id = my_cursor.fetchall()

ani = animation.FuncAnimation(fig, animate, interval=60000)
#ani = animation.FuncAnimation(fig, animate, fargs=(xs, ys), interval=1000)
#ani = animation.FuncAnimation(fig, animate, interval=1000)
#plt.show()

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_20444\2142686262.py:213: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani = animation.FuncAnimation(fig, animate, interval=60000)


[(1,), (0,), (0,)]
(0,): 2
1
1513 :

1514 :

1515 :

26
27
31
84
1 record inserted, ID: 845
[(1,), (0,), (0,)]
(0,): 2
1
1513 :

1514 :

1515 :

26
27
31
84
1 record inserted, ID: 846
[(1,), (1,), (1,)]
(1,): 3
1
1516 :

1517 :

1518 :

25
27
25
77
1 record inserted, ID: 847
[(1,), (1,), (1,)]
(1,): 3
1
1518 :

1519 :

1520 :

25
26
27
78
1 record inserted, ID: 848
[(1,), (1,), (1,)]
(1,): 3
1
1521 :

1522 :

1523 :

28
24
27
79
1 record inserted, ID: 849
[(1,), (1,), (1,)]
(1,): 3
1
1524 :

1525 :

1526 :

26
22
24
72
1 record inserted, ID: 850
[(1,), (0,), (1,)]
(1,): 2
0
1 record inserted, ID: 851
[(1,), (1,), (1,)]
(1,): 3
1
1529 :

1530 :

1531 :

30
31
24
85
1 record inserted, ID: 852
[(1,), (1,), (1,)]
(1,): 3
1
1532 :

1533 :

1534 :

28
28
30
86
1 record inserted, ID: 853
[(1,), (1,), (1,)]
(1,): 3
1
1535 :

1536 :

1537 :

22
30
26
78
1 record inserted, ID: 854
[(1,), (1,), (1,)]
(1,): 3
1
1538 :

1539 :

1540 :

26
24
27
77
1 record inserted, ID: 855
[(1,), (1,), (1,)]
(1,):

In [15]:
#ani = animation.FuncAnimation(fig, animate, interval=60000)
ani.event_source.stop() 
#note: can just randomly run this and it'll stop it luckily

In [16]:
print(graphData)

[[-13608.33, -13650.0, -13625.0, -13633.33, -13625.0, -13625.0, -13575.0, -13608.33, -13625.0, -13616.67, -13666.67, -13658.33, -13625.0, -13633.33, -13675.0, -13600.0, -13608.33, -13625.0, -13625.0, -13616.67, -13625.0, -13583.33, -13583.33, -13575.0, -13583.33, -13600.0, -13716.67, -13541.67, -13616.67, -13600.0, -13583.33, -13575.0, -13583.33, -13575.0, -13625.0, -13583.33, -13575.0, -13608.33, -13608.33, -13591.67, -13625.0, -13633.33, -13600.0, -13633.33, -13633.33, -13600.0, -13608.33, -13625.0, -13608.33, -13575.0, -13608.33, -13525.0, -13583.33, -13525.0, -13558.33, -13608.33, -13491.67, -13550.0, -13575.0, -13566.67, -13558.33, -13583.33, -13608.33, -13566.67, -13583.33, -13583.33, -13583.33, -13591.67, -13583.33, -13600.0, -13616.67, -13575.0, -13658.33, -13633.33, -13733.33, -13675.0, -13650.0, -13683.33, -13633.33, -13591.67, -13658.33, -13675.0, -13633.33, -13666.67, -13633.33, -13600.0, -13600.0, -13575.0, -13608.33, -13591.67, -13616.67, -13616.67, -13625.0, -13650.0, -1

In [ ]:
#for haley: 100 entries

[[-5750.0, -5766.67, -5741.67, -5741.67, -5750.0, -5733.33, -5750.0, -5733.33, -5750.0, -5725.0, -5733.33, -5733.33, -5733.33, -5741.67, -5741.67, -5766.67, -5758.33, -5758.33, -5766.67, -5775.0, -5783.33, -5783.33, -5800.0, -5791.67, -5791.67, -5791.67, -5791.67, -5766.67, -5783.33, -5775.0, -5783.33, -5766.67, -5758.33, -5758.33, -5758.33, -5741.67, -5750.0, -5733.33, -5733.33, -5733.33, -5733.33, -5741.67, -5750.0, -5741.67, -5758.33, -5766.67, -5750.0, -5775.0, -5783.33, -5791.67, -5808.33, -5791.67, -5808.33, -5808.33, -5800.0, -5791.67, -5791.67, -5791.67, -5775.0, -5775.0, -5766.67, -5758.33, -5766.67, -5741.67, -5750.0, -5733.33, -5741.67, -5741.67, -5741.67, -5741.67, -5750.0, -5766.67, -5758.33, -5766.67, -5775.0, -5783.33, -5800.0, -5800.0, -5791.67, -5800.0, -5800.0, -5800.0, -5800.0, -5766.67, -5783.33, -5775.0, -5766.67, -5775.0, -5750.0, -5758.33, -5750.0, -5750.0, -5733.33, -5741.67, -5733.33, -5733.33, -5733.33, -5741.67, -5741.67, -5758.33, -5758.33, -5791.67, -5750.0, -5791.67, -5791.67, -5783.33, -5800.0, -5791.67, -5800.0, -5800.0, -5791.67, -5783.33, -5783.33, -5775.0, -5775.0, -5766.67, -5758.33, -5758.33, -5750.0, -5725.0, -5741.67, -5733.33, -5741.67, -5741.67, -5733.33, -5733.33, -5741.67, -5750.0, -5766.67, -5750.0, -5783.33, -5775.0, -5775.0, -5783.33, -5775.0, -5791.67, -5783.33, -5783.33, -5775.0, -5791.67, -5766.67, -5766.67, -5750.0, -5758.33, -5750.0, -5750.0, -5741.67, -5750.0, -5741.67, -5733.33, -5741.67, -5733.33, -5716.67, -5733.33, -5741.67, -5741.67, -5725.0, -5758.33, -5750.0, -5750.0, -5766.67, -5766.67, -5766.67, -5775.0, -5783.33, -5775.0, -5783.33, -5783.33, -5775.0, -5775.0, -5783.33, -5758.33, -5758.33, -5750.0, -5733.33, -5750.0, -5733.33, -5733.33, -5741.67, -5725.0, -5725.0, -5716.67, -5716.67, -5725.0, -5716.67, -5741.67, -5741.67, -5741.67, -5758.33, -5750.0, -5758.33, -5758.33, -5783.33, -5750.0, -5791.67, -5783.33, -5775.0, -5791.67, -5783.33, -5775.0], [-5766.67, -5775.0, -5766.67, -5783.33, -5783.33, -5783.33, -5766.67, -5783.33, -5766.67, -5775.0, -5758.33, -5750.0, -5758.33, -5750.0, -5741.67, -5733.33, -5733.33, -5725.0, -5733.33, -5725.0, -5725.0, -5733.33, -5716.67, -5716.67, -5725.0, -5733.33, -5716.67, -5750.0, -5741.67, -5741.67, -5750.0, -5766.67, -5758.33, -5766.67, -5766.67, -5783.33, -5775.0, -5775.0, -5783.33, -5766.67, -5766.67, -5758.33, -5758.33, -5758.33, -5741.67, -5750.0, -5733.33, -5750.0, -5741.67, -5741.67, -5741.67, -5725.0, -5733.33, -5733.33, -5725.0, -5733.33, -5716.67, -5725.0, -5725.0, -5725.0, -5733.33, -5750.0, -5741.67, -5741.67, -5741.67, -5758.33, -5741.67, -5783.33, -5766.67, -5758.33, -5766.67, -5766.67, -5766.67, -5758.33, -5750.0, -5741.67, -5741.67, -5741.67, -5741.67, -5741.67, -5733.33, -5716.67, -5741.67, -5716.67, -5716.67, -5666.67, -5650.0, -5650.0, -5633.33, -5716.67, -5708.33, -5691.67, -5633.33, -5633.33, -5641.67, -5608.33, -5616.67, -5616.67, -5591.67, -5600.0, -5616.67, -5583.33, -5600.0, -5566.67, -5575.0, -5566.67, -5541.67, -5566.67, -5558.33, -5591.67, -5625.0, -5600.0, -5616.67, -5616.67, -5575.0, -5591.67, -5625.0, -5583.33, -5508.33, -5466.67, -5483.33, -5483.33, -5450.0, -5550.0, -5566.67, -5483.33, -5558.33, -5533.33, -5508.33, -5466.67, -5383.33, -5008.33, -4975.0, -4800.0, -5158.33, -4908.33, -5066.67, -5183.33, -5341.67, -5191.67, -5050.0, -4991.67, -4933.33, -4983.33, -5041.67, -5033.33, -5033.33, -5091.67, -5116.67, -5083.33, -5116.67, -5100.0, -5108.33, -5108.33, -5100.0, -5108.33, -5108.33, -5125.0, -5125.0, -5141.67, -5133.33, -5100.0, -5108.33, -5091.67, -5058.33, -5066.67, -5108.33, -5116.67, -5133.33, -5158.33, -5158.33, -5208.33, -5183.33, -5241.67, -5300.0, -5225.0, -5250.0, -5283.33, -5275.0, -5275.0, -5316.67, -5275.0, -5266.67, -5266.67, -5183.33, -5291.67, -5233.33, -5283.33, -5258.33, -5291.67, -5250.0, -5300.0, -5308.33, -5300.0, -5300.0, -5325.0, -5316.67, -5316.67, -5308.33, -5358.33], [-5241.67, -5250.0, -5283.33, -5283.33, -5250.0, -5275.0, -5291.67, -5275.0, -5283.33, -5308.33, -5316.67, -5316.67, -5350.0, -5350.0, -5316.67, -5333.33, -5316.67, -5300.0, -5325.0, -5316.67, -5325.0, -5308.33, -5300.0, -5300.0, -5291.67, -5291.67, -5283.33, -5275.0, -5275.0, -5258.33, -5275.0, -5258.33, -5258.33, -5266.67, -5266.67, -5258.33, -5283.33, -5266.67, -5291.67, -5308.33, -5291.67, -5300.0, -5308.33, -5341.67, -5316.67, -5325.0, -5325.0, -5300.0, -5316.67, -5308.33, -5291.67, -5308.33, -5291.67, -5291.67, -5283.33, -5283.33, -5291.67, -5283.33, -5275.0, -5266.67, -5275.0, -5258.33, -5258.33, -5250.0, -5241.67, -5250.0, -5266.67, -5241.67, -5258.33, -5266.67, -5258.33, -5250.0, -5266.67, -5241.67, -5258.33, -5241.67, -5216.67, -5158.33, -5066.67, -4925.0, -4883.33, -4841.67, -4825.0, -4833.33, -4800.0, -4766.67, -4783.33, -4775.0, -4616.67, -4508.33, -4416.67, -4325.0, -4291.67, -4258.33, -4175.0, -4091.67, -4091.67, -3941.67, -3808.33, -3658.33, -3650.0, -3675.0, -3650.0, -3591.67, -3658.33, -3716.67, -3850.0, -3916.67, -4058.33, -4225.0, -4541.67, -4725.0, -4975.0, -5133.33, -5141.67, -5166.67, -5233.33, -5250.0, -5266.67, -5266.67, -5275.0, -5291.67, -5308.33, -5283.33, -5275.0, -5275.0, -5258.33, -5233.33, -5258.33, -5241.67, -5308.33, -5275.0, -5300.0, -5308.33, -5291.67, -5325.0, -5358.33, -5366.67, -5375.0, -5333.33, -5441.67, -5408.33, -5383.33, -5433.33, -5400.0, -5441.67, -5425.0, -5416.67, -5425.0, -5441.67, -5450.0, -5441.67, -5425.0, -5425.0, -5425.0, -5391.67, -5408.33, -5391.67, -5391.67, -5400.0, -5391.67, -5391.67, -5383.33, -5400.0, -5391.67, -5416.67, -5408.33, -5408.33, -5425.0, -5433.33, -5450.0, -5458.33, -5433.33, -5466.67, -5458.33, -5458.33, -5466.67, -5466.67, -5475.0, -5475.0, -5441.67, -5441.67, -5450.0, -5441.67, -5441.67, -5433.33, -5441.67, -5425.0, -5450.0, -5433.33, -5466.67, -5483.33, -5475.0, -5491.67, -5491.67, -5500.0, -5475.0, -5458.33, -5466.67, -5508.33], [-5466.67, -5441.67, -5508.33, -5516.67, -5475.0, -5416.67, -5458.33, -5400.0, -5366.67, -5325.0, -5258.33, -5233.33, -5166.67, -5141.67, -5141.67, -5150.0, -5200.0, -5183.33, -5233.33, -5266.67, -5258.33, -5250.0, -5308.33, -5308.33, -5316.67, -5291.67, -5350.0, -5333.33, -5366.67, -5383.33, -5425.0, -5391.67, -5608.33, -5583.33, -5641.67, -5608.33, -5591.67, -5591.67, -5525.0, -5441.67, -5508.33, -5533.33, -5508.33, -5516.67, -5575.0, -5550.0, -5550.0, -5550.0, -5525.0, -5591.67, -5608.33, -5533.33, -5566.67, -5575.0, -5600.0, -5575.0, -5591.67, -5575.0, -5608.33, -5525.0, -5550.0, -5558.33, -5541.67, -5500.0, -5550.0, -5516.67, -5516.67, -5491.67, -5483.33, -5516.67, -5475.0, -5508.33, -5516.67, -5500.0, -5491.67, -5541.67, -5550.0, -5516.67, -5533.33, -5458.33, -5191.67, -4941.67, -4933.33, -5066.67, -5216.67, -5225.0, -5208.33, -5291.67, -5333.33, -5316.67, -5291.67, -5300.0, -5225.0, -5133.33, -5191.67, -5158.33, -5141.67, -5158.33, -5208.33, -5200.0, -5200.0, -5183.33, -5166.67, -5216.67, -5166.67, -5150.0, -5191.67, -5141.67, -5166.67, -5150.0, -5158.33, -5158.33, -5150.0, -5150.0, -5175.0, -5158.33, -5125.0, -5116.67, -5133.33, -5150.0, -5108.33, -5091.67, -5100.0, -5083.33, -5091.67, -5050.0, -5100.0, -5025.0, -5066.67, -5050.0, -5108.33, -5041.67, -5050.0, -5083.33, -5066.67, -5041.67, -5066.67, -5075.0, -5033.33, -5033.33, -5050.0, -4800.0, -4566.67, -4400.0, -4250.0, -4141.67, -4208.33, -4341.67, -4175.0, -4266.67, -4241.67, -4333.33, -4508.33, -4700.0, -4816.67, -5016.67, -5091.67, -5050.0, -5100.0, -5125.0, -5108.33, -5108.33, -5108.33, -5158.33, -5150.0, -5158.33, -5158.33, -5166.67, -5200.0, -5166.67, -5166.67, -5150.0, -5166.67, -5166.67, -5166.67, -5158.33, -5166.67, -5133.33, -5150.0, -5175.0, -5183.33, -5166.67, -5175.0, -5150.0, -5166.67, -5141.67, -5158.33, -5125.0, -5175.0, -5158.33, -5158.33, -5116.67, -5150.0, -5116.67, -5200.0, -5100.0, -5141.67, -5125.0, -5216.67, -5133.33], [-5150.0, -5208.33, -5208.33, -5275.0, -5208.33, -5225.0, -5258.33, -5266.67, -5233.33, -5225.0, -5258.33, -5258.33, -5225.0, -5258.33, -5241.67, -5291.67, -5216.67, -5241.67, -5258.33, -5258.33, -5266.67, -5250.0, -5275.0, -5233.33, -5241.67, -5225.0, -5225.0, -5266.67, -5200.0, -5275.0, -5200.0, -5200.0, -5250.0, -5308.33, -5216.67, -5183.33, -5258.33, -5191.67, -5216.67, -5216.67, -5208.33, -5191.67, -5183.33, -5208.33, -5175.0, -5158.33, -5108.33, -5150.0, -5116.67, -5133.33, -5116.67, -5100.0, -5125.0, -5066.67, -5083.33, -5075.0, -5058.33, -5066.67, -5075.0, -5058.33, -5066.67, -5058.33, -5075.0, -5041.67, -5058.33, -5025.0, -5033.33, -5016.67, -5008.33, -5008.33, -5016.67, -5016.67, -5008.33, -5016.67, -5000.0, -4991.67, -5008.33, -5016.67, -5016.67, -5000.0, -5008.33, -4966.67, -4991.67, -5066.67, -5000.0, -5033.33, -5000.0, -5000.0, -5016.67, -5025.0, -5033.33, -5033.33, -5041.67, -5041.67, -5008.33, -5050.0, -5050.0, -5025.0, -5025.0, -5016.67, -5008.33, -5025.0, -5033.33, -5016.67, -5016.67, -5050.0, -5016.67, -5033.33, -5041.67, -5016.67, -5066.67, -5050.0, -5025.0, -5050.0, -5066.67, -5041.67, -5058.33, -5066.67, -5058.33, -5075.0, -5083.33, -5075.0, -5125.0, -5066.67, -5075.0, -5150.0, -5116.67, -5116.67, -5091.67, -5108.33, -5100.0, -5083.33, -5091.67, -5058.33, -5058.33, -5075.0, -5058.33, -5075.0, -5058.33, -5025.0, -5016.67, -5058.33, -5050.0, -5016.67, -5025.0, -5008.33, -5008.33, -5033.33, -5000.0, -5033.33, -5033.33, -5041.67, -5033.33, -5050.0, -5041.67, -5050.0, -5016.67, -5025.0, -5050.0, -5066.67, -4983.33, -5033.33, -5016.67, -5041.67, -5000.0, -5008.33, -5008.33, -5083.33, -4975.0, -5000.0, -5008.33, -5050.0, -4991.67, -4991.67, -5008.33, -5058.33, -4958.33, -5016.67, -4975.0, -5025.0, -4966.67, -4975.0, -4966.67, -4958.33, -4975.0, -4983.33, -4966.67, -4950.0, -4891.67, -4958.33, -4916.67, -4908.33, -4933.33, -4916.67, -4875.0, -5075.0, -5025.0, -4975.0, -5041.67, -5025.0], [-5000.0, -4991.67, -5125.0, -5000.0, -5066.67, -5100.0, -5058.33, -5075.0, -5050.0, -5116.67, -5041.67, -5083.33, -5000.0, -5083.33, -5033.33, -5025.0, -4983.33, -5050.0, -5066.67, -4941.67, -5025.0, -5100.0, -5108.33, -5025.0, -5083.33, -5200.0, -5000.0, -5091.67, -5100.0, -5066.67, -5100.0, -5066.67, -5016.67, -5183.33, -5008.33, -5091.67, -5033.33, -5083.33, -5133.33, -5000.0, -5058.33, -5100.0, -5025.0, -5000.0, -5166.67, -5083.33, -5050.0, -5033.33, -5108.33, -5058.33, -5000.0, -5150.0, -5133.33, -5133.33, -5158.33, -5158.33, -5166.67, -5125.0, -5183.33, -5108.33, -5175.0, -5108.33, -5191.67, -5108.33, -5183.33, -5166.67, -5125.0, -5150.0, -5150.0, -5166.67, -5108.33, -5150.0, -5108.33, -5175.0, -5100.0, -5141.67, -5108.33, -5158.33, -5083.33, -5133.33, -5141.67, -5175.0, -5150.0, -5125.0, -5125.0, -5216.67, -5141.67, -5066.67, -5158.33, -5116.67, -5100.0, -5083.33, -5183.33, -5091.67, -5041.67, -5091.67, -5141.67, -5083.33, -5041.67, -5116.67, -5125.0, -5108.33, -5116.67, -5200.0, -5091.67, -5133.33, -5175.0, -5200.0, -5166.67, -5175.0, -5166.67, -5200.0, -5175.0, -5125.0, -5183.33, -5200.0, -5166.67, -5100.0, -5166.67, -5116.67, -5133.33, -5083.33, -5133.33, -5100.0, -5116.67, -5091.67, -5125.0, -5116.67, -5116.67, -5141.67, -5225.0, -5141.67, -5108.33, -5175.0, -5183.33, -5158.33, -5116.67, -5191.67, -5125.0, -5116.67, -5116.67, -5191.67, -5125.0, -5066.67, -5116.67, -5125.0, -5100.0, -5075.0, -5116.67, -5058.33, -5108.33, -5108.33, -5183.33, -5125.0, -5108.33, -5166.67, -5191.67, -5183.33, -5108.33, -5216.67, -5166.67, -5141.67, -5158.33, -5250.0, -5158.33, -5091.67, -5158.33, -5133.33, -5116.67, -5133.33, -5158.33, -5233.33, -5183.33, -5191.67, -5158.33, -5208.33, -5183.33, -5225.0, -5158.33, -5208.33, -5225.0, -5183.33, -5241.67, -5250.0, -5183.33, -5250.0, -5141.67, -5266.67, -5191.67, -5241.67, -5150.0, -5283.33, -5183.33, -5141.67, -5191.67, -5208.33, -5208.33, -5116.67, -5183.33, -5208.33], [-5158.33, -5083.33, -5158.33, -5091.67, -5225.0, -5183.33, -5066.67, -5183.33, -5158.33, -5183.33, -5116.67, -5300.0, -5208.33, -5116.67, -5258.33, -5175.0, -5216.67, -5166.67, -5266.67, -5175.0, -5158.33, -5150.0, -5200.0, -5225.0, -5075.0, -5191.67, -5158.33, -5125.0, -5100.0, -5241.67, -5150.0, -5066.67, -5141.67, -5116.67, -5191.67, -5100.0, -5266.67, -5183.33, -5133.33, -5175.0, -5225.0, -5200.0, -5175.0, -5258.33, -5158.33, -5208.33, -5233.33, -5183.33, -5241.67, -5158.33, -5083.33, -5291.67, -5175.0, -5083.33, -5191.67, -5200.0, -5133.33, -5100.0, -5175.0, -5133.33, -5150.0, -5091.67, -5191.67, -5108.33, -5133.33, -5141.67, -5283.33, -5150.0, -5125.0, -5283.33, -5208.33, -5183.33, -5175.0, -5250.0, -5175.0, -5150.0, -5275.0, -5175.0, -5100.0, -5233.33, -5083.33, -5283.33, -5150.0, -5125.0, -5191.67, -5200.0, -5133.33, -5150.0, -5125.0, -5191.67, -5075.0, -5166.67, -5133.33, -5166.67, -5191.67, -5083.33, -5258.33, -5158.33, -5108.33, -5208.33, -5266.67, -5141.67, -5183.33, -5225.0, -5225.0, -5166.67, -5141.67, -5183.33, -5216.67, -5133.33, -5108.33, -5183.33, -5200.0, -5108.33, -5091.67, -5158.33, -5108.33, -5141.67, -5100.0, -5200.0, -5125.0, -5091.67, -5166.67, -5191.67, -5141.67, -5141.67, -5216.67, -5158.33, -5158.33, -5200.0, -5225.0, -5150.0, -5175.0, -5208.33, -5233.33, -5166.67, -5141.67, -5191.67, -5200.0, -5125.0, -5150.0, -5175.0, -5166.67, -5116.67, -5116.67, -5158.33, -5133.33, -5116.67, -5108.33, -5150.0, -5100.0, -5125.0, -5141.67, -5183.33, -5133.33, -5133.33, -5183.33, -5216.67, -5166.67, -5150.0, -5225.0, -5158.33, -5158.33, -5208.33, -5216.67, -5150.0, -5141.67, -5183.33, -5125.0, -5133.33, -5141.67, -5208.33, -5108.33, -5116.67, -5166.67, -5100.0, -5116.67, -5141.67, -5166.67, -5125.0, -5108.33, -5175.0, -5116.67, -5133.33, -5166.67, -5158.33, -5141.67, -5158.33, -5183.33, -5158.33, -5191.67, -5175.0, -5175.0, -5183.33, -5183.33, -5133.33, -5166.67, -5133.33, -5133.33, -5150.0], [-5166.67, -5175.0, -5183.33, -5166.67, -5158.33, -5200.0, -5133.33, -5141.67, -5166.67, -5091.67, -5166.67, -5091.67, -5191.67, -5083.33, -5075.0, -5241.67, -5025.0, -5258.33, -5058.33, -5125.0, -5158.33, -5083.33, -5166.67, -5025.0, -5250.0, -5066.67, -5183.33, -5108.33, -5141.67, -5175.0, -5108.33, -5225.0, -5150.0, -5216.67, -5125.0, -5175.0, -5191.67, -5116.67, -5191.67, -5116.67, -5150.0, -5141.67, -5125.0, -5133.33, -5125.0, -5125.0, -5133.33, -5125.0, -5108.33, -5125.0, -5125.0, -5116.67, -5125.0, -5116.67, -5125.0, -5125.0, -5133.33, -5141.67, -5133.33, -5133.33, -5150.0, -5150.0, -5150.0, -5166.67, -5166.67, -5166.67, -5166.67, -5183.33, -5150.0, -5166.67, -5150.0, -5141.67, -5141.67, -5133.33, -5125.0, -5133.33, -5141.67, -5108.33, -5108.33, -5125.0, -5125.0, -5125.0, -5116.67, -5108.33, -5116.67, -5116.67, -5116.67, -5116.67, -5125.0, -5116.67, -5133.33, -5133.33, -5133.33, -5133.33, -5150.0, -5133.33, -5158.33, -5150.0, -5166.67, -5150.0, -5175.0, -5166.67, -5158.33, -5158.33, -5158.33, -5150.0, -5141.67, -5133.33, -5133.33, -5125.0, -5141.67, -5116.67, -5125.0, -5116.67, -5125.0, -5108.33, -5108.33, -5108.33, -5125.0, -5125.0, -5141.67, -5141.67, -5133.33, -5133.33, -5150.0, -5150.0, -5166.67, -5158.33, -5175.0, -5150.0, -5175.0, -5191.67, -5175.0, -5191.67, -5208.33, -5183.33, -5200.0, -5166.67, -5200.0, -5166.67, -5158.33, -5141.67, -5141.67, -5133.33, -5141.67, -5116.67, -5133.33, -5125.0, -5141.67, -5100.0, -5166.67, -5133.33, -5125.0, -5150.0, -5191.67, -5141.67, -5183.33, -5200.0, -5200.0, -5158.33, -5191.67, -5175.0, -5216.67, -5158.33, -5158.33, -5158.33, -5191.67, -5133.33, -5133.33, -5141.67, -5125.0, -5125.0, -5108.33, -5133.33, -5116.67, -5108.33, -5116.67, -5125.0, -5141.67, -5100.0, -5116.67, -5150.0, -5166.67, -5108.33, -5133.33, -5166.67, -5200.0, -5133.33, -5158.33, -5175.0, -5166.67, -5166.67, -5150.0, -5175.0, -5125.0, -5158.33, -5141.67, -5175.0, -5108.33, -5133.33], [-5158.33, -5166.67, -5158.33, -5150.0, -5158.33, -5133.33, -5150.0, -5133.33, -5116.67, -5125.0, -5116.67, -5108.33, -5116.67, -5116.67, -5108.33, -5100.0, -5108.33, -5100.0, -5108.33, -5125.0, -5125.0, -5125.0, -5150.0, -5150.0, -5133.33, -5158.33, -5166.67, -5158.33, -5166.67, -5175.0, -5150.0, -5166.67, -5166.67, -5125.0, -5158.33, -5158.33, -5133.33, -5133.33, -5133.33, -5125.0, -5125.0, -5116.67, -5108.33, -5091.67, -5116.67, -5116.67, -5108.33, -5116.67, -5116.67, -5133.33, -5116.67, -5116.67, -5116.67, -5133.33, -5150.0, -5116.67, -5141.67, -5141.67, -5166.67, -5150.0, -5158.33, -5166.67, -5141.67, -5166.67, -5133.33, -5166.67, -5116.67, -5116.67, -5133.33, -5166.67, -5108.33, -5100.0, -5133.33, -5116.67, -5091.67, -5083.33, -5116.67, -5116.67, -5083.33, -5100.0, -5125.0, -5150.0, -5108.33, -5116.67, -5141.67, -5150.0, -5125.0, -5150.0, -5158.33, -5158.33, -5150.0, -5133.33, -5166.67, -5133.33, -5141.67, -5150.0, -5125.0, -5116.67, -5125.0, -5125.0, -5108.33, -5100.0, -5108.33, -5100.0, -5108.33, -5083.33, -5091.67, -5100.0, -5100.0, -5108.33, -5083.33, -5108.33, -5100.0, -5100.0, -5100.0, -5116.67, -5108.33, -5125.0, -5125.0, -5141.67, -5133.33, -5150.0, -5150.0, -5150.0, -5150.0, -5150.0, -5141.67, -5133.33, -5141.67, -5116.67, -5133.33, -5125.0, -5083.33, -5108.33, -5116.67, -5100.0, -5091.67, -5091.67, -5100.0, -5075.0, -5108.33, -5083.33, -5091.67, -5108.33, -5133.33, -5108.33, -5125.0, -5141.67, -5166.67, -5116.67, -5133.33, -5166.67, -5158.33, -5133.33, -5150.0, -5158.33, -5150.0, -5108.33, -5133.33, -5125.0, -5150.0, -5100.0, -5125.0, -5116.67, -5108.33, -5091.67, -5116.67, -5116.67, -5091.67, -5083.33, -5100.0, -5116.67, -5100.0, -5125.0, -5116.67, -5125.0, -5133.33, -5150.0, -5125.0, -5166.67, -5150.0, -5150.0, -5150.0, -5150.0, -5141.67, -5141.67, -5150.0, -5125.0, -5133.33, -5125.0, -5116.67, -5125.0, -5116.67, -5116.67, -5100.0, -5108.33, -5108.33, -5083.33, -5108.33, -5108.33], [-5108.33, -5100.0, -5100.0, -5100.0, -5100.0, -5091.67, -5091.67, -5100.0, -5091.67, -5108.33, -5100.0, -5116.67, -5116.67, -5133.33, -5125.0, -5133.33, -5141.67, -5150.0, -5133.33, -5150.0, -5150.0, -5141.67, -5141.67, -5133.33, -5116.67, -5116.67, -5116.67, -5108.33, -5108.33, -5100.0, -5100.0, -5100.0, -5100.0, -5100.0, -5066.67, -5108.33, -5100.0, -5100.0, -5108.33, -5108.33, -5125.0, -5133.33, -5141.67, -5150.0, -5133.33, -5166.67, -5150.0, -5141.67, -5141.67, -5141.67, -5141.67, -5116.67, -5141.67, -5108.33, -5125.0, -5116.67, -5100.0, -5108.33, -5108.33, -5091.67, -5091.67, -5091.67, -5108.33, -5075.0, -5100.0, -5100.0, -5091.67, -5116.67, -5116.67, -5116.67, -5133.33, -5125.0, -5150.0, -5133.33, -5150.0, -5133.33, -5133.33, -5150.0, -5150.0, -5125.0, -5125.0, -5133.33, -5108.33, -5125.0, -5116.67, -5091.67, -5108.33, -5083.33, -5083.33, -5083.33, -5091.67, -5083.33, -5091.67, -5091.67, -5091.67, -5108.33, -5100.0, -5116.67, -5125.0, -5116.67, -5150.0, -5133.33, -5166.67, -5150.0, -5150.0, -5141.67, -5150.0, -5141.67, -5133.33, -5141.67, -5108.33, -5125.0, -5108.33, -5108.33, -5116.67, -5091.67, -5091.67, -5083.33, -5075.0, -5100.0, -5075.0, -5091.67, -5091.67, -5083.33, -5100.0, -5100.0, -5116.67, -5116.67, -5125.0, -5116.67, -5133.33, -5141.67, -5133.33, -5133.33, -5141.67, -5133.33, -5150.0, -5133.33, -5108.33, -5125.0, -5116.67, -5116.67, -5116.67, -5108.33, -5100.0, -5100.0, -5091.67, -5100.0, -5066.67, -5083.33, -5066.67, -5075.0, -5083.33, -5091.67, -5100.0, -5091.67, -5091.67, -5108.33, -5108.33, -5108.33, -5116.67, -5108.33, -5133.33, -5133.33, -5141.67, -5133.33, -5133.33, -5116.67, -5133.33, -5125.0, -5108.33, -5116.67, -5100.0, -5091.67, -5100.0, -5100.0, -5083.33, -5091.67, -5083.33, -5083.33, -5091.67, -5075.0, -5066.67, -5083.33, -5083.33, -5083.33, -5075.0, -5075.0, -5083.33, -5083.33, -5083.33, -5108.33, -5108.33, -5100.0, -5116.67, -5133.33, -5116.67, -5141.67, -5125.0, -5158.33], [-5133.33, -5116.67, -5150.0, -5125.0, -5158.33, -5141.67, -5183.33, -5150.0, -5150.0, -5125.0, -5141.67, -5133.33, -5116.67, -5100.0, -5150.0, -5116.67, -5100.0, -5083.33, -5166.67, -5133.33, -5091.67, -5116.67, -5116.67, -5083.33, -5083.33, -5141.67, -5133.33, -5108.33, -5141.67, -5100.0, -5083.33, -5133.33, -5133.33, -5150.0, -5133.33, -5150.0, -5133.33, -5175.0, -5133.33, -5200.0, -5133.33, -5183.33, -5116.67, -5175.0, -5100.0, -5191.67, -5108.33, -5150.0, -5091.67, -5216.67, -5100.0, -5125.0, -5116.67, -5175.0, -5125.0, -5100.0, -5108.33, -5116.67, -5116.67, -5083.33, -5158.33, -5066.67, -5108.33, -5075.0, -5141.67, -5066.67, -5108.33, -5100.0, -5116.67, -5116.67, -5108.33, -5158.33, -5116.67, -5150.0, -5108.33, -5200.0, -5091.67, -5133.33, -5116.67, -5166.67, -5100.0, -5100.0, -5133.33, -5100.0, -5116.67, -5108.33, -5183.33, -5083.33, -5091.67, -5100.0, -5175.0, -5083.33, -5075.0, -5133.33, -5175.0, -5166.67, -5158.33, -5175.0, -5158.33, -5100.0, -5125.0, -5141.67, -5083.33, -5100.0, -5141.67, -5116.67, -5125.0, -5116.67, -5091.67, -5116.67, -5091.67, -5091.67, -5083.33, -5125.0, -5100.0, -5108.33, -5108.33, -5108.33, -5125.0, -5141.67, -5125.0, -5125.0, -5125.0, -5150.0, -5133.33, -5141.67, -5133.33, -5108.33, -5141.67, -5100.0, -5175.0, -5175.0, -5141.67, -5183.33, -5175.0, -5158.33, -5150.0, -5133.33, -5133.33, -5091.67, -5166.67, -5133.33, -5125.0, -5166.67, -5133.33, -5208.33, -5125.0, -5208.33, -5158.33, -5250.0, -5075.0, -5216.67, -5158.33, -5166.67, -5158.33, -5183.33, -5175.0, -5108.33, -5133.33, -5191.67, -5116.67, -5050.0, -5108.33, -5183.33, -5008.33, -5058.33, -5233.33, -5050.0, -5083.33, -5208.33, -5216.67, -5025.0, -5166.67, -5208.33, -5108.33, -5141.67, -5216.67, -5208.33, -5058.33, -5275.0, -5208.33, -5091.67, -5200.0, -5108.33, -5191.67, -5075.0, -5241.67, -5108.33, -5058.33, -5158.33, -5133.33, -5075.0, -5108.33, -5166.67, -5091.67, -5100.0, -5200.0, -5208.33, -5183.33, -5075.0], [-5116.67, -5116.67, -5108.33, -5125.0, -5058.33, -5150.0, -5100.0, -5100.0, -5158.33, -5183.33, -5091.67, -5183.33, -5158.33, -5141.67, -5175.0, -5108.33, -5200.0, -5183.33, -5116.67, -5150.0, -5083.33, -5166.67, -5183.33, -5058.33, -5158.33, -5133.33, -5133.33, -5116.67, -5125.0, -5108.33, -5100.0, -5108.33, -5075.0, -5133.33, -5133.33, -5158.33, -5133.33, -5125.0, -5133.33, -5125.0, -5116.67, -5141.67, -5075.0, -5091.67, -5091.67, -5091.67, -5091.67, -5083.33, -5141.67, -5133.33, -5125.0, -5100.0, -5116.67, -5158.33, -5166.67, -5141.67, -5166.67, -5133.33, -5108.33, -5183.33, -5116.67, -5116.67, -5150.0, -5100.0, -5116.67, -5116.67, -5133.33, -5141.67, -5150.0, -5133.33, -5150.0, -5175.0, -5158.33, -5108.33, -5175.0, -5175.0, -5150.0, -5125.0, -5166.67, -5166.67, -5158.33, -5125.0, -5183.33, -5183.33, -5141.67, -5091.67, -5141.67, -5091.67, -5141.67, -5116.67, -5125.0, -5100.0, -5066.67, -5100.0, -5158.33, -5141.67, -5108.33, -5175.0, -5158.33, -5141.67, -5083.33, -5191.67, -5108.33, -5108.33, -5133.33, -5208.33, -5141.67, -5075.0, -5133.33, -5100.0, -5108.33, -5125.0, -5200.0, -5141.67, -5108.33, -5183.33, -5241.67, -5125.0, -5166.67, -5166.67, -5208.33, -5191.67, -5125.0, -5175.0, -5166.67, -5166.67, -5166.67, -5183.33, -5116.67, -5125.0, -5183.33, -5083.33, -5200.0, -5158.33, -5075.0, -5125.0, -5158.33, -5091.67, -5083.33, -5100.0, -5108.33, -5091.67, -5033.33, -5133.33, -5150.0, -4958.33, -5116.67, -5125.0, -5008.33, -5116.67, -5091.67, -5091.67, -5083.33, -5075.0, -5133.33, -5066.67, -5058.33, -5091.67, -5075.0, -4958.33, -4908.33, -4858.33, -4483.33, -4416.67, -4508.33, -4525.0, -4408.33, -4458.33, -4525.0, -4541.67, -4583.33, -4650.0, -4741.67, -4908.33, -5025.0, -5191.67, -5550.0, -6016.67, -6391.67, -6616.67, -6775.0, -6891.67, -7025.0, -7175.0, -7191.67, -7208.33, -7200.0, -7233.33, -7216.67, -7216.67, -7233.33, -7316.67, -7483.33, -7133.33, -7216.67, -7275.0, -6875.0, -6708.33, -6675.0, -6550.0], [-7908.33, -7991.67, -8016.67, -7991.67, -8108.33, -8175.0, -7891.67, -7600.0, -7591.67, -7533.33, -7358.33, -7266.67, -6908.33, -6425.0, -6050.0, -6108.33, -6366.67, -6800.0, -7250.0, -7641.67, -7991.67, -8283.33, -8225.0, -8125.0, -7866.67, -7508.33, -7116.67, -6733.33, -6383.33, -6158.33, -6050.0, -5933.33, -5816.67, -5558.33, -5491.67, -5575.0, -5458.33, -5466.67, -5658.33, -5775.0, -6008.33, -6325.0, -6866.67, -7183.33, -7408.33, -7491.67, -7266.67, -6700.0, -6750.0, -6841.67, -6616.67, -6633.33, -6766.67, -6825.0, -6716.67, -6716.67, -6766.67, -6741.67, -6708.33, -6758.33, -6750.0, -6750.0, -6758.33, -6775.0, -6750.0, -6766.67, -6775.0, -6783.33, -6791.67, -6800.0, -6800.0, -6791.67, -6800.0, -6783.33, -6775.0, -6766.67, -6750.0, -6766.67, -6741.67, -6766.67, -6750.0, -6741.67, -6741.67, -6750.0, -6741.67, -6750.0, -6775.0, -6758.33, -6741.67, -6750.0, -6750.0, -6733.33, -6733.33, -6758.33, -6741.67, -6750.0, -6741.67, -6741.67, -6733.33, -6741.67, -6725.0, -6716.67, -6716.67, -6733.33, -6716.67, -6716.67, -6750.0, -6716.67, -6741.67, -6750.0, -6716.67, -6733.33, -6708.33, -6708.33, -6725.0, -6733.33, -6716.67, -6725.0, -6716.67, -6716.67, -6716.67, -6725.0, -6725.0, -6725.0, -6716.67, -6716.67, -6666.67, -6758.33, -6800.0, -6741.67, -6791.67, -6758.33, -6733.33, -6733.33, -6733.33, -6766.67, -6741.67, -6758.33, -6741.67, -6700.0, -6766.67, -6733.33, -6733.33, -6733.33, -6725.0, -6725.0, -6741.67, -6716.67, -6725.0, -6733.33, -6750.0, -6741.67, -6758.33, -6783.33, -6766.67, -6783.33, -6766.67, -6758.33, -6816.67, -6758.33, -6750.0, -6766.67, -6750.0, -6741.67, -6750.0, -6725.0, -6733.33, -6716.67, -6700.0, -6716.67, -6700.0, -6700.0, -6708.33, -6708.33, -6708.33, -6708.33, -6700.0, -6708.33, -6700.0, -6708.33, -6716.67, -6691.67, -6725.0, -6725.0, -6733.33, -6725.0, -6716.67, -6716.67, -6725.0, -6708.33, -6691.67, -6700.0, -6716.67, -6691.67, -6675.0, -6700.0, -6683.33, -6700.0, -6691.67, -6658.33], [-6708.33, -6716.67, -6691.67, -6708.33, -6700.0, -6691.67, -6675.0, -6675.0, -6675.0, -6675.0, -6683.33, -6691.67, -6700.0, -6691.67, -6700.0, -6733.33, -6700.0, -6741.67, -6741.67, -6725.0, -6733.33, -6741.67, -6733.33, -6733.33, -6750.0, -6716.67, -6733.33, -6716.67, -6708.33, -6725.0, -6708.33, -6716.67, -6708.33, -6708.33, -6783.33, -6933.33, -7175.0, -7625.0, -7875.0, -8133.33, -8333.33, -8725.0, -9008.33, -9058.33, -8966.67, -8000.0, -8616.67, -8466.67, -8041.67, -7666.67, -7550.0, -7358.33, -7275.0, -7091.67, -7075.0, -7116.67, -7116.67, -7100.0, -7075.0, -7075.0, -7033.33, -7050.0, -7041.67, -7058.33, -7050.0, -7033.33, -7041.67, -7083.33, -7050.0, -7058.33, -7041.67, -7075.0, -7141.67, -7058.33, -6983.33, -6733.33, -6458.33, -6550.0, -6541.67, -6616.67, -6725.0, -6908.33, -6875.0, -6883.33, -6816.67, -6983.33, -6916.67, -6950.0, -6825.0, -6875.0, -7033.33, -6925.0, -6850.0, -6925.0, -6958.33, -6916.67, -6908.33, -6883.33, -6908.33, -6916.67, -6900.0, -6875.0, -6925.0, -6875.0, -6908.33, -6858.33, -6850.0, -6925.0, -6858.33, -6841.67, -6866.67, -6858.33, -6875.0, -6841.67, -6883.33, -6850.0, -6908.33, -6925.0, -6875.0, -6866.67, -6841.67, -6941.67, -6875.0, -6866.67, -6883.33, -6875.0, -6900.0, -6891.67, -6908.33, -6900.0, -6875.0, -6908.33, -6900.0, -6883.33, -6908.33, -6900.0, -6925.0, -6891.67, -6933.33, -6941.67, -6908.33, -6925.0, -6916.67, -6908.33, -6908.33, -6916.67, -6900.0, -6908.33, -6900.0, -6933.33, -6933.33, -6908.33, -6958.33, -6916.67, -6916.67, -6908.33, -6891.67, -6883.33, -6908.33, -6891.67, -6883.33, -6891.67, -6908.33, -6891.67, -6900.0, -6891.67, -6841.67, -6850.0, -6866.67, -6883.33, -6841.67, -6875.0, -6875.0, -6908.33, -6916.67, -6891.67, -6916.67, -6833.33, -6866.67, -6766.67, -6833.33, -6833.33, -6775.0, -6866.67, -6816.67, -6833.33, -6833.33, -6800.0, -6841.67, -6833.33, -6816.67, -6825.0, -6816.67, -6775.0, -6825.0, -6766.67, -6758.33, -6808.33, -6791.67, -6783.33], [-6825.0, -6758.33, -6758.33, -6791.67, -6758.33, -6775.0, -6758.33, -6766.67, -6750.0, -6758.33, -6775.0, -6758.33, -6758.33, -6775.0, -6775.0, -6783.33, -6791.67, -6808.33, -6766.67, -6791.67, -6758.33, -6783.33, -6741.67, -6766.67, -6758.33, -6783.33, -6800.0, -6783.33, -6783.33, -6791.67, -6758.33, -6800.0, -6783.33, -6783.33, -6783.33, -6791.67, -6791.67, -6783.33, -6800.0, -6766.67, -6800.0, -6783.33, -6783.33, -6791.67, -6783.33, -6783.33, -6775.0, -6775.0, -6766.67, -6791.67, -6766.67, -6758.33, -6766.67, -6758.33, -6758.33, -6758.33, -6741.67, -6741.67, -6725.0, -6725.0, -6750.0, -6750.0, -6741.67, -6758.33, -6758.33, -6775.0, -6800.0, -6791.67, -6825.0, -6833.33, -6800.0, -6808.33, -6791.67, -6775.0, -6783.33, -6783.33, -6766.67, -6783.33, -6758.33, -6758.33, -6816.67, -6750.0, -6775.0, -6766.67, -6766.67, -6775.0, -6708.33, -6791.67, -6816.67, -6775.0, -6783.33, -6850.0, -6808.33, -6791.67, -6766.67, -6791.67, -6841.67, -6783.33, -6766.67, -6800.0, -6791.67, -6800.0, -6800.0, -6808.33, -6808.33, -6841.67, -6800.0, -6825.0, -6850.0, -6791.67, -6841.67, -6791.67, -6825.0, -6825.0, -6783.33, -6783.33, -6775.0, -6791.67, -6775.0, -6758.33, -6733.33, -6750.0, -6783.33, -6758.33, -6741.67, -6733.33, -6741.67, -6766.67, -6766.67, -6766.67, -6791.67, -6800.0, -6783.33, -6808.33, -6791.67, -6791.67, -6775.0, -6775.0, -6783.33, -6783.33, -6750.0, -6758.33, -6750.0, -6725.0, -6733.33, -6733.33, -6733.33, -6741.67, -6733.33, -6758.33, -6741.67, -6758.33, -6766.67, -6766.67, -6766.67, -6783.33, -6791.67, -6808.33, -6791.67, -6800.0, -6783.33, -6791.67, -6791.67, -6791.67, -6783.33, -6775.0, -6766.67, -6758.33, -6741.67, -6758.33, -6750.0, -6750.0, -6741.67, -6741.67, -6733.33, -6733.33, -6725.0, -6733.33, -6750.0, -6700.0, -6675.0, -6675.0, -6658.33, -6616.67, -6641.67, -6641.67, -6616.67, -6658.33, -6691.67, -6700.0, -6700.0, -6783.33, -6750.0, -6833.33, -6816.67, -6791.67, -6766.67, -6750.0, -6808.33, -6758.33], [-6633.33, -6600.0, -6591.67, -6641.67, -6675.0, -6725.0, -6791.67, -6841.67, -6908.33, -6991.67, -7158.33, -7458.33, -7633.33, -7741.67, -7741.67, -7766.67, -7775.0, -7783.33, -7875.0, -7900.0, -7908.33, -7975.0, -7700.0, -7550.0, -7658.33, -7541.67, -7316.67, -7291.67, -7266.67, -7191.67, -7150.0, -7200.0, -7208.33, -7208.33, -7200.0, -7166.67, -7166.67, -7175.0, -7150.0, -7158.33, -7141.67, -7141.67, -7125.0, -7100.0, -7108.33, -7100.0, -7100.0, -7108.33, -7108.33, -7100.0, -7125.0, -7216.67, -7500.0, -7733.33, -7791.67, -7650.0, -7425.0, -7516.67, -7675.0, -7741.67, -8316.67, -8066.67, -8183.33, -8775.0, -7250.0, -7883.33, -8283.33, -7558.33, -7316.67, -7275.0, -6983.33, -7016.67, -6941.67, -6766.67, -6891.67, -6850.0, -6666.67, -6750.0, -6950.0, -6741.67, -6800.0, -6716.67, -6841.67, -6716.67, -6766.67, -6750.0, -6708.33, -6741.67, -6675.0, -6758.33, -6641.67, -6858.33, -6691.67, -6908.33, -6633.33, -6791.67, -6658.33, -6700.0, -6666.67, -6633.33, -6725.0, -6550.0, -6600.0, -6558.33, -6525.0, -6583.33, -6625.0, -6666.67, -6650.0, -6658.33, -6658.33, -6650.0, -6641.67, -6633.33, -6616.67, -6625.0, -6616.67, -6616.67, -6616.67, -6608.33, -6591.67, -6591.67, -6600.0, -6575.0, -6600.0, -6616.67, -6608.33, -6608.33, -6608.33, -6633.33, -6641.67, -6650.0, -6633.33, -6650.0, -6666.67, -6641.67, -6641.67, -6633.33, -6625.0, -6625.0, -6608.33, -6583.33, -6583.33, -6575.0, -6575.0, -6575.0, -6575.0, -6583.33, -6583.33, -6591.67, -6600.0, -6591.67, -6608.33, -6608.33, -6608.33, -6608.33, -6608.33, -6591.67, -6625.0, -6575.0, -6591.67, -6575.0, -6558.33, -6566.67, -6575.0, -6550.0, -6550.0, -6575.0, -6550.0, -6558.33, -6550.0, -6558.33, -6558.33, -6550.0, -6541.67, -6558.33, -6525.0, -6550.0, -6533.33, -6541.67, -6541.67, -6541.67, -6533.33, -6533.33, -6541.67, -6541.67, -6541.67, -6541.67, -6558.33, -6550.0, -6558.33, -6566.67, -6558.33, -6575.0, -6575.0, -6600.0, -6558.33, -6541.67, -6533.33, -6525.0], [-6516.67, -6533.33, -6516.67, -6516.67, -6508.33, -6508.33, -6516.67, -6516.67, -6508.33, -6516.67, -6508.33, -6508.33, -6516.67, -6500.0, -6525.0, -6508.33, -6508.33, -6533.33, -6525.0, -6541.67, -6541.67, -6550.0, -6558.33, -6541.67, -6558.33, -6533.33, -6525.0, -6516.67, -6508.33, -6516.67, -6500.0, -6491.67, -6491.67, -6491.67, -6491.67, -6500.0, -6500.0, -6500.0, -6508.33, -6508.33, -6525.0, -6525.0, -6508.33, -6516.67, -6541.67, -6525.0, -6525.0, -6516.67, -6525.0, -6508.33, -6500.0, -6500.0, -6500.0, -6491.67, -6491.67, -6483.33, -6483.33, -6475.0, -6483.33, -6491.67, -6491.67, -6508.33, -6516.67, -6541.67, -6541.67, -6566.67, -6591.67, -6650.0, -6658.33, -6633.33, -6625.0, -6658.33, -6575.0, -6608.33, -6566.67, -6583.33, -6591.67, -6600.0, -6608.33, -6616.67, -6608.33, -6625.0, -6616.67, -6625.0, -6633.33, -6608.33, -6625.0, -6591.67, -6608.33, -6633.33, -6591.67, -6616.67, -6625.0, -6616.67, -6608.33, -6583.33, -6458.33, -6108.33, -5883.33, -5691.67, -5608.33, -5433.33, -5258.33, -5208.33, -5358.33, -5308.33, -5291.67, -5358.33, -5641.67, -5800.0, -5608.33, -5608.33, -5741.67, -5841.67, -6100.0, -6258.33, -6391.67, -6441.67, -6500.0, -6508.33, -6516.67, -6541.67, -6558.33, -6583.33, -6600.0, -6658.33, -6600.0, -6666.67, -6625.0, -6625.0, -6666.67, -6633.33, -6666.67, -6650.0, -6675.0, -6666.67, -6675.0, -6675.0, -6641.67, -6675.0, -6641.67, -6650.0, -6633.33, -6633.33, -6641.67, -6616.67, -6625.0, -6625.0, -6608.33, -6625.0, -6633.33, -6633.33, -6683.33, -6658.33, -6733.33, -6716.67, -6800.0, -6733.33, -6758.33, -6725.0, -6733.33, -6675.0, -6683.33, -6683.33, -6683.33, -6683.33, -6691.67, -6700.0, -6708.33, -6683.33, -6700.0, -6700.0, -6675.0, -6700.0, -6683.33, -6675.0, -6691.67, -6641.67, -6733.33, -6658.33, -6666.67, -6675.0, -6691.67, -6683.33, -6683.33, -6650.0, -6658.33, -6666.67, -6658.33, -6675.0, -6650.0, -6658.33, -6666.67, -6650.0, -6666.67, -6641.67, -6658.33, -6641.67, -6633.33, -6633.33], [-6633.33, -6641.67, -6625.0, -6616.67, -6616.67, -6625.0, -6616.67, -6625.0, -6608.33, -6608.33, -6625.0, -6608.33, -6608.33, -6608.33, -6608.33, -6608.33, -6600.0, -6608.33, -6600.0, -6600.0, -6616.67, -6741.67, -6866.67, -6916.67, -7033.33, -7150.0, -7275.0, -7266.67, -7333.33, -7350.0, -7425.0, -7341.67, -6975.0, -6883.33, -6891.67, -6666.67, -6475.0, -6466.67, -6425.0, -6408.33, -6516.67, -6683.33, -6658.33, -6675.0, -6675.0, -6675.0, -6641.67, -6625.0, -6600.0, -6616.67, -6658.33, -6658.33, -6641.67, -6666.67, -6650.0, -6666.67, -6675.0, -6683.33, -6691.67, -6683.33, -6691.67, -6691.67, -6691.67, -6683.33, -6675.0, -6683.33, -6658.33, -6675.0, -6666.67, -6675.0, -6658.33, -6666.67, -6666.67, -6658.33, -6658.33, -6675.0, -6658.33, -6650.0, -6675.0, -6666.67, -6658.33, -6675.0, -6666.67, -6658.33, -6666.67, -6658.33, -6658.33, -6650.0, -6650.0, -6650.0, -6650.0, -6658.33, -6658.33, -6658.33, -6683.33, -6675.0, -6666.67, -6683.33, -6700.0, -6658.33, -6700.0, -6691.67, -6683.33, -6683.33, -6675.0, -6675.0, -6658.33, -6666.67, -6666.67, -6650.0, -6650.0, -6641.67, -6625.0, -6641.67, -6633.33, -6641.67, -6641.67, -6633.33, -6641.67, -6641.67, -6650.0, -6650.0, -6650.0, -6666.67, -6666.67, -6666.67, -6683.33, -6675.0, -6683.33, -6675.0, -6666.67, -6666.67, -6666.67, -6658.33, -6650.0, -6666.67, -6641.67, -6650.0, -6641.67, -6633.33, -6641.67, -6633.33, -6641.67, -6641.67, -6633.33, -6658.33, -6641.67, -6650.0, -6650.0, -6625.0, -6616.67, -6591.67, -6566.67, -6566.67, -6516.67, -6491.67, -6516.67, -6566.67, -6608.33, -6641.67, -6408.33, -6425.0, -6541.67, -6366.67, -6491.67, -6591.67, -6733.33, -6775.0, -6783.33, -6758.33, -6741.67, -6708.33, -6691.67, -6675.0, -6675.0, -6650.0, -6650.0, -6666.67, -6650.0, -6641.67, -6650.0, -6650.0, -6641.67, -6650.0, -6658.33, -6658.33, -6666.67, -6666.67, -6675.0, -6675.0, -6700.0, -6675.0, -6683.33, -6683.33, -6675.0, -6675.0, -6666.67, -6666.67, -6650.0, -6641.67], [-6650.0, -6641.67, -6641.67, -6641.67, -6633.33, -6633.33, -6633.33, -6633.33, -6633.33, -6616.67, -6633.33, -6633.33, -6625.0, -6633.33, -6633.33, -6625.0, -6633.33, -6641.67, -6658.33, -6641.67, -6658.33, -6658.33, -6658.33, -6658.33, -6650.0, -6650.0, -6641.67, -6650.0, -6658.33, -6641.67, -6641.67, -6633.33, -6633.33, -6633.33, -6633.33, -6633.33, -6625.0, -6625.0, -6616.67, -6625.0, -6616.67, -6625.0, -6625.0, -6625.0, -6641.67, -6641.67, -6641.67, -6641.67, -6641.67, -6650.0, -6641.67, -6666.67, -6658.33, -6658.33, -6658.33, -6650.0, -6658.33, -6650.0, -6641.67, -6625.0, -6641.67, -6625.0, -6625.0, -6633.33, -6625.0, -6633.33, -6616.67, -6625.0, -6616.67, -6616.67, -6616.67, -6616.67, -6625.0, -6616.67, -6625.0, -6633.33, -6633.33, -6641.67, -6633.33, -6658.33, -6650.0, -6666.67, -6658.33, -6650.0, -6658.33, -6666.67, -6616.67, -6591.67, -6608.33, -6525.0, -6600.0, -6566.67, -6583.33, -6575.0, -6583.33, -6500.0, -6525.0, -6516.67, -6466.67, -6466.67, -6425.0, -6433.33, -6416.67, -6408.33, -6466.67, -6533.33, -6583.33, -6558.33, -6591.67, -6575.0, -6583.33, -6608.33, -6608.33, -6608.33, -6616.67, -6616.67, -6608.33, -6616.67, -6583.33, -6600.0, -6600.0, -6591.67, -6600.0, -6583.33, -6600.0, -6583.33, -6583.33, -6575.0, -6583.33, -6575.0, -6575.0, -6575.0, -6575.0, -6566.67, -6575.0, -6575.0, -6558.33, -6583.33, -6566.67, -6575.0, -6575.0, -6566.67, -6583.33, -6575.0, -6566.67, -6591.67, -6575.0, -6575.0, -6600.0, -6575.0, -6575.0, -6600.0, -6583.33, -6583.33, -6600.0, -6591.67, -6608.33, -6575.0, -6600.0, -6575.0, -6575.0, -6591.67, -6591.67, -6608.33, -6625.0, -6658.33, -6708.33, -6925.0, -7166.67, -7508.33, -7491.67, -7283.33, -7100.0, -6908.33, -6541.67, -5858.33, -5033.33, -4775.0, -5775.0, -8441.67, -10091.67, -10783.33, -10975.0, -11191.67, -11458.33, -11408.33, -11408.33, -11991.67, -12108.33, -12125.0, -12483.33, -12341.67, -12341.67, -12241.67, -12375.0, -12491.67, -12558.33, -12700.0, -12891.67, -13058.33], [-13016.67, -12958.33, -13108.33, -13433.33, -13341.67, -13458.33, -13575.0, -13616.67, -13541.67, -13583.33, -13408.33, -13375.0, -13283.33, -13425.0, -13391.67, -13341.67, -13358.33, -13441.67, -13408.33, -13408.33, -13508.33, -13508.33, -13516.67, -13500.0, -13466.67, -13391.67, -13400.0, -13358.33, -13400.0, -13366.67, -13333.33, -13291.67, -13283.33, -13150.0, -13025.0, -12925.0, -12850.0, -12941.67, -12816.67, -12891.67, -12966.67, -12866.67, -12891.67, -12941.67, -13041.67, -13025.0, -12991.67, -12991.67, -13058.33, -13016.67, -12733.33, -13066.67, -12683.33, -12741.67, -12958.33, -12858.33, -12958.33, -12933.33, -12966.67, -13025.0, -12991.67, -12983.33, -13058.33, -13000.0, -13083.33, -13050.0, -13083.33, -13033.33, -13091.67, -13083.33, -13116.67, -13083.33, -13125.0, -13125.0, -13175.0, -13183.33, -13208.33, -13191.67, -13308.33, -13258.33, -13233.33, -13241.67, -13266.67, -13308.33, -13283.33, -13316.67, -13300.0, -13308.33, -13283.33, -13316.67, -13341.67, -13333.33, -13341.67, -13383.33, -13366.67, -13366.67, -13425.0, -13400.0, -13408.33, -13391.67, -13400.0, -13433.33, -13400.0, -13450.0, -13333.33, -13316.67, -13308.33, -13191.67, -13016.67, -13025.0, -13016.67, -13025.0, -13058.33, -13191.67, -13158.33, -13125.0, -13158.33, -13158.33, -13175.0, -13158.33, -13175.0, -13175.0, -13191.67, -13208.33, -13208.33, -13200.0, -13200.0, -13225.0, -13225.0, -13233.33, -13233.33, -13250.0, -13283.33, -13291.67, -13283.33, -13300.0, -13300.0, -13300.0, -13308.33, -13316.67, -13308.33, -13316.67, -13308.33, -13341.67, -13325.0, -13341.67, -13316.67, -13316.67, -13350.0, -13358.33, -13341.67, -13341.67, -13333.33, -13350.0, -13366.67, -13375.0, -13375.0, -13358.33, -13383.33, -13366.67, -13375.0, -13358.33, -13408.33, -13383.33, -13425.0, -13375.0, -13400.0, -13400.0, -13425.0, -13391.67, -13416.67, -13400.0, -13391.67, -13391.67, -13408.33, -13416.67, -13400.0, -13400.0, -13408.33, -13416.67, -13375.0, -13408.33, -13400.0, -13400.0, -13416.67, -13416.67, -13425.0, -13433.33, -13416.67, -13408.33, -13400.0, -13425.0, -13425.0, -13416.67, -13425.0, -13400.0, -13416.67, -13425.0, -13433.33, -13441.67], [-13541.67, -13541.67, -13541.67, -13583.33, -13566.67, -13566.67, -13550.0, -13558.33, -13575.0, -13600.0, -13575.0, -13583.33, -13591.67, -13600.0, -13591.67, -13600.0, -13600.0, -13600.0, -13550.0, -13600.0, -13558.33, -13566.67, -13583.33, -13558.33, -13558.33, -13566.67, -13558.33, -13550.0, -13550.0, -13558.33, -13558.33, -13558.33, -13566.67, -13558.33, -13541.67, -13550.0, -13533.33, -13541.67, -13525.0, -13541.67, -13558.33, -13558.33, -13525.0, -13500.0, -13541.67, -13533.33, -13525.0, -13525.0, -13516.67, -13533.33, -13533.33, -13516.67, -13516.67, -13525.0, -13541.67, -13525.0, -13516.67, -13525.0, -13525.0, -13516.67, -13533.33, -13516.67, -13525.0, -13516.67, -13508.33, -13500.0, -13491.67, -13500.0, -13508.33, -13500.0, -13483.33, -13500.0, -13483.33, -13508.33, -13508.33, -13491.67, -13483.33, -13500.0, -13491.67, -13508.33, -13500.0, -13475.0, -13458.33, -13416.67, -13450.0, -13375.0, -13366.67, -13383.33, -13358.33, -13391.67, -13375.0, -13333.33, -13341.67, -13350.0, -13300.0, -13325.0, -13316.67, -13341.67, -13300.0, -13275.0, -13275.0, -13283.33, -13258.33, -13291.67, -13258.33, -13225.0, -13233.33, -13258.33, -13216.67, -13216.67, -13183.33, -13216.67, -13191.67, -13216.67, -13183.33, -13158.33, -13200.0, -13158.33, -13191.67, -13108.33, -13191.67, -13158.33, -13150.0, -13158.33, -13158.33, -13133.33, -13116.67, -13108.33, -13125.0, -13083.33, -13108.33, -13008.33, -13108.33, -13091.67, -13066.67, -13075.0, -13083.33, -13083.33, -13100.0, -13050.0, -13075.0, -13050.0, -13050.0, -13041.67, -13041.67, -13058.33, -13041.67, -13058.33, -13033.33, -13016.67, -13033.33, -13025.0, -13050.0, -13016.67, -13025.0, -13008.33, -13016.67, -13000.0, -13016.67, -12991.67, -13000.0, -12983.33, -12966.67, -12991.67, -12991.67, -12975.0, -12983.33, -12966.67, -12975.0, -12966.67, -12958.33, -12966.67, -12950.0, -12941.67, -12950.0, -12941.67, -12933.33, -12958.33, -12950.0, -12950.0, -12950.0, -12941.67, -12933.33, -12925.0, -12916.67, -12916.67, -12916.67, -12908.33, -12916.67, -12900.0, -12908.33, -12900.0, -12916.67, -12883.33, -12891.67, -12891.67, -12900.0, -12883.33, -12883.33, -12875.0], [-12816.67, -12816.67, -12808.33, -12816.67, -12816.67, -12800.0, -12791.67, -12808.33, -12800.0, -12800.0, -12800.0, -12791.67, -12791.67, -12800.0, -12808.33, -12775.0, -12800.0, -12775.0, -12758.33, -12766.67, -12750.0, -12775.0, -12766.67, -12766.67, -12766.67, -12758.33, -12750.0, -12758.33, -12733.33, -12741.67, -12733.33, -12716.67, -12733.33, -12708.33, -12716.67, -12725.0, -12733.33, -12708.33, -12741.67, -12758.33, -12750.0, -12750.0, -12758.33, -12758.33, -12733.33, -12750.0, -12750.0, -12758.33, -12750.0, -12758.33, -12750.0, -12741.67, -12741.67, -12750.0, -12791.67, -12741.67, -12800.0, -12775.0, -12800.0, -12800.0, -12783.33, -12750.0, -12783.33, -12783.33, -12741.67, -12766.67, -12750.0, -12741.67, -12741.67, -12725.0, -12725.0, -12750.0, -12733.33, -12733.33, -12741.67, -12733.33, -12716.67, -12725.0, -12725.0, -12733.33, -12716.67, -12708.33, -12725.0, -12708.33, -12708.33, -12708.33, -12708.33, -12716.67, -12725.0, -12708.33, -12716.67, -12700.0, -12700.0, -12691.67, -12700.0, -12683.33, -12691.67, -12691.67, -12708.33, -12666.67, -12691.67, -12675.0, -12675.0, -12683.33, -12666.67, -12675.0, -12675.0, -12658.33, -12683.33, -12666.67, -12675.0, -12658.33, -12650.0, -12641.67, -12641.67, -12641.67, -12650.0, -12650.0, -12633.33, -12650.0, -12650.0, -12600.0, -12608.33, -12616.67, -12608.33, -12608.33, -12608.33, -12608.33, -12608.33, -12608.33, -12600.0, -12591.67, -12600.0, -12591.67, -12591.67, -12591.67, -12591.67, -12583.33, -12583.33, -12575.0, -12583.33, -12575.0, -12575.0, -12575.0, -12566.67, -12575.0, -12583.33, -12575.0, -12550.0, -12575.0, -12558.33, -12558.33, -12550.0, -12550.0, -12600.0, -12566.67, -12541.67, -12541.67, -12541.67, -12533.33, -12550.0, -12541.67, -12541.67, -12525.0, -12525.0, -12533.33, -12558.33, -12533.33, -12533.33, -12525.0, -12533.33, -12516.67, -12500.0, -12525.0, -12500.0, -12525.0, -12525.0, -12516.67, -12533.33, -12508.33, -12508.33, -12491.67, -12491.67, -12483.33, -12491.67, -12500.0, -12491.67, -12500.0, -12500.0, -12475.0, -12525.0, -12491.67, -12508.33, -12525.0, -12516.67, -12491.67, -12458.33, -12458.33, -12491.67, -12458.33], [-12408.33, -12416.67, -12441.67, -12408.33, -12400.0, -12400.0, -12425.0, -12366.67, -12341.67, -12341.67, -12325.0, -12316.67, -12325.0, -12325.0, -12316.67, -12325.0, -12325.0, -12316.67, -12308.33, -12291.67, -12308.33, -12300.0, -12291.67, -12266.67, -12291.67, -12291.67, -12291.67, -12291.67, -12250.0, -12241.67, -12250.0, -12241.67, -12225.0, -12241.67, -12225.0, -12233.33, -12233.33, -12216.67, -12258.33, -12225.0, -12225.0, -12233.33, -12233.33, -12233.33, -12225.0, -12225.0, -12208.33, -12225.0, -12191.67, -12225.0, -12208.33, -12225.0, -12208.33, -12200.0, -12208.33, -12208.33, -12191.67, -12200.0, -12200.0, -12200.0, -12191.67, -12191.67, -12200.0, -12183.33, -12183.33, -12183.33, -12183.33, -12166.67, -12166.67, -12175.0, -12175.0, -12166.67, -12166.67, -12158.33, -12166.67, -12166.67, -12166.67, -12150.0, -12158.33, -12166.67, -12166.67, -12166.67, -12150.0, -12141.67, -12150.0, -12158.33, -12133.33, -12141.67, -12141.67, -12141.67, -12141.67, -12133.33, -12133.33, -12125.0, -12141.67, -12125.0, -12116.67, -12133.33, -12125.0, -12125.0, -12125.0, -12125.0, -12116.67, -12116.67, -12125.0, -12116.67, -12108.33, -12116.67, -12116.67, -12100.0, -12116.67, -12116.67, -12100.0, -12100.0, -12108.33, -12108.33, -12091.67, -12100.0, -12100.0, -12100.0, -12100.0, -12083.33, -12091.67, -12083.33, -12108.33, -12100.0, -12075.0, -12091.67, -12075.0, -12100.0, -12075.0, -12075.0, -12058.33, -12100.0, -12041.67, -12041.67, -12025.0, -12025.0, -12033.33, -12016.67, -12008.33, -11991.67, -12000.0, -11975.0, -11991.67, -11983.33, -11966.67, -11950.0, -11950.0, -11958.33, -11950.0, -11933.33, -11925.0, -11916.67, -11916.67, -11900.0, -11791.67, -11766.67, -11825.0, -11383.33, -11541.67, -11541.67, -11533.33, -11358.33, -11483.33, -11816.67, -11483.33, -11133.33, -11066.67, -9958.33, -8041.67, -6541.67, -6333.33, -6066.67, -5541.67, -4550.0, -3425.0, -2908.33, -2616.67, -2658.33, -2408.33, -2658.33, -2291.67, -2508.33, -2525.0, -1933.33, -2000.0, -2300.0, -2083.33, -1941.67, -2316.67, -2075.0, -1825.0, -1025.0, -866.67, -166.67, 33.33, -33.33, 216.67, 133.33], [-9616.67, -9950.0, -9500.0, -9808.33, -9766.67, -9841.67, -9875.0, -9883.33, -9958.33, -10041.67, -10108.33, -10183.33, -10275.0, -10300.0, -10308.33, -10325.0, -10333.33, -10400.0, -10408.33, -10491.67, -10208.33, -10475.0, -10775.0, -10725.0, -10766.67, -10425.0, -10675.0, -9925.0, -10666.67, -10483.33, -9525.0, -10616.67, -10458.33, -10575.0, -10608.33, -10541.67, -10516.67, -10500.0, -10525.0, -10500.0, -10550.0, -10416.67, -10450.0, -10350.0, -10275.0, -10325.0, -10466.67, -10250.0, -10383.33, -10341.67, -10308.33, -10275.0, -10233.33, -10225.0, -10116.67, -10125.0, -10108.33, -10075.0, -10050.0, -10033.33, -10025.0, -10016.67, -10008.33, -10000.0, -10008.33, -9991.67, -9983.33, -10000.0, -9991.67, -9983.33, -9958.33, -9958.33, -9958.33, -9916.67, -9933.33, -9933.33, -9908.33, -9900.0, -9908.33, -9891.67, -9891.67, -9883.33, -9866.67, -9858.33, -9858.33, -9858.33, -9841.67, -9833.33, -9833.33, -9833.33, -9808.33, -9808.33, -9800.0, -9800.0, -9775.0, -9791.67, -9758.33, -9791.67, -9758.33, -9750.0, -9741.67, -9750.0, -9758.33, -9725.0, -9716.67, -9716.67, -9708.33, -9700.0, -9725.0, -9716.67, -9691.67, -9675.0, -9683.33, -9658.33, -9691.67, -9658.33, -9541.67, -9666.67, -9650.0, -9650.0, -9675.0, -9650.0, -9633.33, -9633.33, -9625.0, -9600.0, -9616.67, -9575.0, -9600.0, -9591.67, -9575.0, -9566.67, -9591.67, -9558.33, -9516.67, -9508.33, -9516.67, -9516.67, -9525.0, -9516.67, -9441.67, -9516.67, -9483.33, -9475.0, -9483.33, -9483.33, -9475.0, -9450.0, -9466.67, -9450.0, -9458.33, -9433.33, -9441.67, -9450.0, -9433.33, -9400.0, -9441.67, -9408.33, -9391.67, -9383.33, -9375.0, -9375.0, -9375.0, -9375.0, -9375.0, -9391.67, -9366.67, -9366.67, -9366.67, -9358.33, -9358.33, -9358.33, -9350.0, -9333.33, -9333.33, -9333.33, -9325.0, -9325.0, -9325.0, -9325.0, -9316.67, -9316.67, -9325.0, -9316.67, -9316.67, -9308.33, -9291.67, -9300.0, -9291.67, -9283.33, -9291.67, -9275.0, -9275.0, -9283.33, -9258.33, -9475.0, -9258.33, -9275.0, -9258.33, -9233.33], [-9150.0, -9166.67, -9150.0, -9133.33, -9116.67, -9158.33, -9133.33, -9133.33, -9116.67, -9108.33, -9108.33, -9108.33, -9100.0, -9125.0, -9091.67, -9108.33, -9083.33, -9091.67, -9066.67, -9075.0, -9041.67, -9066.67, -9066.67, -9075.0, -9058.33, -9041.67, -9033.33, -9083.33, -8991.67, -9075.0, -9025.0, -9016.67, -9025.0, -8983.33, -8950.0, -8983.33, -9033.33, -9116.67, -9108.33, -8933.33, -8875.0, -8950.0, -8983.33, -8983.33, -8916.67, -8883.33, -9000.0, -8925.0, -8841.67, -8858.33, -8941.67, -8933.33, -8858.33, -8741.67, -8541.67, -8750.0, -8683.33, -8750.0, -8708.33, -8708.33, -8650.0, -8675.0, -8691.67, -8658.33, -8650.0, -8633.33, -8641.67, -8608.33, -8633.33, -8641.67, -8633.33, -8641.67, -8616.67, -8608.33, -8600.0, -8591.67, -8608.33, -8583.33, -8583.33, -8583.33, -8500.0, -8591.67, -8566.67, -8575.0, -8550.0, -8566.67, -8558.33, -8566.67, -8541.67, -8550.0, -8541.67, -8533.33, -8558.33, -8533.33, -8541.67, -8533.33, -8516.67, -8525.0, -8500.0, -8525.0, -8508.33, -8500.0, -8508.33, -8491.67, -8500.0, -8491.67, -8500.0, -8491.67, -8491.67, -8483.33, -8475.0, -8483.33, -8483.33, -8466.67, -8458.33, -7975.0, -8475.0, -8458.33, -8458.33, -8450.0, -8475.0, -8441.67, -8416.67, -8475.0, -8433.33, -8416.67, -8425.0, -8433.33, -8416.67, -8416.67, -8408.33, -8400.0, -8400.0, -8416.67, -8391.67, -8391.67, -8391.67, -8400.0, -8383.33, -8391.67, -8391.67, -8391.67, -8383.33, -8375.0, -8366.67, -8375.0, -8375.0, -8358.33, -8366.67, -8375.0, -8350.0, -8358.33, -8366.67, -8358.33, -8350.0, -8358.33, -8350.0, -8341.67, -8358.33, -8341.67, -8333.33, -8341.67, -8325.0, -8341.67, -8333.33, -8333.33, -8325.0, -8333.33, -8308.33, -8333.33, -8325.0, -8325.0, -8333.33, -8308.33, -8325.0, -8316.67, -8308.33, -8300.0, -8316.67, -8308.33, -8375.0, -8300.0, -8283.33, -8283.33, -8300.0, -8291.67, -8291.67, -8291.67, -8283.33, -7933.33, -7908.33, -8283.33, -8275.0, -8258.33, -8266.67, -8258.33, -8266.67, -8250.0, -8250.0, -8250.0], [-8225.0, -8225.0, -8200.0, -8191.67, -8216.67, -8225.0, -8200.0, -8200.0, -8200.0, -8183.33, -8191.67, -8191.67, -8191.67, -8191.67, -8208.33, -8183.33, -8200.0, -8200.0, -8191.67, -8183.33, -8183.33, -8191.67, -8175.0, -8166.67, -8183.33, -8175.0, -8166.67, -8150.0, -8166.67, -8166.67, -8166.67, -8158.33, -8166.67, -8166.67, -8166.67, -8150.0, -8158.33, -8166.67, -8133.33, -8158.33, -8141.67, -8141.67, -8141.67, -8141.67, -8125.0, -8150.0, -8175.0, -8141.67, -8141.67, -8125.0, -8133.33, -8125.0, -8141.67, -8133.33, -8125.0, -8133.33, -8133.33, -8125.0, -8125.0, -8116.67, -8108.33, -8133.33, -8116.67, -8116.67, -8116.67, -8116.67, -8108.33, -8116.67, -8108.33, -8108.33, -8100.0, -8100.0, -8100.0, -8091.67, -8100.0, -8091.67, -8100.0, -8100.0, -8108.33, -8100.0, -8100.0, -8091.67, -8091.67, -8091.67, -8091.67, -8091.67, -8083.33, -8083.33, -8083.33, -8091.67, -8083.33, -8075.0, -8083.33, -8075.0, -8075.0, -8075.0, -8075.0, -8075.0, -8075.0, -8066.67, -8058.33, -8075.0, -8075.0, -8058.33, -8066.67, -8058.33, -8066.67, -8050.0, -8066.67, -8058.33, -8050.0, -8050.0, -8058.33, -8050.0, -8050.0, -8058.33, -8050.0, -8041.67, -8050.0, -8041.67, -8033.33, -8041.67, -8041.67, -8058.33, -8033.33, -8041.67, -8033.33, -8025.0, -8025.0, -8008.33, -8025.0, -8033.33, -8033.33, -8050.0, -7225.0, -8033.33, -8008.33, -8000.0, -8016.67, -8016.67, -7991.67, -8008.33, -8000.0, -8008.33, -8000.0, -8008.33, -8008.33, -7975.0, -7991.67, -8008.33, -8000.0, -7983.33, -7991.67, -8000.0, -7991.67, -7983.33, -7991.67, -7958.33, -7991.67, -7983.33, -7983.33, -7991.67, -8008.33, -7975.0, -7983.33, -7975.0, -7983.33, -7983.33, -7983.33, -7966.67, -7975.0, -7975.0, -7991.67, -7975.0, -7966.67, -7958.33, -7966.67, -7966.67, -7966.67, -7958.33, -7966.67, -7966.67, -7950.0, -7966.67, -7958.33, -7941.67, -7966.67, -7966.67, -7958.33, -7958.33, -7950.0, -7966.67, -7950.0, -7941.67, -7950.0, -7950.0, -7941.67, -7941.67, -7950.0, -7941.67], [-7925.0, -7916.67, -7916.67, -7908.33, -7933.33, -7925.0, -7925.0, -7908.33, -7933.33, -7916.67, -7908.33, -7925.0, -7916.67, -7916.67, -7916.67, -7908.33, -7916.67, -7908.33, -7908.33, -7908.33, -7900.0, -7908.33, -7908.33, -7925.0, -7916.67, -7900.0, -7883.33, -7900.0, -7941.67, -7900.0, -7891.67, -7891.67, -7900.0, -7916.67, -7891.67, -7908.33, -7900.0, -7916.67, -7908.33, -7875.0, -7925.0, -7950.0, -7900.0, -8091.67, -8300.0, -8150.0, -8141.67, -8033.33, -7991.67, -7991.67, -7908.33, -8108.33, -7983.33, -8158.33, -7958.33, -8208.33, -8000.0, -8041.67, -7991.67, -8033.33, -8008.33, -8033.33, -7975.0, -8000.0, -7958.33, -7975.0, -7950.0, -7983.33, -7925.0, -7941.67, -7925.0, -7950.0, -7925.0, -7933.33, -7941.67, -7908.33, -7925.0, -7916.67, -7925.0, -7916.67, -7916.67, -7908.33, -7891.67, -7916.67, -7925.0, -7900.0, -7900.0, -7891.67, -7900.0, -7891.67, -7900.0, -7883.33, -7891.67, -7875.0, -7883.33, -7875.0, -7891.67, -7883.33, -7875.0, -7883.33, -7883.33, -7875.0, -7883.33, -7883.33, -7858.33, -7883.33, -7858.33, -7875.0, -7858.33, -7866.67, -7850.0, -7841.67, -7866.67, -7866.67, -7850.0, -7858.33, -7883.33, -7850.0, -7866.67, -7858.33, -7850.0, -7858.33, -7858.33, -7841.67, -7866.67, -7850.0, -7825.0, -7850.0, -7733.33, -7950.0, -7891.67, -7833.33, -7833.33, -7133.33, -8066.67, -8291.67, -8358.33, -8866.67, -8866.67, -9216.67, -9733.33, -10216.67, -11058.33, -10858.33, -9733.33, -9316.67, -8866.67, -8550.0, -8516.67, -8483.33, -8525.0, -8358.33, -8400.0, -8408.33, -8358.33, -8541.67, -8333.33, -8416.67, -8325.0, -8500.0, -8358.33, -8000.0, -8350.0, -7975.0, -8341.67, -8300.0, -8891.67, -8300.0, -7116.67, -8308.33, -8300.0, -8283.33, -8325.0, -8283.33, -8325.0, -8266.67, -8283.33, -8258.33, -8241.67, -8325.0, -8308.33, -8216.67, -8241.67, -8183.33, -8183.33, -8241.67, -8108.33, -8125.0, -8133.33, -8108.33, -8108.33, -8050.0, -7708.33, -8016.67, -8008.33, -7991.67, -8416.67, -7991.67, -7983.33, -8000.0], [-7916.67, -7933.33, -7933.33, -7916.67, -7916.67, -7916.67, -7916.67, -7916.67, -7916.67, -7908.33, -7908.33, -7916.67, -7908.33, -7891.67, -7916.67, -7891.67, -7900.0, -7891.67, -7883.33, -7941.67, -7891.67, -7925.0, -7908.33, -7891.67, -7891.67, -7891.67, -7875.0, -7916.67, -7900.0, -7883.33, -7850.0, -7891.67, -7858.33, -7875.0, -7891.67, -7875.0, -7858.33, -7875.0, -7866.67, -7850.0, -7875.0, -7841.67, -7866.67, -7858.33, -7833.33, -7833.33, -7858.33, -7858.33, -7850.0, -7850.0, -7850.0, -7841.67, -7858.33, -7841.67, -7841.67, -7850.0, -7841.67, -7850.0, -7841.67, -7825.0, -7850.0, -7833.33, -7833.33, -7833.33, -7833.33, -7841.67, -7825.0, -7833.33, -7833.33, -7825.0, -7841.67, -7825.0, -7825.0, -7833.33, -7833.33, -7841.67, -7850.0, -7833.33, -7825.0, -7825.0, -7816.67, -7833.33, -7833.33, -7825.0, -7825.0, -7825.0, -7808.33, -7808.33, -7808.33, -7816.67, -7816.67, -7816.67, -7816.67, -7808.33, -7800.0, -7808.33, -7816.67, -7791.67, -7808.33, -7808.33, -7808.33, -7808.33, -7808.33, -7800.0, -7808.33, -7800.0, -7808.33, -7800.0, -7808.33, -7800.0, -7800.0, -7808.33, -7808.33, -7791.67, -7800.0, -7800.0, -7783.33, -7800.0, -7800.0, -7783.33, -7783.33, -7808.33, -7791.67, -7783.33, -7791.67, -7783.33, -7791.67, -7783.33, -7783.33, -7791.67, -7783.33, -7791.67, -7791.67, -7783.33, -7775.0, -7783.33, -7783.33, -7783.33, -7791.67, -7800.0, -7775.0, -7775.0, -7783.33, -7783.33, -7775.0, -7783.33, -7766.67, -7783.33, -7766.67, -7775.0, -7800.0, -7775.0, -7750.0, -7700.0, -7758.33, -7758.33, -7758.33, -7750.0, -7733.33, -7741.67, -7766.67, -7750.0, -7766.67, -7733.33, -7741.67, -7758.33, -7725.0, -7758.33, -7750.0, -7758.33, -7750.0, -7716.67, -7733.33, -7758.33, -7741.67, -7725.0, -7733.33, -7766.67, -7725.0, -7741.67, -7725.0, -7750.0, -7733.33, -7741.67, -7758.33, -7741.67, -7758.33, -7766.67, -7675.0, -7758.33, -7733.33, -7708.33, -7725.0, -7741.67, -7750.0, -7725.0, -7733.33, -7733.33, -7716.67, -7716.67], [-7716.67, -7725.0, -7716.67, -7725.0, -7708.33, -7716.67, -7725.0, -7708.33, -7733.33, -7691.67, -7716.67, -7691.67, -7733.33, -7700.0, -7733.33, -7683.33, -7733.33, -7733.33, -7700.0, -7725.0, -7716.67, -7733.33, -7700.0, -7708.33, -7716.67, -7716.67, -7716.67, -7725.0, -7708.33, -7700.0, -7708.33, -7708.33, -7700.0, -7716.67, -7708.33, -7700.0, -7708.33, -7708.33, -7700.0, -7700.0, -7700.0, -7708.33, -7708.33, -7700.0, -7700.0, -7691.67, -7708.33, -7700.0, -7691.67, -7708.33, -7700.0, -7700.0, -7700.0, -7691.67, -7691.67, -7691.67, -7683.33, -7716.67, -7700.0, -7700.0, -7691.67, -7675.0, -7708.33, -7700.0, -7683.33, -7675.0, -7683.33, -7691.67, -7683.33, -7691.67, -7683.33, -7683.33, -7683.33, -7675.0, -7683.33, -7675.0, -7683.33, -7691.67, -7683.33, -7683.33, -7683.33, -7683.33, -7683.33, -7658.33, -7675.0, -7691.67, -7675.0, -7675.0, -7683.33, -7675.0, -7675.0, -7675.0, -7675.0, -7666.67, -7683.33, -7666.67, -7675.0, -7658.33, -7666.67, -7658.33, -7675.0, -7666.67, -7675.0, -7675.0, -7666.67, -7666.67, -7666.67, -7658.33, -7666.67, -7658.33, -7675.0, -7650.0, -7675.0, -7658.33, -7666.67, -7658.33, -7650.0, -7666.67, -7658.33, -7658.33, -7666.67, -7666.67, -7658.33, -7650.0, -7666.67, -7666.67, -7658.33, -7658.33, -7633.33, -7666.67, -7625.0, -7650.0, -7633.33, -7650.0, -7641.67, -7641.67, -7641.67, -7650.0, -7641.67, -7650.0, -7633.33, -7641.67, -7616.67, -7650.0, -7650.0, -7633.33, -7641.67, -7641.67, -7650.0, -7650.0, -7641.67, -7658.33, -7633.33, -7641.67, -7641.67, -7633.33, -7641.67, -7641.67, -7633.33, -7641.67, -7633.33, -7633.33, -7625.0, -7633.33, -7633.33, -7633.33, -7633.33, -7625.0, -7625.0, -7633.33, -7633.33, -7633.33, -7625.0, -7633.33, -7641.67, -7633.33, -7616.67, -7625.0, -7616.67, -7633.33, -7625.0, -7625.0, -7625.0, -7625.0, -7625.0, -7633.33, -7608.33, -7608.33, -7608.33, -7625.0, -7625.0, -7616.67, -7616.67, -7625.0, -7608.33, -7608.33, -7600.0, -7625.0, -7616.67, -7608.33], [-7616.67, -7608.33, -7608.33, -7616.67, -7608.33, -7608.33, -7600.0, -7600.0, -7608.33, -7600.0, -7600.0, -7600.0, -7608.33, -7600.0, -7591.67, -7608.33, -7600.0, -7600.0, -7591.67, -7591.67, -7591.67, -7591.67, -7600.0, -7600.0, -7600.0, -7583.33, -7600.0, -7616.67, -7608.33, -7550.0, -7600.0, -7591.67, -7566.67, -7616.67, -7591.67, -7591.67, -7575.0, -7583.33, -7583.33, -7583.33, -7591.67, -7591.67, -7583.33, -7591.67, -7591.67, -7591.67, -7583.33, -7583.33, -7591.67, -7583.33, -7591.67, -7600.0, -7591.67, -7583.33, -7575.0, -7583.33, -7583.33, -7591.67, -7583.33, -7600.0, -7583.33, -7608.33, -7583.33, -7583.33, -7583.33, -7583.33, -7575.0, -7591.67, -7583.33, -7575.0, -7575.0, -7575.0, -7575.0, -7575.0, -7575.0, -7583.33, -7575.0, -7583.33, -7575.0, -7583.33, -7591.67, -7575.0, -7575.0, -7591.67, -7591.67, -7583.33, -7575.0, -7583.33, -7575.0, -7566.67, -7575.0, -7575.0, -7575.0, -7566.67, -7575.0, -7583.33, -7575.0, -7575.0, -7575.0, -7575.0, -7575.0, -7558.33, -7558.33, -7575.0, -7566.67, -7566.67, -7566.67, -7575.0, -7575.0, -7575.0, -7575.0, -7566.67, -7575.0, -7583.33, -7566.67, -7566.67, -7583.33, -7591.67, -7550.0, -7575.0, -7566.67, -7600.0, -7591.67, -7583.33, -7583.33, -7575.0, -7575.0, -7575.0, -7550.0, -7541.67, -7516.67, -7533.33, -7600.0, -7591.67, -7550.0, -7541.67, -7558.33, -7566.67, -7550.0, -7550.0, -7550.0, -7558.33, -7550.0, -7550.0, -7550.0, -7541.67, -7558.33, -7533.33, -7541.67, -7550.0, -7541.67, -7541.67, -7566.67, -7533.33, -7525.0, -7533.33, -7525.0, -7541.67, -7533.33, -7533.33, -7533.33, -7533.33, -7533.33, -7525.0, -7516.67, -7525.0, -7533.33, -7533.33, -7516.67, -7533.33, -7525.0, -7541.67, -7541.67, -7541.67, -7525.0, -7508.33, -7516.67, -7541.67, -7533.33, -7516.67, -7525.0, -7533.33, -7516.67, -7533.33, -7508.33, -7541.67, -7525.0, -7525.0, -7525.0, -7533.33, -7516.67, -7525.0, -7533.33, -7516.67, -7533.33, -7500.0, -7525.0, -7516.67, -7508.33, -7525.0], [-7550.0, -7508.33, -7508.33, -7533.33, -7525.0, -7516.67, -7533.33, -7508.33, -7541.67, -7516.67, -7525.0, -7516.67, -7541.67, -7508.33, -7525.0, -7525.0, -7558.33, -7525.0, -7516.67, -7516.67, -7475.0, -7516.67, -7525.0, -7525.0, -7500.0, -7475.0, -7500.0, -7508.33, -7516.67, -7508.33, -7550.0, -7491.67, -7508.33, -7558.33, -7516.67, -7541.67, -7525.0, -7550.0, -7533.33, -7491.67, -7533.33, -7550.0, -6750.0, -7533.33, -8575.0, -6975.0, -7708.33, -7466.67, -7500.0, -7500.0, -7458.33, -7483.33, -7508.33, -7475.0, -7508.33, -7491.67, -7483.33, -7483.33, -7491.67, -7500.0, -7458.33, -7466.67, -7466.67, -7475.0, -7466.67, -7483.33, -7475.0, -7458.33, -7475.0, -7458.33, -7458.33, -7491.67, -7391.67, -7466.67, -7483.33, -7458.33, -7466.67, -7466.67, -7458.33, -7475.0, -7466.67, -7466.67, -7458.33, -7458.33, -7475.0, -7458.33, -7458.33, -7458.33, -7466.67, -7458.33, -7441.67, -7466.67, -7458.33, -7458.33, -7458.33, -7458.33, -7466.67, -7450.0, -7475.0, -7466.67, -7466.67, -7458.33, -7475.0, -7458.33, -7450.0, -7458.33, -7466.67, -7466.67, -7408.33, -7475.0, -7408.33, -7450.0, -7433.33, -7441.67, -7425.0, -7441.67, -7450.0, -7425.0, -7433.33, -7441.67, -7441.67, -7441.67, -7433.33, -7458.33, -7450.0, -7458.33, -7483.33, -7458.33, -7483.33, -7450.0, -7458.33, -7458.33, -7458.33, -7475.0, -7475.0, -7466.67, -7458.33, -7458.33, -7475.0, -7466.67, -7458.33, -7475.0, -7483.33, -7466.67, -7458.33, -7466.67, -7458.33, -7283.33, -7533.33, -7466.67, -7450.0, -7466.67, -7475.0, -7466.67, -7450.0, -7466.67, -7475.0, -7475.0, -7475.0, -7466.67, -7433.33, -7458.33, -7450.0, -7458.33, -7466.67, -7475.0, -7458.33, -7458.33, -7458.33, -7466.67, -7441.67, -7441.67, -7491.67, -7458.33, -7475.0, -7441.67, -7483.33, -7450.0, -7466.67, -7458.33, -7475.0, -7458.33, -7466.67, -7450.0, -7458.33, -7458.33, -7450.0, -7441.67, -7466.67, -7450.0, -7458.33, -7458.33, -7458.33, -7458.33, -7450.0, -7458.33, -7208.33, -7508.33, -7466.67, -7450.0], [-7450.0, -7375.0, -7400.0, -7425.0, -7391.67, -7391.67, -7433.33, -7366.67, -7408.33, -7358.33, -7383.33, -7408.33, -7391.67, -7400.0, -7375.0, -7383.33, -7366.67, -7383.33, -7375.0, -7383.33, -7408.33, -7408.33, -7383.33, -7375.0, -7350.0, -7375.0, -7383.33, -7341.67, -7375.0, -7475.0, -7491.67, -7400.0, -7391.67, -7375.0, -7400.0, -7391.67, -7383.33, -7450.0, -7391.67, -7341.67, -7391.67, -7400.0, -7400.0, -7416.67, -7400.0, -7425.0, -7416.67, -7350.0, -7341.67, -7333.33, -7358.33, -7366.67, -7366.67, -7358.33, -7366.67, -7358.33, -7366.67, -7366.67, -7366.67, -7358.33, -7366.67, -7358.33, -7358.33, -7366.67, -7358.33, -7350.0, -7383.33, -7341.67, -7325.0, -7350.0, -7341.67, -7341.67, -7358.33, -7341.67, -7358.33, -7325.0, -7333.33, -7333.33, -7325.0, -7358.33, -7291.67, -7325.0, -7341.67, -7283.33, -7333.33, -7341.67, -7291.67, -7316.67, -7325.0, -7333.33, -7341.67, -7316.67, -7341.67, -7333.33, -7325.0, -7350.0, -7333.33, -7325.0, -7316.67, -7350.0, -7333.33, -7333.33, -7366.67, -7316.67, -7333.33, -7333.33, -7325.0, -7316.67, -7325.0, -7341.67, -7316.67, -7333.33, -7300.0, -7333.33, -7358.33, -7375.0, -7350.0, -7308.33, -7241.67, -7275.0, -7283.33, -7366.67, -7283.33, -7325.0, -7250.0, -7383.33, -7350.0, -7341.67, -7358.33, -7325.0, -7316.67, -7316.67, -7316.67, -7275.0, -7350.0, -7350.0, -7291.67, -7333.33, -7325.0, -7350.0, -7341.67, -7350.0, -7325.0, -7325.0, -7291.67, -7325.0, -7375.0, -7358.33, -7341.67, -7325.0, -7325.0, -7383.33, -7316.67, -7333.33, -7375.0, -7391.67, -7400.0, -7333.33, -7258.33, -7325.0, -7383.33, -7475.0, -7400.0, -7375.0, -7291.67, -7408.33, -7358.33, -7316.67, -7333.33, -7241.67, -7375.0, -7433.33, -7358.33, -7325.0, -7316.67, -7308.33, -7350.0, -7333.33, -7341.67, -7358.33, -7366.67, -7325.0, -7375.0, -7341.67, -7325.0, -7341.67, -7333.33, -7483.33, -7441.67, -8008.33, -8566.67, -9341.67, -10316.67, -11675.0, -14083.33, -16991.67, -18025.0, -18116.67, -18483.33, -17716.67], [-7491.67, -7458.33, -7425.0, -7450.0, -7416.67, -7391.67, -7400.0, -7400.0, -7375.0, -7350.0, -7341.67, -7316.67, -7316.67, -7316.67, -7250.0, -7125.0, -7475.0, -6950.0, -7308.33, -7766.67, -7141.67, -7241.67, -7050.0, -7233.33, -7275.0, -7200.0, -7300.0, -7533.33, -7283.33, -7283.33, -7408.33, -7383.33, -7425.0, -7391.67, -7425.0, -7450.0, -7416.67, -7383.33, -7441.67, -7450.0, -7475.0, -7466.67, -7508.33, -7466.67, -7475.0, -7508.33, -7450.0, -7441.67, -7450.0, -7383.33, -7458.33, -7558.33, -7500.0, -7508.33, -7508.33, -7508.33, -7500.0, -7466.67, -7466.67, -7508.33, -7466.67, -7491.67, -7491.67, -7466.67, -7483.33, -7458.33, -7466.67, -7500.0, -7441.67, -7483.33, -7475.0, -7466.67, -7450.0, -7441.67, -7458.33, -7466.67, -7466.67, -7458.33, -7458.33, -7466.67, -7441.67, -7466.67, -7458.33, -7475.0, -7433.33, -7433.33, -7441.67, -7458.33, -7458.33, -7466.67, -7450.0, -7441.67, -7466.67, -7450.0, -7450.0, -7441.67, -7441.67, -7466.67, -7458.33, -7466.67, -7466.67, -7466.67, -7525.0, -7483.33, -7433.33, -7483.33, -7508.33, -7241.67, -7450.0, -7433.33, -7441.67, -7441.67, -7450.0, -7541.67, -7466.67, -7433.33, -7483.33, -7441.67, -7416.67, -7450.0, -7366.67, -7483.33, -7433.33, -7483.33, -7433.33, -7375.0, -7416.67, -7500.0, -7375.0, -7458.33, -7458.33, -7391.67, -7375.0, -7425.0, -7433.33, -7400.0, -7416.67, -7400.0, -7450.0, -7400.0, -7425.0, -7441.67, -7441.67, -7400.0, -7433.33, -7383.33, -7416.67, -7433.33, -7358.33, -7425.0, -7500.0, -7433.33, -7491.67, -7416.67, -7458.33, -7500.0, -7433.33, -7433.33, -7458.33, -7475.0, -7416.67, -7433.33, -7491.67, -7425.0, -7525.0, -7483.33, -7525.0, -7758.33, -7266.67, -7575.0, -7258.33, -7458.33, -7425.0, -7341.67, -7408.33, -7491.67, -7400.0, -7441.67, -7416.67, -7416.67, -7516.67, -7600.0, -7408.33, -7233.33, -7283.33, -7358.33, -7425.0, -7458.33, -7325.0, -7308.33, -7383.33, -7291.67, -7508.33, -7275.0, -7283.33, -7366.67, -7308.33, -7333.33, -7375.0, -7191.67], [-7375.0, -7333.33, -7316.67, -7366.67, -7366.67, -7341.67, -7350.0, -7341.67, -7333.33, -7325.0, -7350.0, -7358.33, -7341.67, -7341.67, -7333.33, -7333.33, -7350.0, -7358.33, -7333.33, -7341.67, -7341.67, -7350.0, -7350.0, -7341.67, -7358.33, -7350.0, -7350.0, -7325.0, -7341.67, -7350.0, -7350.0, -7358.33, -7325.0, -7325.0, -7325.0, -7358.33, -7350.0, -7325.0, -7358.33, -7375.0, -7325.0, -7266.67, -7383.33, -7341.67, -7350.0, -7350.0, -7341.67, -7325.0, -7341.67, -7341.67, -7275.0, -6966.67, -7291.67, -7525.0, -7200.0, -7266.67, -7483.33, -6933.33, -8316.67, -10583.33, -10883.33, -11250.0, -10075.0, -11033.33, -9958.33, -9158.33, -8700.0, -8550.0, -8375.0, -8291.67, -8283.33, -8183.33, -8150.0, -8175.0, -8133.33, -8116.67, -7941.67, -8100.0, -8033.33, -8025.0, -7925.0, -7933.33, -7925.0, -7841.67, -7791.67, -7791.67, -7758.33, -7750.0, -7725.0, -7708.33, -7708.33, -7700.0, -7708.33, -7708.33, -7716.67, -7708.33, -7700.0, -7691.67, -7700.0, -7700.0, -7700.0, -7683.33, -7658.33, -7700.0, -7675.0, -7683.33, -7683.33, -7675.0, -7691.67, -7675.0, -7683.33, -7691.67, -7691.67, -7683.33, -7683.33, -7683.33, -7691.67, -7683.33, -7683.33, -7691.67, -7675.0, -7675.0, -7700.0, -7683.33, -7683.33, -7691.67, -7675.0, -7683.33, -7683.33, -7675.0, -7650.0, -7691.67, -7675.0, -7675.0, -7675.0, -7675.0, -7650.0, -7700.0, -7683.33, -7675.0, -7683.33, -7666.67, -7683.33, -7683.33, -7666.67, -7675.0, -7633.33, -7675.0, -7650.0, -7666.67, -7683.33, -7666.67, -7658.33, -7675.0, -7675.0, -7633.33, -7691.67, -7675.0, -7641.67, -7650.0, -7633.33, -7725.0, -7591.67, -7700.0, -7641.67, -7666.67, -7658.33, -7666.67, -7658.33, -7675.0, -7650.0, -7658.33, -7675.0, -7658.33, -7633.33, -7658.33, -7650.0, -7666.67, -7641.67, -7633.33, -7650.0, -7658.33, -7641.67, -7658.33, -7633.33, -7650.0, -7650.0, -7641.67, -7658.33, -7641.67, -7650.0, -7641.67, -7625.0, -7633.33, -7625.0, -7641.67, -7616.67, -7583.33, -7591.67, -7616.67], [-7583.33, -7608.33, -7600.0, -7600.0, -7583.33, -7608.33, -7583.33, -7600.0, -7583.33, -7591.67, -7583.33, -7575.0, -7591.67, -7583.33, -7591.67, -7591.67, -7575.0, -7600.0, -7591.67, -7600.0, -7583.33, -7608.33, -7591.67, -7600.0, -7591.67, -7591.67, -7608.33, -7616.67, -7600.0, -7600.0, -7600.0, -7600.0, -7600.0, -7600.0, -7600.0, -7591.67, -7591.67, -7583.33, -7600.0, -7600.0, -7600.0, -7583.33, -7583.33, -7575.0, -7591.67, -7591.67, -7591.67, -7600.0, -7600.0, -7608.33, -7600.0, -7583.33, -7583.33, -7583.33, -7591.67, -7591.67, -7591.67, -7591.67, -7591.67, -7591.67, -7591.67, -7583.33, -7591.67, -7600.0, -7583.33, -7591.67, -7591.67, -7591.67, -7591.67, -7608.33, -7583.33, -7583.33, -7583.33, -7575.0, -7583.33, -7583.33, -7583.33, -7591.67, -7591.67, -7583.33, -7591.67, -7583.33, -7583.33, -7608.33, -7583.33, -7583.33, -7583.33, -7575.0, -7558.33, -7558.33, -7575.0, -7566.67, -7575.0, -7575.0, -7600.0, -7566.67, -7591.67, -7583.33, -7583.33, -7575.0, -7591.67, -7583.33, -7566.67, -7541.67, -7575.0, -7558.33, -7591.67, -7533.33, -7583.33, -7575.0, -7575.0, -7533.33, -7550.0, -7541.67, -7541.67, -7566.67, -7541.67, -7566.67, -7575.0, -7566.67, -7566.67, -7566.67, -7566.67, -7566.67, -7558.33, -7575.0, -7583.33, -7550.0, -7558.33, -7558.33, -7550.0, -7566.67, -7566.67, -7558.33, -7575.0, -7575.0, -7566.67, -7566.67, -7566.67, -7575.0, -7558.33, -7575.0, -7566.67, -7566.67, -7550.0, -7566.67, -7558.33, -7558.33, -7550.0, -7558.33, -7558.33, -7558.33, -7558.33, -7566.67, -7566.67, -7558.33, -7566.67, -7566.67, -7566.67, -7558.33, -7558.33, -7566.67, -7566.67, -7566.67, -7558.33, -7566.67, -7566.67, -7566.67, -7558.33, -7566.67, -7558.33, -7550.0, -7566.67, -7558.33, -7541.67, -7566.67, -7550.0, -7566.67, -7558.33, -7566.67, -7566.67, -7558.33, -7558.33, -7558.33, -7541.67, -7550.0, -7550.0, -7558.33, -7558.33, -7558.33, -7566.67, -7533.33, -7566.67, -7558.33, -7558.33, -7566.67, -7558.33, -7558.33, -7566.67, -7550.0], [-7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7558.33, -7566.67, -7566.67, -7533.33, -7550.0, -7533.33, -7550.0, -7550.0, -7541.67, -7541.67, -7533.33, -7533.33, -7541.67, -7558.33, -7541.67, -7550.0, -7550.0, -7508.33, -7541.67, -7525.0, -7541.67, -7533.33, -7533.33, -7533.33, -7533.33, -7533.33, -7533.33, -7525.0, -7533.33, -7533.33, -7533.33, -7533.33, -7533.33, -7533.33, -7525.0, -7525.0, -7516.67, -7533.33, -7516.67, -7541.67, -7541.67, -7575.0, -7550.0, -7541.67, -7516.67, -7541.67, -7533.33, -7516.67, -7533.33, -7533.33, -7533.33, -7525.0, -7533.33, -7533.33, -7533.33, -7533.33, -7525.0, -7525.0, -7525.0, -7516.67, -7533.33, -7525.0, -7533.33, -7550.0, -7525.0, -7516.67, -7525.0, -7525.0, -7533.33, -7541.67, -7533.33, -7516.67, -7525.0, -7525.0, -7533.33, -7533.33, -7533.33, -7525.0, -7525.0, -7533.33, -7516.67, -7525.0, -7525.0, -7516.67, -7516.67, -7550.0, -7541.67, -7525.0, -7541.67, -7516.67, -7516.67, -7516.67, -7525.0, -7525.0, -7516.67, -7516.67, -7525.0, -7525.0, -7525.0, -7516.67, -7533.33, -7525.0, -7525.0, -7541.67, -7508.33, -7541.67, -7516.67, -7525.0, -7525.0, -7525.0, -7516.67, -7525.0, -7525.0, -7508.33, -7541.67, -7508.33, -7525.0, -7508.33, -7533.33, -7525.0, -7516.67, -7533.33, -7541.67, -7500.0, -7516.67, -7525.0, -7550.0, -7541.67, -7500.0, -7491.67, -7516.67, -7533.33, -7516.67, -7466.67, -7483.33, -7491.67, -7500.0, -7491.67, -7458.33, -7475.0, -7508.33, -7491.67, -7475.0, -7466.67, -7483.33, -7500.0, -7450.0, -7416.67, -7475.0, -7483.33, -7483.33, -7458.33, -7483.33, -7500.0, -7475.0, -7500.0, -7483.33, -7516.67, -7475.0, -7516.67, -7483.33, -7491.67, -7475.0, -7525.0, -7525.0, -7508.33, -7483.33, -7500.0, -7491.67, -7491.67, -7433.33, -7500.0, -7508.33, -7508.33, -7500.0, -7500.0, -7475.0, -7500.0, -7475.0, -7508.33, -7483.33, -7508.33, -7458.33, -7508.33, -7475.0, -7508.33, -7483.33, -7525.0, -7483.33, -7508.33, -7483.33, -7508.33, -7450.0, -7516.67, -7441.67], [-7466.67, -7483.33, -7475.0, -7466.67, -7483.33, -7483.33, -7483.33, -7466.67, -7475.0, -7475.0, -7483.33, -7475.0, -7483.33, -7483.33, -7466.67, -7475.0, -7475.0, -7466.67, -7466.67, -7458.33, -7466.67, -7466.67, -7466.67, -7475.0, -7483.33, -7466.67, -7458.33, -7483.33, -7475.0, -7466.67, -7483.33, -7483.33, -7475.0, -7466.67, -7475.0, -7475.0, -7458.33, -7450.0, -7475.0, -7466.67, -7458.33, -7458.33, -7458.33, -7458.33, -7466.67, -7466.67, -7466.67, -7483.33, -7458.33, -7458.33, -7450.0, -7450.0, -7466.67, -7466.67, -7466.67, -7458.33, -7475.0, -7466.67, -7458.33, -7475.0, -7466.67, -7458.33, -7466.67, -7466.67, -7458.33, -7458.33, -7458.33, -7475.0, -7458.33, -7475.0, -7475.0, -7450.0, -7458.33, -7466.67, -7466.67, -7450.0, -7466.67, -7458.33, -7458.33, -7458.33, -7458.33, -7466.67, -7458.33, -7458.33, -7475.0, -7458.33, -7458.33, -7466.67, -7466.67, -7466.67, -7466.67, -7458.33, -7450.0, -7450.0, -7458.33, -7450.0, -7466.67, -7466.67, -7450.0, -7475.0, -7458.33, -7466.67, -7458.33, -7458.33, -7466.67, -7458.33, -7458.33, -7466.67, -7458.33, -7441.67, -7458.33, -7441.67, -7458.33, -7458.33, -7450.0, -7458.33, -7483.33, -7466.67, -7441.67, -7458.33, -7450.0, -7450.0, -7458.33, -7450.0, -7450.0, -7450.0, -7458.33, -7441.67, -7458.33, -7450.0, -7441.67, -7441.67, -7441.67, -7450.0, -7416.67, -7441.67, -7450.0, -7450.0, -7433.33, -7441.67, -7425.0, -7433.33, -7408.33, -7416.67, -7408.33, -7416.67, -7408.33, -7408.33, -7416.67, -7408.33, -7416.67, -7416.67, -7416.67, -7400.0, -7425.0, -7425.0, -7416.67, -7416.67, -7425.0, -7408.33, -7408.33, -7408.33, -7408.33, -7408.33, -7416.67, -7408.33, -7408.33, -7416.67, -7408.33, -7425.0, -7408.33, -7416.67, -7400.0, -7416.67, -7416.67, -7408.33, -7408.33, -7425.0, -7408.33, -7408.33, -7400.0, -7416.67, -7425.0, -7416.67, -7416.67, -7416.67, -7416.67, -7408.33, -7416.67, -7391.67, -7425.0, -7433.33, -7416.67, -7408.33, -7408.33, -7408.33, -7425.0, -7425.0, -7408.33, -7425.0], [-7408.33, -7408.33, -7408.33, -7416.67, -7400.0, -7391.67, -7391.67, -7400.0, -7400.0, -7391.67, -7333.33, -7375.0, -7391.67, -7400.0, -7391.67, -7400.0, -7408.33, -7400.0, -7400.0, -7400.0, -7400.0, -7400.0, -7400.0, -7400.0, -7408.33, -7408.33, -7400.0, -7400.0, -7400.0, -7391.67, -7400.0, -7408.33, -7425.0, -7408.33, -7400.0, -7408.33, -7391.67, -7400.0, -7400.0, -7391.67, -7391.67, -7400.0, -7391.67, -7408.33, -7391.67, -7400.0, -7400.0, -7400.0, -7408.33, -7391.67, -7400.0, -7391.67, -7400.0, -7391.67, -7400.0, -7400.0, -7383.33, -7383.33, -7383.33, -7383.33, -7383.33, -7391.67, -7375.0, -7383.33, -7375.0, -7383.33, -7391.67, -7383.33, -7391.67, -7383.33, -7400.0, -7375.0, -7383.33, -7383.33, -7400.0, -7391.67, -7391.67, -7400.0, -7400.0, -7383.33, -7383.33, -7375.0, -7391.67, -7391.67, -7391.67, -7383.33, -7383.33, -7391.67, -7383.33, -7375.0, -7400.0, -7383.33, -7375.0, -7391.67, -7383.33, -7383.33, -7383.33, -7383.33, -7383.33, -7375.0, -7383.33, -7375.0, -7366.67, -7375.0, -7383.33, -7383.33, -7383.33, -7375.0, -7366.67, -7375.0, -7383.33, -7383.33, -7375.0, -7383.33, -7375.0, -7366.67, -7383.33, -7375.0, -7391.67, -7383.33, -7383.33, -7383.33, -7383.33, -7391.67, -7375.0, -7366.67, -7383.33, -7383.33, -7358.33, -7383.33, -7383.33, -7366.67, -7366.67, -7375.0, -7375.0, -7375.0, -7375.0, -7375.0, -7358.33, -7358.33, -7358.33, -7358.33, -7375.0, -7350.0, -7350.0, -7350.0, -7366.67, -7341.67, -7358.33, -7341.67, -7383.33, -7325.0, -7375.0, -7341.67, -7358.33, -7358.33, -7358.33, -7350.0, -7366.67, -7350.0, -7366.67, -7366.67, -7366.67, -7358.33, -7366.67, -7358.33, -7350.0, -7375.0, -7350.0, -7358.33, -7358.33, -7366.67, -7358.33, -7350.0, -7341.67, -7358.33, -7358.33, -7350.0, -7358.33, -7366.67, -7358.33, -7358.33, -7366.67, -7350.0, -7350.0, -7341.67, -7350.0, -7350.0, -7350.0, -7350.0, -7350.0, -7350.0, -7358.33, -7358.33, -7358.33, -7358.33, -7333.33, -7341.67, -7375.0, -7333.33], [-7333.33, -7350.0, -7366.67, -7308.33, -7358.33, -7333.33, -7341.67, -7333.33, -7350.0, -7341.67, -7350.0, -7350.0, -7325.0, -7350.0, -7341.67, -7341.67, -7350.0, -7325.0, -7350.0, -7350.0, -7341.67, -7325.0, -7366.67, -7316.67, -7341.67, -7350.0, -7350.0, -7341.67, -7350.0, -7358.33, -7333.33, -7316.67, -7333.33, -7341.67, -7333.33, -7333.33, -7358.33, -7333.33, -7358.33, -7333.33, -7350.0, -7333.33, -7316.67, -7341.67, -7325.0, -7325.0, -7333.33, -7358.33, -7333.33, -7325.0, -7325.0, -7341.67, -7333.33, -7316.67, -7308.33, -7533.33, -7325.0, -7316.67, -7258.33, -7341.67, -7333.33, -7308.33, -7341.67, -7241.67, -7333.33, -7316.67, -7308.33, -7350.0, -7333.33, -7333.33, -7308.33, -7308.33, -7308.33, -7308.33, -7333.33, -7325.0, -7325.0, -7333.33, -7325.0, -7333.33, -7325.0, -7316.67, -7316.67, -7325.0, -7325.0, -7316.67, -7316.67, -7316.67, -7341.67, -7316.67, -7316.67, -7325.0, -7325.0, -7333.33, -7308.33, -7325.0, -7316.67, -7308.33, -7341.67, -7333.33, -7316.67, -7325.0, -7308.33, -7325.0, -7308.33, -7325.0, -7325.0, -7316.67, -7308.33, -7316.67, -7316.67, -7325.0, -7300.0, -7325.0, -7308.33, -7308.33, -7325.0, -7316.67, -7308.33, -7316.67, -7325.0, -7316.67, -7325.0, -7316.67, -7316.67, -7325.0, -7316.67, -7325.0, -7308.33, -7316.67, -7316.67, -7308.33, -7316.67, -7325.0, -7308.33, -7316.67, -7316.67, -7308.33, -7325.0, -7316.67, -7308.33, -7316.67, -7308.33, -7316.67, -7308.33, -7308.33, -7316.67, -7316.67, -7308.33, -7300.0, -7325.0, -7308.33, -7316.67, -7300.0, -7333.33, -7308.33, -7308.33, -7316.67, -7300.0, -7308.33, -7308.33, -7291.67, -7308.33, -7316.67, -7308.33, -7300.0, -7308.33, -7300.0, -7308.33, -7316.67, -7316.67, -7300.0, -7325.0, -7308.33, -7291.67, -7333.33, -7300.0, -7283.33, -7300.0, -7316.67, -7291.67, -7308.33, -7300.0, -7308.33, -7300.0, -7300.0, -7266.67, -7308.33, -7316.67, -7316.67, -7308.33, -7316.67, -7300.0, -7300.0, -7300.0, -7316.67, -7316.67, -7308.33, -7291.67, -7308.33], [-7300.0, -7283.33, -7325.0, -7300.0, -7308.33, -7325.0, -7308.33, -7275.0, -7333.33, -7291.67, -7308.33, -7316.67, -7275.0, -7325.0, -7275.0, -7325.0, -7266.67, -7291.67, -7300.0, -7283.33, -7300.0, -7283.33, -7325.0, -7300.0, -7291.67, -7291.67, -7308.33, -7308.33, -7291.67, -7325.0, -7291.67, -7308.33, -7325.0, -7258.33, -7308.33, -7325.0, -7283.33, -7250.0, -7325.0, -7291.67, -7283.33, -7300.0, -7325.0, -7300.0, -7275.0, -7291.67, -7241.67, -7308.33, -7275.0, -7291.67, -7291.67, -7300.0, -7275.0, -7308.33, -7308.33, -7316.67, -7308.33, -7291.67, -7216.67, -7316.67, -7266.67, -7300.0, -7291.67, -7283.33, -7300.0, -7308.33, -7325.0, -7300.0, -7308.33, -7358.33, -7300.0, -7300.0, -7266.67, -7275.0, -7291.67, -7258.33, -7283.33, -7275.0, -7283.33, -7266.67, -7275.0, -7275.0, -7291.67, -7258.33, -7250.0, -7258.33, -7266.67, -7283.33, -7266.67, -7283.33, -7275.0, -7275.0, -7375.0, -7275.0, -7250.0, -7275.0, -7266.67, -7266.67, -7266.67, -7308.33, -7283.33, -7275.0, -7275.0, -7258.33, -7266.67, -7283.33, -7258.33, -7258.33, -7266.67, -7258.33, -7250.0, -7266.67, -7275.0, -7266.67, -7266.67, -7258.33, -7266.67, -7250.0, -7266.67, -7258.33, -7275.0, -7258.33, -7258.33, -7266.67, -7283.33, -7266.67, -7258.33, -7266.67, -7291.67, -7266.67, -7241.67, -7250.0, -7275.0, -7266.67, -7258.33, -7250.0, -7266.67, -7266.67, -7225.0, -7275.0, -7291.67, -7233.33, -7250.0, -7266.67, -7258.33, -7250.0, -7275.0, -7233.33, -7258.33, -7250.0, -7266.67, -7258.33, -7258.33, -7258.33, -7250.0, -7266.67, -7241.67, -7266.67, -7233.33, -7241.67, -7241.67, -7258.33, -7250.0, -7258.33, -7225.0, -7258.33, -7308.33, -7250.0, -7233.33, -7208.33, -7291.67, -7241.67, -7266.67, -7233.33, -7291.67, -7250.0, -7241.67, -7250.0, -7250.0, -7241.67, -7250.0, -7250.0, -7250.0, -7258.33, -7266.67, -7275.0, -7233.33, -7241.67, -7258.33, -7241.67, -7241.67, -7250.0, -7275.0, -7241.67, -7208.33, -7258.33, -7266.67, -7233.33, -7250.0, -7241.67], [-7258.33, -7250.0, -7258.33, -7241.67, -7216.67, -7258.33, -7258.33, -7241.67, -7250.0, -7300.0, -7366.67, -7258.33, -7275.0, -7291.67, -7233.33, -7241.67, -7208.33, -7266.67, -7191.67, -7141.67, -7225.0, -7250.0, -7266.67, -7266.67, -7208.33, -7233.33, -7233.33, -7241.67, -7233.33, -7241.67, -7233.33, -7233.33, -7225.0, -7200.0, -7225.0, -7225.0, -7216.67, -7258.33, -7208.33, -7233.33, -7250.0, -7233.33, -7241.67, -7250.0, -7225.0, -7216.67, -7216.67, -7241.67, -7241.67, -7233.33, -7250.0, -7241.67, -7241.67, -7225.0, -7225.0, -7250.0, -7241.67, -7241.67, -7241.67, -7250.0, -7233.33, -7241.67, -7241.67, -7233.33, -7250.0, -7233.33, -7241.67, -7250.0, -7250.0, -7250.0, -7258.33, -7241.67, -7250.0, -7241.67, -7250.0, -7258.33, -7258.33, -7250.0, -7250.0, -7258.33, -7291.67, -7266.67, -7350.0, -7333.33, -7333.33, -7383.33, -7991.67, -10100.0, -9450.0, -10416.67, -13700.0, -14841.67, -15691.67, -17183.33, -17466.67, -15308.33, -15350.0, -17000.0, -15941.67, -16133.33, -16500.0, -16150.0, -15700.0, -16458.33, -16891.67, -16183.33, -16300.0, -16016.67, -16225.0, -16141.67, -15908.33, -16091.67, -15991.67, -15725.0, -15966.67, -16150.0, -15808.33, -15841.67, -15808.33, -15791.67, -15675.0, -15675.0, -15566.67, -15616.67, -15533.33, -15433.33, -15450.0, -15325.0, -15325.0, -15316.67, -15308.33, -15266.67, -15216.67, -15175.0, -15183.33, -15150.0, -15108.33, -15141.67, -15066.67, -15041.67, -15041.67, -15033.33, -14991.67, -14941.67, -14941.67, -14950.0, -14933.33, -14925.0, -14900.0, -14883.33, -14875.0, -14833.33, -14850.0, -14800.0, -14791.67, -14783.33, -14766.67, -14741.67, -14766.67, -14691.67, -14666.67, -14708.33, -14675.0, -14675.0, -14641.67, -14633.33, -14625.0, -14625.0, -14600.0, -14550.0, -14700.0, -14850.0, -15008.33, -15366.67, -15258.33, -16016.67, -16666.67, -17258.33, -17133.33, -16516.67, -15891.67, -16150.0, -15541.67, -14750.0, -15158.33, -14900.0, -14716.67, -14825.0, -14783.33, -14141.67, -13908.33, -14350.0, -14241.67, -14125.0, -14116.67, -14200.0, -14108.33, -14108.33, -14050.0, -13900.0], [-14125.0, -14475.0, -13575.0, -13666.67, -13341.67, -13333.33, -13500.0, -13383.33, -13133.33, -13208.33, -13250.0, -13175.0, -13166.67, -13108.33, -13091.67, -13091.67, -13041.67, -13025.0, -13050.0, -12975.0, -13008.33, -12975.0, -12975.0, -12966.67, -12941.67, -12916.67, -12908.33, -12858.33, -12883.33, -12850.0, -12850.0, -12825.0, -12791.67, -12791.67, -12800.0, -12783.33, -12758.33, -12733.33, -12650.0, -12766.67, -12808.33, -13241.67, -13441.67, -13341.67, -13350.0, -13058.33, -13341.67, -13200.0, -13191.67, -13241.67, -13183.33, -13108.33, -13025.0, -12975.0, -12891.67, -12758.33, -12808.33, -12700.0, -12691.67, -12633.33, -12591.67, -12591.67, -12516.67, -12550.0, -12558.33, -12475.0, -12500.0, -12466.67, -12433.33, -12500.0, -12408.33, -12466.67, -12433.33, -12416.67, -12433.33, -12433.33, -12400.0, -12425.0, -12400.0, -12416.67, -12416.67, -12400.0, -12358.33, -12375.0, -12491.67, -12366.67, -12358.33, -12275.0, -12333.33, -12283.33, -12266.67, -12283.33, -12275.0, -12275.0, -12216.67, -12216.67, -12250.0, -12200.0, -12200.0, -12233.33, -12200.0, -12175.0, -12216.67, -12191.67, -12225.0, -12208.33, -12200.0, -12191.67, -12200.0, -12200.0, -12191.67, -12200.0, -12191.67, -12200.0, -12175.0, -12175.0, -12150.0, -12150.0, -12133.33, -12141.67, -12116.67, -12133.33, -12141.67, -12100.0, -12108.33, -12108.33, -12075.0, -12083.33, -12100.0, -12058.33, -12100.0, -12075.0, -12091.67, -12083.33, -12066.67, -12066.67, -12050.0, -12050.0, -12041.67, -12050.0, -12050.0, -12025.0, -12025.0, -12016.67, -12016.67, -11983.33, -12008.33, -11983.33, -12008.33, -11975.0, -11966.67, -11991.67, -11958.33, -11966.67, -11958.33, -11925.0, -11983.33, -11966.67, -11933.33, -11933.33, -11958.33, -11950.0, -11966.67, -11966.67, -11941.67, -11950.0, -11941.67, -11908.33, -11925.0, -11908.33, -11900.0, -11875.0, -11900.0, -11891.67, -11908.33, -11866.67, -11866.67, -11891.67, -11841.67, -11858.33, -11858.33, -11833.33, -11883.33, -11850.0, -11866.67, -11858.33, -11883.33, -11866.67, -11858.33, -11883.33, -11875.0, -11875.0, -11858.33, -11833.33, -11816.67, -11758.33, -11866.67, -12025.0, -12383.33, -12758.33], [-10591.67, -10458.33, -10375.0, -10283.33, -10200.0, -10158.33, -10100.0, -10025.0, -9975.0, -9941.67, -9875.0, -9825.0, -9791.67, -9783.33, -9725.0, -9675.0, -9666.67, -9633.33, -9600.0, -9725.0, -9541.67, -9525.0, -9516.67, -9483.33, -9458.33, -9450.0, -9375.0, -9433.33, -9400.0, -9391.67, -9366.67, -9325.0, -9325.0, -9341.67, -9316.67, -9291.67, -9275.0, -9258.33, -9250.0, -9233.33, -9241.67, -9225.0, -9216.67, -9200.0, -9183.33, -9183.33, -9150.0, -9158.33, -9150.0, -9125.0, -9116.67, -9108.33, -9058.33, -9133.33, -9091.67, -9075.0, -9033.33, -9058.33, -9041.67, -9025.0, -9033.33, -9008.33, -9016.67, -8983.33, -8991.67, -8991.67, -8708.33, -8966.67, -8975.0, -8966.67, -8950.0, -8958.33, -8933.33, -8941.67, -8916.67, -8925.0, -8891.67, -8925.0, -8908.33, -8891.67, -8883.33, -8883.33, -8891.67, -8866.67, -8858.33, -8841.67, -8858.33, -8841.67, -8833.33, -8833.33, -8825.0, -8825.0, -8800.0, -8816.67, -8800.0, -8791.67, -8783.33, -8783.33, -8783.33, -8783.33, -8766.67, -8775.0, -8758.33, -8750.0, -8758.33, -8741.67, -8741.67, -8725.0, -8216.67, -8766.67, -8758.33, -8741.67, -8725.0, -8733.33, -8716.67, -8733.33, -8708.33, -8716.67, -8708.33, -8708.33, -8700.0, -8700.0, -8700.0, -8691.67, -8700.0, -8666.67, -8675.0, -8675.0, -8666.67, -8666.67, -8675.0, -8641.67, -8625.0, -8691.67, -8658.33, -8658.33, -8633.33, -8650.0, -8650.0, -8633.33, -8616.67, -8600.0, -8591.67, -8566.67, -8541.67, -8508.33, -8583.33, -8550.0, -8566.67, -8541.67, -8533.33, -8525.0, -8533.33, -8533.33, -8541.67, -8541.67, -8508.33, -8516.67, -8525.0, -8533.33, -8533.33, -8533.33, -8508.33, -8508.33, -8508.33, -8508.33, -8516.67, -8491.67, -8491.67, -8483.33, -8483.33, -8491.67, -8516.67, -8483.33, -8500.0, -8466.67, -8483.33, -8500.0, -8491.67, -8483.33, -8450.0, -8483.33, -8475.0, -8466.67, -8450.0, -8450.0, -8466.67, -8458.33, -8458.33, -8433.33, -8450.0, -8441.67, -8441.67, -8450.0, -8441.67, -8441.67, -8433.33, -8433.33, -8408.33, -8425.0], [-8375.0, -8391.67, -8391.67, -8375.0, -8383.33, -8358.33, -8366.67, -8366.67, -8366.67, -8366.67, -8366.67, -8358.33, -8358.33, -8366.67, -8358.33, -8341.67, -8375.0, -8350.0, -8366.67, -8358.33, -8333.33, -8358.33, -8333.33, -8350.0, -8333.33, -8333.33, -8325.0, -8350.0, -8341.67, -8333.33, -8325.0, -8333.33, -8350.0, -8308.33, -8341.67, -8325.0, -8341.67, -8325.0, -8308.33, -8325.0, -8341.67, -8341.67, -8308.33, -8316.67, -8316.67, -8291.67, -8308.33, -8325.0, -8308.33, -8308.33, -8333.33, -8291.67, -8316.67, -8300.0, -8291.67, -8316.67, -8283.33, -8291.67, -8308.33, -8283.33, -8316.67, -8308.33, -8300.0, -8250.0, -8283.33, -8291.67, -8283.33, -8300.0, -8275.0, -8300.0, -8275.0, -8283.33, -8275.0, -8300.0, -8266.67, -8283.33, -8283.33, -8291.67, -8266.67, -8283.33, -8283.33, -8258.33, -8266.67, -8266.67, -8266.67, -8275.0, -8258.33, -8275.0, -8275.0, -8258.33, -8275.0, -8250.0, -8275.0, -8233.33, -8258.33, -8250.0, -8258.33, -8258.33, -8258.33, -8258.33, -8266.67, -8250.0, -8258.33, -8250.0, -8241.67, -8266.67, -8250.0, -8250.0, -8316.67, -8216.67, -8241.67, -8233.33, -8191.67, -8308.33, -8225.0, -8241.67, -8233.33, -8216.67, -8250.0, -8300.0, -8183.33, -8225.0, -8225.0, -8241.67, -8200.0, -8241.67, -8191.67, -8241.67, -8175.0, -8233.33, -8258.33, -8233.33, -8216.67, -8233.33, -8216.67, -8200.0, -8208.33, -8183.33, -8250.0, -8200.0, -8216.67, -8200.0, -8200.0, -8200.0, -8258.33, -8233.33, -8208.33, -8175.0, -8266.67, -8241.67, -8208.33, -8200.0, -8208.33, -8208.33, -8191.67, -8183.33, -8208.33, -8216.67, -8200.0, -8175.0, -8208.33, -8175.0, -8208.33, -8208.33, -8183.33, -8191.67, -8175.0, -8175.0, -8183.33, -8183.33, -8191.67, -8183.33, -8183.33, -8183.33, -8183.33, -8166.67, -8166.67, -8158.33, -8158.33, -8158.33, -8175.0, -8166.67, -8166.67, -8175.0, -8175.0, -8158.33, -8175.0, -8175.0, -8166.67, -8175.0, -8183.33, -8158.33, -8150.0, -8183.33, -8166.67, -8166.67, -8150.0, -8175.0, -8191.67, -8175.0], [-8150.0, -8141.67, -8133.33, -8158.33, -8150.0, -8150.0, -8125.0, -8133.33, -8133.33, -8133.33, -8141.67, -8150.0, -8141.67, -8141.67, -8116.67, -8141.67, -8141.67, -8141.67, -8133.33, -8116.67, -8141.67, -8125.0, -8125.0, -8150.0, -8133.33, -8125.0, -8116.67, -8133.33, -8125.0, -8125.0, -8125.0, -8100.0, -8125.0, -8116.67, -8141.67, -8108.33, -8116.67, -8133.33, -8116.67, -8108.33, -8125.0, -8100.0, -8141.67, -8116.67, -8116.67, -8108.33, -8133.33, -8125.0, -8116.67, -8116.67, -8116.67, -8108.33, -8116.67, -8125.0, -8116.67, -8125.0, -8116.67, -8125.0, -8108.33, -8125.0, -8108.33, -8108.33, -8116.67, -8108.33, -8108.33, -8116.67, -8108.33, -8116.67, -8108.33, -8100.0, -8108.33, -8108.33, -8091.67, -8108.33, -8116.67, -8100.0, -8100.0, -8100.0, -8108.33, -8108.33, -8108.33, -8100.0, -8108.33, -8108.33, -8108.33, -8116.67, -8091.67, -8100.0, -8091.67, -8100.0, -8116.67, -8100.0, -8100.0, -8091.67, -8116.67, -8100.0, -8091.67, -8100.0, -8116.67, -8116.67, -8108.33, -8091.67, -8091.67, -8133.33, -8100.0, -8075.0, -8100.0, -8100.0, -8116.67, -8075.0, -8075.0, -8125.0, -8075.0, -8091.67, -8083.33, -8108.33, -8108.33, -8100.0, -8091.67, -8091.67, -8100.0, -8125.0, -8066.67, -8066.67, -8108.33, -8083.33, -8091.67, -8083.33, -8091.67, -8083.33, -8100.0, -8091.67, -8083.33, -8091.67, -8083.33, -8083.33, -8083.33, -8083.33, -8083.33, -8075.0, -8091.67, -8083.33, -8075.0, -8075.0, -8108.33, -8075.0, -8066.67, -8066.67, -8091.67, -8083.33, -8075.0, -8091.67, -8058.33, -8066.67, -8108.33, -8075.0, -8066.67, -8075.0, -8066.67, -8075.0, -8066.67, -8083.33, -8066.67, -8083.33, -8075.0, -8083.33, -8075.0, -8075.0, -8066.67, -8058.33, -8058.33, -8066.67, -8058.33, -8066.67, -8058.33, -8066.67, -8083.33, -8066.67, -8058.33, -8066.67, -8066.67, -8066.67, -8066.67, -8075.0, -8075.0, -8050.0, -8075.0, -8050.0, -8075.0, -8050.0, -8066.67, -8075.0, -8058.33, -8075.0, -8066.67, -8066.67, -8066.67, -8058.33, -8066.67, -8058.33], [-8058.33, -8050.0, -8058.33, -8050.0, -8050.0, -8041.67, -8050.0, -8050.0, -8058.33, -8050.0, -8050.0, -8058.33, -8050.0, -8050.0, -8050.0, -8058.33, -8050.0, -8041.67, -8041.67, -8066.67, -8050.0, -8050.0, -8041.67, -8041.67, -8050.0, -8025.0, -8033.33, -8066.67, -8033.33, -8050.0, -8050.0, -8041.67, -8041.67, -8041.67, -8041.67, -8050.0, -8050.0, -8025.0, -8066.67, -8058.33, -8041.67, -8041.67, -8058.33, -8041.67, -8033.33, -8033.33, -8050.0, -8041.67, -8041.67, -8050.0, -8025.0, -8058.33, -8025.0, -8025.0, -8041.67, -8041.67, -8033.33, -8041.67, -8050.0, -8033.33, -8050.0, -8025.0, -8041.67, -8033.33, -8041.67, -8033.33, -8025.0, -8041.67, -8041.67, -8033.33, -8050.0, -8033.33, -8033.33, -8033.33, -8025.0, -8025.0, -8033.33, -8025.0, -8041.67, -8025.0, -8025.0, -8016.67, -8041.67, -8041.67, -8016.67, -8016.67, -8025.0, -8025.0, -8025.0, -8033.33, -8025.0, -8016.67, -8016.67, -8033.33, -8016.67, -8033.33, -8025.0, -8025.0, -8033.33, -8025.0, -8025.0, -8025.0, -8016.67, -8025.0, -8008.33, -8041.67, -8000.0, -8075.0, -8041.67, -8041.67, -8050.0, -8016.67, -8008.33, -8008.33, -7991.67, -8108.33, -8016.67, -8033.33, -8008.33, -8033.33, -8025.0, -8016.67, -8016.67, -8016.67, -8016.67, -8016.67, -8016.67, -8016.67, -8016.67, -8008.33, -8008.33, -8033.33, -8016.67, -8016.67, -8016.67, -8016.67, -8025.0, -8008.33, -8016.67, -8016.67, -8025.0, -8000.0, -8008.33, -8016.67, -8008.33, -8016.67, -8016.67, -8008.33, -8025.0, -8008.33, -8025.0, -8016.67, -8008.33, -8008.33, -8008.33, -8008.33, -8025.0, -8008.33, -8000.0, -8016.67, -7991.67, -8016.67, -8016.67, -8016.67, -8008.33, -7991.67, -8008.33, -8016.67, -7991.67, -8000.0, -8000.0, -8008.33, -8008.33, -8000.0, -8008.33, -8008.33, -8000.0, -8008.33, -8000.0, -8000.0, -8000.0, -8000.0, -8000.0, -8008.33, -8008.33, -8008.33, -8000.0, -8008.33, -7991.67, -8000.0, -8000.0, -8000.0, -8000.0, -8000.0, -8000.0, -8008.33, -8000.0, -8000.0, -7991.67, -8000.0], [-7991.67, -7991.67, -7991.67, -8000.0, -8000.0, -7991.67, -7975.0, -7958.33, -7975.0, -7958.33, -7975.0, -7983.33, -7966.67, -7975.0, -7966.67, -7950.0, -7950.0, -7966.67, -7958.33, -7958.33, -7958.33, -7966.67, -7966.67, -7958.33, -7966.67, -7950.0, -7975.0, -7966.67, -7975.0, -7950.0, -7975.0, -7975.0, -7958.33, -8000.0, -7958.33, -7966.67, -7958.33, -7975.0, -7958.33, -7958.33, -7983.33, -7966.67, -7958.33, -7975.0, -7966.67, -7983.33, -7958.33, -7966.67, -7975.0, -7958.33, -7966.67, -7966.67, -7966.67, -7958.33, -7975.0, -7966.67, -7958.33, -7966.67, -7958.33, -7966.67, -7958.33, -7966.67, -7958.33, -7966.67, -7958.33, -7958.33, -7950.0, -7975.0, -7966.67, -7941.67, -7966.67, -7958.33, -7950.0, -7966.67, -7933.33, -8033.33, -7941.67, -7958.33, -7983.33, -7925.0, -7975.0, -7975.0, -7950.0, -7950.0, -7958.33, -7958.33, -7966.67, -7966.67, -7958.33, -7966.67, -7958.33, -7950.0, -7958.33, -7958.33, -7958.33, -7958.33, -7958.33, -7958.33, -7950.0, -7958.33, -7950.0, -7958.33, -7958.33, -7958.33, -7966.67, -7958.33, -7950.0, -7958.33, -7958.33, -7950.0, -7950.0, -7958.33, -7958.33, -7941.67, -7950.0, -7958.33, -7950.0, -7958.33, -7950.0, -7950.0, -7958.33, -7958.33, -7958.33, -7950.0, -7958.33, -7958.33, -7958.33, -7941.67, -7958.33, -7958.33, -7950.0, -7958.33, -7950.0, -7950.0, -7950.0, -7950.0, -7950.0, -7941.67, -7950.0, -7958.33, -7950.0, -7950.0, -7950.0, -7950.0, -7950.0, -7950.0, -7950.0, -7941.67, -7958.33, -7950.0, -7950.0, -7950.0, -7933.33, -7958.33, -7950.0, -7941.67, -7941.67, -7950.0, -7950.0, -7950.0, -7941.67, -7950.0, -7950.0, -7933.33, -7941.67, -7941.67, -7933.33, -7941.67, -7950.0, -7950.0, -7941.67, -7958.33, -7950.0, -7950.0, -7925.0, -7916.67, -7933.33, -7975.0, -7958.33, -7941.67, -7925.0, -7966.67, -7900.0, -7966.67, -7950.0, -7958.33, -7925.0, -7966.67, -7933.33, -7958.33, -7958.33, -7941.67, -7941.67, -7950.0, -7925.0, -7966.67, -7941.67, -7941.67, -7933.33, -7950.0], [-7975.0, -7900.0, -7958.33, -7933.33, -7941.67, -7916.67, -7941.67, -7908.33, -7950.0, -7933.33, -7916.67, -7933.33, -7941.67, -7933.33, -7941.67, -7925.0, -7958.33, -7916.67, -7950.0, -7925.0, -7900.0, -8016.67, -7858.33, -7975.0, -7916.67, -7950.0, -7933.33, -7916.67, -7941.67, -7941.67, -7925.0, -7941.67, -7933.33, -7908.33, -7925.0, -7941.67, -7925.0, -7941.67, -7950.0, -7941.67, -7908.33, -7916.67, -7933.33, -7958.33, -7900.0, -7950.0, -7941.67, -7933.33, -7950.0, -7925.0, -7933.33, -7916.67, -7941.67, -7908.33, -7958.33, -7933.33, -7933.33, -7900.0, -7925.0, -7925.0, -7933.33, -7925.0, -7908.33, -8000.0, -7841.67, -7950.0, -7925.0, -7891.67, -7983.33, -7875.0, -7950.0, -7916.67, -7916.67, -7941.67, -7916.67, -7916.67, -7925.0, -7925.0, -7925.0, -7925.0, -7916.67, -7925.0, -7916.67, -7916.67, -7916.67, -7925.0, -7925.0, -7916.67, -7900.0, -7933.33, -7925.0, -7933.33, -7916.67, -7916.67, -7925.0, -7916.67, -7908.33, -7916.67, -7916.67, -7916.67, -7925.0, -7933.33, -7891.67, -7925.0, -7925.0, -7916.67, -7916.67, -7908.33, -7916.67, -7925.0, -7916.67, -7916.67, -7925.0, -7916.67, -7916.67, -7950.0, -7908.33, -7925.0, -7916.67, -7925.0, -7916.67, -7925.0, -7933.33, -7900.0, -7925.0, -7925.0, -7908.33, -7933.33, -7908.33, -7908.33, -7933.33, -7900.0, -7916.67, -7925.0, -7916.67, -7908.33, -7925.0, -7916.67, -7916.67, -7916.67, -7933.33, -7908.33, -7916.67, -7925.0, -7925.0, -7925.0, -7916.67, -7908.33, -7908.33, -7916.67, -7891.67, -7925.0, -7925.0, -7908.33, -7916.67, -7916.67, -7908.33, -7908.33, -7908.33, -7925.0, -7916.67, -7908.33, -7933.33, -7908.33, -7916.67, -7925.0, -7925.0, -7883.33, -7933.33, -7916.67, -7883.33, -7925.0, -7925.0, -7891.67, -7933.33, -7916.67, -7900.0, -7950.0, -7900.0, -7916.67, -7925.0, -7908.33, -7933.33, -7900.0, -7908.33, -7916.67, -7916.67, -7925.0, -7916.67, -7908.33, -7875.0, -7933.33, -7891.67, -7916.67, -7925.0, -7883.33, -7933.33, -7900.0, -7916.67, -7900.0], [-7908.33, -7891.67, -7900.0, -7900.0, -7908.33, -7908.33, -7916.67, -7908.33, -7908.33, -7916.67, -7908.33, -7908.33, -7908.33, -7925.0, -7900.0, -7916.67, -7916.67, -7891.67, -7908.33, -7908.33, -7916.67, -7916.67, -7908.33, -7908.33, -7908.33, -7900.0, -7908.33, -7908.33, -7908.33, -7908.33, -7916.67, -7908.33, -7891.67, -7925.0, -7900.0, -7891.67, -7916.67, -7908.33, -7908.33, -7891.67, -7900.0, -7900.0, -7891.67, -7933.33, -7900.0, -7908.33, -7908.33, -7908.33, -7916.67, -7908.33, -7908.33, -7900.0, -7900.0, -7908.33, -7916.67, -7900.0, -7900.0, -7908.33, -7908.33, -7908.33, -7908.33, -7900.0, -7916.67, -7891.67, -7900.0, -7908.33, -7900.0, -7908.33, -7891.67, -7908.33, -7916.67, -7883.33, -7908.33, -7891.67, -7916.67, -7908.33, -7875.0, -7958.33, -7900.0, -7891.67, -7933.33, -7891.67, -7900.0, -7933.33, -7900.0, -7875.0, -7908.33, -7900.0, -7908.33, -7908.33, -7908.33, -7891.67, -7900.0, -7900.0, -7883.33, -7900.0, -7900.0, -7900.0, -7908.33, -7891.67, -7900.0, -7883.33, -7900.0, -7908.33, -7900.0, -7900.0, -7900.0, -7916.67, -7891.67, -7900.0, -7908.33, -7908.33, -7891.67, -7908.33, -7900.0, -7891.67, -7900.0, -7908.33, -7900.0, -7891.67, -7891.67, -7891.67, -7900.0, -7883.33, -7916.67, -7891.67, -7900.0, -7891.67, -7908.33, -7891.67, -7891.67, -7891.67, -7900.0, -7883.33, -7900.0, -7891.67, -7891.67, -7891.67, -7916.67, -7875.0, -7900.0, -7908.33, -7891.67, -7891.67, -7900.0, -7891.67, -7891.67, -7900.0, -7866.67, -7875.0, -7916.67, -7916.67, -7875.0, -7891.67, -7908.33, -7900.0, -7900.0, -7891.67, -7883.33, -7891.67, -7925.0, -7883.33, -7900.0, -7900.0, -7891.67, -7908.33, -7875.0, -7900.0, -7908.33, -7883.33, -7875.0, -7891.67, -7866.67, -7891.67, -7891.67, -7891.67, -7883.33, -7875.0, -7908.33, -7900.0, -7891.67, -7916.67, -7875.0, -7866.67, -7883.33, -7883.33, -7875.0, -7883.33, -7858.33, -7891.67, -7883.33, -7858.33, -7916.67, -7866.67, -7908.33, -7883.33, -7875.0, -7866.67, -7866.67, -7883.33], [-7891.67, -7866.67, -7891.67, -7875.0, -7875.0, -7866.67, -7891.67, -7925.0, -7825.0, -7900.0, -7925.0, -7900.0, -7833.33, -7941.67, -7866.67, -7850.0, -7908.33, -7850.0, -7916.67, -7816.67, -7883.33, -7891.67, -7875.0, -7875.0, -7925.0, -7816.67, -7891.67, -7875.0, -7841.67, -7908.33, -7850.0, -7866.67, -7858.33, -7875.0, -7891.67, -7866.67, -7883.33, -7866.67, -7883.33, -7883.33, -7875.0, -7875.0, -7875.0, -7875.0, -7875.0, -7866.67, -7866.67, -7875.0, -7875.0, -7866.67, -7875.0, -7875.0, -7866.67, -7875.0, -7850.0, -7875.0, -7875.0, -7866.67, -7875.0, -7850.0, -7883.33, -7866.67, -7875.0, -7858.33, -7866.67, -7850.0, -7883.33, -7866.67, -7866.67, -7858.33, -7875.0, -7866.67, -7875.0, -7875.0, -7858.33, -7883.33, -7883.33, -7841.67, -7858.33, -7866.67, -7850.0, -7866.67, -7866.67, -7858.33, -7841.67, -7866.67, -7875.0, -7866.67, -7866.67, -7866.67, -7866.67, -7875.0, -7866.67, -7866.67, -7866.67, -7866.67, -7875.0, -7875.0, -7858.33, -7858.33, -7858.33, -7866.67, -7866.67, -7866.67, -7866.67, -7858.33, -7858.33, -7866.67, -7875.0, -7858.33, -7858.33, -7866.67, -7858.33, -7858.33, -7850.0, -7866.67, -7858.33, -7883.33, -7866.67, -7850.0, -7858.33, -7858.33, -7850.0, -7875.0, -7866.67, -7866.67, -7866.67, -7858.33, -7866.67, -7875.0, -7858.33, -7858.33, -7858.33, -7875.0, -7875.0, -7875.0, -7858.33, -7875.0, -7850.0, -7866.67, -7866.67, -7841.67, -7875.0, -7841.67, -7866.67, -7866.67, -7875.0, -7841.67, -7875.0, -7883.33, -7858.33, -7841.67, -7875.0, -7808.33, -7883.33, -7866.67, -7850.0, -7866.67, -7858.33, -7891.67, -7883.33, -7858.33, -7833.33, -7841.67, -7858.33, -7866.67, -7891.67, -7858.33, -7825.0, -7875.0, -7900.0, -7850.0, -7900.0, -7875.0, -7850.0, -7866.67, -7858.33, -7875.0, -7858.33, -7883.33, -7883.33, -7850.0, -7825.0, -7825.0, -7858.33, -7841.67, -7850.0, -7841.67, -7866.67, -7850.0, -7850.0, -7833.33, -7833.33, -7841.67, -7850.0, -7841.67, -7858.33, -7850.0, -7833.33, -7841.67], [-7841.67, -7833.33, -7850.0, -7858.33, -7858.33, -7833.33, -7841.67, -7850.0, -7833.33, -7833.33, -7833.33, -7850.0, -7841.67, -7850.0, -7833.33, -7841.67, -7841.67, -7850.0, -7841.67, -7850.0, -7850.0, -7833.33, -7833.33, -7850.0, -7850.0, -7841.67, -7850.0, -7850.0, -7833.33, -7833.33, -7883.33, -7858.33, -7841.67, -7866.67, -7841.67, -7850.0, -7858.33, -7858.33, -7858.33, -7875.0, -7833.33, -7850.0, -7833.33, -7850.0, -7850.0, -7850.0, -7858.33, -7825.0, -7841.67, -7858.33, -7833.33, -7841.67, -7850.0, -7850.0, -7800.0, -7866.67, -7875.0, -7775.0, -7841.67, -7850.0, -7800.0, -7866.67, -7841.67, -7850.0, -7850.0, -7833.33, -7850.0, -7850.0, -7841.67, -7850.0, -7841.67, -7841.67, -7858.33, -7841.67, -7850.0, -7833.33, -7850.0, -7841.67, -7841.67, -7866.67, -7833.33, -7825.0, -7841.67, -7841.67, -7850.0, -7850.0, -7833.33, -7841.67, -7833.33, -7825.0, -7825.0, -7841.67, -7858.33, -7858.33, -7833.33, -7825.0, -7850.0, -7833.33, -7833.33, -7858.33, -7833.33, -7833.33, -7833.33, -7858.33, -7833.33, -7850.0, -7816.67, -7858.33, -7825.0, -7850.0, -7833.33, -7841.67, -7841.67, -7850.0, -7825.0, -7841.67, -7825.0, -7850.0, -7850.0, -7841.67, -7850.0, -7841.67, -7841.67, -7841.67, -7833.33, -7850.0, -7850.0, -7850.0, -7833.33, -7833.33, -7858.33, -7833.33, -7850.0, -7841.67, -7841.67, -7833.33, -7833.33, -7833.33, -7841.67, -7833.33, -7825.0, -7858.33, -7850.0, -7833.33, -7833.33, -7841.67, -7850.0, -7825.0, -7841.67, -7841.67, -7833.33, -7850.0, -7825.0, -7850.0, -7841.67, -7833.33, -7825.0, -7833.33, -7833.33, -7841.67, -7833.33, -7850.0, -7841.67, -7841.67, -7833.33, -7841.67, -7833.33, -7833.33, -7841.67, -7841.67, -7833.33, -7833.33, -7833.33, -7841.67, -7841.67, -7825.0, -7833.33, -7833.33, -7841.67, -7833.33, -7816.67, -7841.67, -7833.33, -7841.67, -7833.33, -7833.33, -7833.33, -7841.67, -7833.33, -7833.33, -7833.33, -7833.33, -7833.33, -7833.33, -7841.67, -7833.33, -7841.67, -7841.67, -7841.67, -7833.33], [-7833.33, -7866.67, -7808.33, -7858.33, -7833.33, -7825.0, -7841.67, -7841.67, -7833.33, -7833.33, -7825.0, -7833.33, -7825.0, -7841.67, -7833.33, -7833.33, -7850.0, -7833.33, -7816.67, -7816.67, -7841.67, -7841.67, -7833.33, -7833.33, -7841.67, -7825.0, -7833.33, -7833.33, -7833.33, -7833.33, -7816.67, -7841.67, -7800.0, -7866.67, -7800.0, -7858.33, -7825.0, -7825.0, -7841.67, -7833.33, -7841.67, -7833.33, -7833.33, -7850.0, -7825.0, -7841.67, -7833.33, -7866.67, -7808.33, -7891.67, -7783.33, -7825.0, -7866.67, -7783.33, -7875.0, -7816.67, -7833.33, -7825.0, -7808.33, -7825.0, -7833.33, -7833.33, -7833.33, -7841.67, -7841.67, -7825.0, -7833.33, -7841.67, -7833.33, -7833.33, -7825.0, -7833.33, -7825.0, -7825.0, -7816.67, -7825.0, -7825.0, -7833.33, -7858.33, -7816.67, -7841.67, -7825.0, -7833.33, -7825.0, -7833.33, -7825.0, -7833.33, -7825.0, -7833.33, -7816.67, -7833.33, -7825.0, -7833.33, -7841.67, -7833.33, -7833.33, -7833.33, -7841.67, -7816.67, -7850.0, -7841.67, -7858.33, -7858.33, -7841.67, -7858.33, -7800.0, -7841.67, -7816.67, -7850.0, -7833.33, -7850.0, -7833.33, -7833.33, -7841.67, -7841.67, -7825.0, -7808.33, -7833.33, -7833.33, -7833.33, -7825.0, -7858.33, -7825.0, -7825.0, -7833.33, -7800.0, -7858.33, -7800.0, -7816.67, -7825.0, -7816.67, -7833.33, -7816.67, -7808.33, -7816.67, -7825.0, -7808.33, -7858.33, -7825.0, -7850.0, -7858.33, -7841.67, -7858.33, -7825.0, -7783.33, -7791.67, -7950.0, -7808.33, -7858.33, -7858.33, -7800.0, -7850.0, -7833.33, -7841.67, -7816.67, -7791.67, -7766.67, -7825.0, -7783.33, -7791.67, -7783.33, -7783.33, -7783.33, -7791.67, -7783.33, -7791.67, -7800.0, -7783.33, -7758.33, -7766.67, -7775.0, -7791.67, -7766.67, -7733.33, -7783.33, -7758.33, -7783.33, -7766.67, -7775.0, -7758.33, -7775.0, -7775.0, -7766.67, -7791.67, -7766.67, -7791.67, -7775.0, -7791.67, -7775.0, -7800.0, -7775.0, -7783.33, -7791.67, -7766.67, -7800.0, -7783.33, -7775.0, -7783.33, -7783.33, -7766.67], [-7775.0, -7783.33, -7758.33, -7766.67, -7783.33, -7783.33, -7791.67, -7791.67, -7758.33, -7775.0, -7791.67, -7775.0, -7791.67, -7775.0, -7766.67, -7775.0, -7800.0, -7783.33, -7775.0, -7766.67, -7775.0, -7783.33, -7783.33, -7775.0, -7766.67, -7783.33, -7775.0, -7766.67, -7791.67, -7775.0, -7775.0, -7800.0, -7775.0, -7800.0, -7741.67, -7783.33, -7783.33, -7766.67, -7775.0, -7775.0, -7800.0, -7766.67, -7750.0, -7791.67, -7783.33, -7783.33, -7775.0, -7783.33, -7750.0, -7783.33, -7800.0, -7775.0, -7775.0, -7783.33, -7766.67, -7800.0, -7775.0, -7775.0, -7783.33, -7783.33, -7783.33, -7775.0, -7783.33, -7758.33, -7783.33, -7783.33, -7775.0, -7766.67, -7766.67, -7766.67, -7775.0, -7783.33, -7775.0, -7783.33, -7791.67, -7775.0, -7766.67, -7783.33, -7775.0, -7775.0, -7758.33, -7750.0, -7791.67, -7791.67, -7775.0, -7783.33, -7783.33, -7783.33, -7783.33, -7800.0, -7741.67, -7791.67, -7783.33, -7783.33, -7766.67, -7791.67, -7808.33, -7791.67, -7775.0, -7775.0, -7783.33, -7783.33, -7758.33, -7775.0, -7783.33, -7750.0, -7783.33, -7791.67, -7783.33, -7741.67, -7758.33, -7775.0, -7758.33, -7750.0, -7783.33, -7775.0, -7791.67, -7775.0, -7800.0, -7775.0, -7800.0, -7783.33, -7775.0, -7775.0, -7783.33, -7766.67, -7775.0, -7775.0, -7775.0, -7766.67, -7775.0, -7775.0, -7775.0, -7775.0, -7775.0, -7766.67, -7775.0, -7775.0, -7783.33, -7766.67, -7766.67, -7791.67, -7775.0, -7783.33, -7783.33, -7783.33, -7783.33, -7775.0, -7808.33, -7783.33, -7758.33, -7766.67, -7775.0, -7783.33, -7800.0, -7775.0, -7775.0, -7775.0, -7775.0, -7775.0, -7783.33, -7783.33, -7766.67, -7791.67, -7816.67, -7783.33, -7783.33, -7750.0, -7791.67, -7766.67, -7766.67, -7766.67, -7775.0, -7775.0, -7783.33, -7775.0, -7758.33, -7775.0, -7775.0, -7783.33, -7750.0, -7775.0, -7775.0, -7758.33, -7775.0, -7775.0, -7775.0, -7775.0, -7783.33, -7783.33, -7766.67, -7775.0, -7783.33, -7775.0, -7791.67, -7766.67, -7766.67, -7783.33, -7766.67, -7775.0], [-7783.33, -7791.67, -7791.67, -7775.0, -7775.0, -7800.0, -7791.67, -7775.0, -7783.33, -7791.67, -7783.33, -7783.33, -7758.33, -7791.67, -7775.0, -7750.0, -7775.0, -7758.33, -7758.33, -7775.0, -7775.0, -7766.67, -7766.67, -7775.0, -7766.67, -7791.67, -7766.67, -7766.67, -7766.67, -7775.0, -7766.67, -7775.0, -7775.0, -7766.67, -7766.67, -7775.0, -7775.0, -7775.0, -7791.67, -7775.0, -7775.0, -7766.67, -7775.0, -7766.67, -7766.67, -7775.0, -7766.67, -7791.67, -7750.0, -7750.0, -7808.33, -7750.0, -7766.67, -7766.67, -7775.0, -7758.33, -7775.0, -7808.33, -7725.0, -7791.67, -7783.33, -7725.0, -7808.33, -7741.67, -7758.33, -7808.33, -7733.33, -7808.33, -7733.33, -7741.67, -7850.0, -7725.0, -7775.0, -7825.0, -7683.33, -7825.0, -7766.67, -7700.0, -7825.0, -7733.33, -7766.67, -7766.67, -7766.67, -7758.33, -7766.67, -7750.0, -7750.0, -7766.67, -7758.33, -7758.33, -7766.67, -7750.0, -7750.0, -7758.33, -7766.67, -7758.33, -7758.33, -7750.0, -7766.67, -7758.33, -7775.0, -7766.67, -7750.0, -7766.67, -7750.0, -7750.0, -7766.67, -7750.0, -7750.0, -7758.33, -7758.33, -7758.33, -7741.67, -7758.33, -7750.0, -7741.67, -7750.0, -7758.33, -7741.67, -7758.33, -7750.0, -7750.0, -7758.33, -7758.33, -7758.33, -7758.33, -7741.67, -7758.33, -7758.33, -7750.0, -7758.33, -7766.67, -7750.0, -7733.33, -7758.33, -7741.67, -7750.0, -7741.67, -7758.33, -7750.0, -7766.67, -7716.67, -7741.67, -7716.67, -7750.0, -7733.33, -7750.0, -7775.0, -7733.33, -7750.0, -7750.0, -7766.67, -7758.33, -7741.67, -7758.33, -7750.0, -7766.67, -7750.0, -7766.67, -7750.0, -7758.33, -7750.0, -7766.67, -7758.33, -7758.33, -7750.0, -7758.33, -7758.33, -7766.67, -7766.67, -7766.67, -7750.0, -7758.33, -7758.33, -7758.33, -7758.33, -7766.67, -7750.0, -7750.0, -7758.33, -7750.0, -7758.33, -7758.33, -7783.33, -7741.67, -7750.0, -7750.0, -7750.0, -7750.0, -7750.0, -7750.0, -7750.0, -7741.67, -7750.0, -7766.67, -7750.0, -7741.67, -7750.0, -7750.0, -7750.0], [-7750.0, -7741.67, -7750.0, -7725.0, -7741.67, -7733.33, -7750.0, -7741.67, -7733.33, -7750.0, -7733.33, -7733.33, -7716.67, -7775.0, -7733.33, -7750.0, -7725.0, -7741.67, -7766.67, -7716.67, -7750.0, -7741.67, -7725.0, -7716.67, -7750.0, -7758.33, -7750.0, -7766.67, -7733.33, -7750.0, -7750.0, -7758.33, -7733.33, -7741.67, -7741.67, -7708.33, -7758.33, -7741.67, -7758.33, -7741.67, -7733.33, -7741.67, -7750.0, -7750.0, -7758.33, -7741.67, -7750.0, -7750.0, -7775.0, -7716.67, -7733.33, -7741.67, -7733.33, -7750.0, -7758.33, -7733.33, -7750.0, -7750.0, -7733.33, -7750.0, -7750.0, -7750.0, -7733.33, -7766.67, -7758.33, -7733.33, -7733.33, -7733.33, -7725.0, -7725.0, -7733.33, -7741.67, -7733.33, -7766.67, -7725.0, -7733.33, -7741.67, -7733.33, -7733.33, -7733.33, -7750.0, -7733.33, -7733.33, -7750.0, -7758.33, -7725.0, -7750.0, -7716.67, -7741.67, -7725.0, -7733.33, -7733.33, -7741.67, -7733.33, -7733.33, -7716.67, -7733.33, -7733.33, -7725.0, -7750.0, -7725.0, -7733.33, -7716.67, -7741.67, -7716.67, -7716.67, -7733.33, -7741.67, -7733.33, -7733.33, -7741.67, -7741.67, -7733.33, -7725.0, -7733.33, -7733.33, -7750.0, -7733.33, -7725.0, -7741.67, -7733.33, -7725.0, -7733.33, -7733.33, -7716.67, -7725.0, -7733.33, -7741.67, -7733.33, -7741.67, -7733.33, -7741.67, -7733.33, -7733.33, -7725.0, -7733.33, -7741.67, -7716.67, -7725.0, -7733.33, -7733.33, -7733.33, -7733.33, -7725.0, -7725.0, -7741.67, -7725.0, -7725.0, -7733.33, -7716.67, -7733.33, -7716.67, -7733.33, -7733.33, -7733.33, -7700.0, -7733.33, -7716.67, -7725.0, -7708.33, -7733.33, -7725.0, -7725.0, -7725.0, -7733.33, -7733.33, -7733.33, -7725.0, -7733.33, -7733.33, -7716.67, -7750.0, -7725.0, -7733.33, -7733.33, -7750.0, -7725.0, -7741.67, -7733.33, -7758.33, -7733.33, -7725.0, -7758.33, -7766.67, -7725.0, -7750.0, -7708.33, -7716.67, -7725.0, -7741.67, -7750.0, -7733.33, -7741.67, -7733.33, -7741.67, -7758.33, -7758.33, -7758.33, -7741.67, -7741.67], [-7741.67, -7725.0, -7750.0, -7750.0, -7750.0, -7741.67, -7750.0, -7716.67, -7741.67, -7750.0, -7750.0, -7750.0, -7750.0, -7750.0, -7741.67, -7700.0, -7766.67, -7750.0, -7750.0, -7750.0, -7741.67, -7741.67, -7750.0, -7758.33, -7758.33, -7766.67, -7750.0, -7775.0, -7766.67, -7758.33, -7716.67, -7733.33, -7775.0, -7750.0, -7775.0, -7741.67, -7733.33, -7750.0, -7750.0, -7766.67, -7750.0, -7783.33, -7741.67, -7708.33, -7758.33, -7741.67, -7791.67, -7733.33, -7750.0, -7741.67, -7750.0, -7733.33, -7750.0, -7733.33, -7741.67, -7750.0, -7750.0, -7733.33, -7750.0, -7750.0, -7741.67, -7750.0, -7750.0, -7741.67, -7758.33, -7733.33, -7741.67, -7783.33, -7733.33, -7750.0, -7750.0, -7758.33, -7766.67, -7741.67, -7766.67, -7766.67, -7766.67, -7741.67, -7775.0, -7741.67, -7766.67, -7741.67, -7716.67, -7850.0, -7691.67, -7733.33, -7866.67, -7691.67, -7700.0, -7841.67, -7675.0, -7766.67, -7758.33, -7683.33, -7850.0, -7708.33, -7708.33, -7783.33, -7708.33, -7791.67, -7725.0, -7741.67, -7766.67, -7725.0, -7725.0, -7775.0, -7733.33, -7750.0, -7725.0, -7758.33, -7733.33, -7750.0, -7733.33, -7741.67, -7741.67, -7750.0, -7733.33, -7750.0, -7741.67, -7733.33, -7733.33, -7741.67, -7741.67, -7741.67, -7733.33, -7750.0, -7741.67, -7741.67, -7750.0, -7741.67, -7750.0, -7741.67, -7733.33, -7725.0, -7733.33, -7750.0, -7733.33, -7741.67, -7741.67, -7741.67, -7733.33, -7758.33, -7741.67, -7733.33, -7716.67, -7733.33, -7733.33, -7733.33, -7741.67, -7733.33, -7733.33, -7741.67, -7750.0, -7716.67, -7708.33, -7716.67, -7725.0, -7700.0, -7708.33, -7716.67, -7725.0, -7716.67, -7716.67, -7741.67, -7725.0, -7716.67, -7716.67, -7708.33, -7683.33, -7741.67, -7708.33, -7741.67, -7708.33, -7725.0, -7733.33, -7716.67, -7725.0, -7741.67, -7716.67, -7725.0, -7725.0, -7725.0, -7708.33, -7716.67, -7725.0, -7716.67, -7716.67, -7716.67, -7725.0, -7716.67, -7716.67, -7725.0, -7725.0, -7708.33, -7716.67, -7725.0, -7716.67, -7716.67, -7725.0, -7716.67], [-7725.0, -7716.67, -7708.33, -7783.33, -7833.33, -7700.0, -7733.33, -7733.33, -7708.33, -7650.0, -7775.0, -7783.33, -7766.67, -7775.0, -7633.33, -7725.0, -7791.67, -7725.0, -7766.67, -7733.33, -7733.33, -7700.0, -7733.33, -7716.67, -7716.67, -7691.67, -7700.0, -7708.33, -7700.0, -7700.0, -7691.67, -7708.33, -7708.33, -7683.33, -7700.0, -7700.0, -7691.67, -7708.33, -7691.67, -7691.67, -7691.67, -7700.0, -7700.0, -7683.33, -7700.0, -7700.0, -7708.33, -7683.33, -7691.67, -7700.0, -7691.67, -7725.0, -7691.67, -7700.0, -7691.67, -7691.67, -7691.67, -7691.67, -7691.67, -7700.0, -7691.67, -7691.67, -7675.0, -7700.0, -7700.0, -7691.67, -7691.67, -7700.0, -7700.0, -7691.67, -7783.33, -7641.67, -7758.33, -7658.33, -7708.33, -7775.0, -7633.33, -7758.33, -7691.67, -7766.67, -7658.33, -7766.67, -7666.67, -7750.0, -7716.67, -7700.0, -7691.67, -7675.0, -7675.0, -7783.33, -7650.0, -7733.33, -7683.33, -7700.0, -7691.67, -7700.0, -7700.0, -7700.0, -7708.33, -7700.0, -7691.67, -7691.67, -7700.0, -7683.33, -7700.0, -7691.67, -7700.0, -7700.0, -7691.67, -7691.67, -7691.67, -7691.67, -7700.0, -7691.67, -7700.0, -7700.0, -7700.0, -7700.0, -7700.0, -7691.67, -7708.33, -7700.0, -7708.33, -7700.0, -7691.67, -7691.67, -7700.0, -7700.0, -7716.67, -7691.67, -7708.33, -7700.0, -7691.67, -7700.0, -7700.0, -7691.67, -7708.33, -7700.0, -7708.33, -7691.67, -7691.67, -7683.33, -7708.33, -7691.67, -7691.67, -7691.67, -7700.0, -7741.67, -7675.0, -7783.33, -7600.0, -7708.33, -7575.0, -7675.0, -7666.67, -7558.33, -7866.67, -7516.67, -8016.67, -7450.0, -7883.33, -7575.0, -7658.33, -7825.0, -7491.67, -8033.33, -7450.0, -7933.33, -7525.0, -7891.67, -7633.33, -7716.67, -7691.67, -7658.33, -7666.67, -7666.67, -7708.33, -7666.67, -7666.67, -7800.0, -7658.33, -7675.0, -7750.0, -7600.0, -7725.0, -7683.33, -7666.67, -7683.33, -7750.0, -7675.0, -7683.33, -7733.33, -7658.33, -7741.67, -7733.33, -7700.0, -7666.67, -7708.33, -7733.33, -7666.67], [-7675.0, -7708.33, -7683.33, -7683.33, -7741.67, -7625.0, -7725.0, -7716.67, -7625.0, -7716.67, -7700.0, -7675.0, -7766.67, -7650.0, -7783.33, -7650.0, -7725.0, -7708.33, -7675.0, -7725.0, -7683.33, -7683.33, -7700.0, -7691.67, -7708.33, -7691.67, -7683.33, -7683.33, -7716.67, -7691.67, -7716.67, -7675.0, -7691.67, -7683.33, -7700.0, -7633.33, -7708.33, -7691.67, -7708.33, -7633.33, -7741.67, -7625.0, -7816.67, -7591.67, -7683.33, -7633.33, -7675.0, -7716.67, -7683.33, -7833.33, -7591.67, -7833.33, -7641.67, -7766.67, -7641.67, -7783.33, -7608.33, -7800.0, -7583.33, -7766.67, -7641.67, -7750.0, -7650.0, -7658.33, -7808.33, -7550.0, -7891.67, -7550.0, -7783.33, -7633.33, -7708.33, -7650.0, -7733.33, -7666.67, -7691.67, -7725.0, -7666.67, -7700.0, -7675.0, -7691.67, -7683.33, -7683.33, -7650.0, -7691.67, -7675.0, -7716.67, -7650.0, -7691.67, -7666.67, -7658.33, -7700.0, -7666.67, -7683.33, -7658.33, -7683.33, -7641.67, -7675.0, -7658.33, -7658.33, -7658.33, -7666.67, -7650.0, -7675.0, -7658.33, -7641.67, -7666.67, -7666.67, -7666.67, -7658.33, -7675.0, -7683.33, -7658.33, -7650.0, -7666.67, -7658.33, -7641.67, -7700.0, -7675.0, -7675.0, -7666.67, -7675.0, -7650.0, -7675.0, -7675.0, -7675.0, -7683.33, -7691.67, -7675.0, -7683.33, -7683.33, -7650.0, -7658.33, -7666.67, -7683.33, -7675.0, -7666.67, -7675.0, -7666.67, -7683.33, -7683.33, -7650.0, -7700.0, -7675.0, -7675.0, -7683.33, -7691.67, -7683.33, -7675.0, -7650.0, -7633.33, -7733.33, -7616.67, -7841.67, -7541.67, -7841.67, -7583.33, -7825.0, -7591.67, -7833.33, -7566.67, -7733.33, -7658.33, -7675.0, -7666.67, -7683.33, -7683.33, -7675.0, -7666.67, -7658.33, -7675.0, -7708.33, -7666.67, -7675.0, -7683.33, -7650.0, -7675.0, -7666.67, -7675.0, -7666.67, -7658.33, -7683.33, -7641.67, -7658.33, -7741.67, -7591.67, -7650.0, -7650.0, -7625.0, -7733.33, -7591.67, -7725.0, -7608.33, -7750.0, -7658.33, -7641.67, -7716.67, -7650.0, -7691.67, -7666.67, -7675.0], [-7683.33, -7683.33, -7666.67, -7675.0, -7666.67, -7666.67, -7683.33, -7675.0, -7666.67, -7683.33, -7683.33, -7666.67, -7683.33, -7666.67, -7666.67, -7675.0, -7675.0, -7683.33, -7691.67, -7683.33, -7675.0, -7666.67, -7691.67, -7683.33, -7650.0, -7700.0, -7683.33, -7683.33, -7675.0, -7683.33, -7691.67, -7708.33, -7675.0, -7691.67, -7683.33, -7691.67, -7641.67, -7683.33, -7675.0, -7700.0, -7708.33, -7650.0, -7716.67, -7675.0, -7700.0, -7658.33, -7700.0, -7725.0, -7691.67, -7691.67, -7741.67, -7700.0, -7675.0, -7733.33, -7733.33, -7741.67, -7641.67, -7691.67, -7666.67, -7733.33, -7683.33, -7725.0, -7641.67, -7700.0, -7691.67, -7708.33, -7733.33, -7675.0, -7708.33, -7700.0, -7708.33, -7675.0, -7691.67, -7700.0, -7708.33, -7658.33, -7716.67, -7683.33, -7708.33, -7666.67, -7716.67, -7683.33, -7700.0, -7741.67, -7633.33, -7725.0, -7666.67, -7691.67, -7691.67, -7691.67, -7675.0, -7675.0, -7683.33, -7683.33, -7725.0, -7691.67, -7675.0, -7700.0, -7666.67, -7700.0, -7700.0, -7675.0, -7708.33, -7691.67, -7658.33, -7658.33, -7700.0, -7708.33, -7675.0, -7708.33, -7725.0, -7700.0, -7650.0, -7716.67, -7683.33, -7650.0, -7700.0, -7700.0, -7691.67, -7700.0, -7633.33, -7716.67, -7675.0, -7725.0, -7650.0, -7683.33, -7708.33, -7683.33, -7700.0, -7666.67, -7700.0, -7725.0, -7591.67, -7716.67, -7708.33, -7625.0, -7716.67, -7641.67, -7716.67, -7708.33, -7650.0, -7708.33, -7658.33, -7741.67, -7691.67, -7675.0, -7733.33, -7641.67, -7675.0, -7725.0, -7650.0, -7658.33, -7675.0, -7700.0, -7700.0, -7683.33, -7658.33, -7708.33, -7683.33, -7658.33, -7716.67, -7700.0, -7658.33, -7725.0, -7691.67, -7666.67, -7725.0, -7658.33, -7700.0, -7650.0, -7716.67, -7708.33, -7608.33, -7725.0, -7675.0, -7658.33, -7700.0, -7683.33, -7700.0, -7675.0, -7716.67, -7658.33, -7691.67, -7708.33, -7675.0, -7750.0, -7675.0, -7591.67, -7683.33, -7691.67, -7666.67, -7625.0, -7708.33, -7658.33, -7666.67, -7658.33, -7716.67, -7583.33, -7725.0, -7675.0], [-7650.0, -7666.67, -7650.0, -7658.33, -7650.0, -7658.33, -7666.67, -7675.0, -7658.33, -7658.33, -7658.33, -7683.33, -7666.67, -7666.67, -7666.67, -7666.67, -7650.0, -7666.67, -7658.33, -7666.67, -7650.0, -7675.0, -7658.33, -7650.0, -7666.67, -7650.0, -7658.33, -7658.33, -7683.33, -7658.33, -7650.0, -7666.67, -7658.33, -7658.33, -7641.67, -7650.0, -7675.0, -7666.67, -7666.67, -7650.0, -7666.67, -7666.67, -7691.67, -7666.67, -7658.33, -7650.0, -7675.0, -7666.67, -7658.33, -7675.0, -7666.67, -7658.33, -7658.33, -7666.67, -7666.67, -7700.0, -7650.0, -7683.33, -7650.0, -7641.67, -7625.0, -7641.67, -7658.33, -7633.33, -7733.33, -7625.0, -7641.67, -7691.67, -7600.0, -7725.0, -7591.67, -7683.33, -7658.33, -7675.0, -7675.0, -7725.0, -7650.0, -7708.33, -7666.67, -7683.33, -7641.67, -7658.33, -7675.0, -7666.67, -7658.33, -7666.67, -7683.33, -7650.0, -7683.33, -7666.67, -7633.33, -7733.33, -7608.33, -7683.33, -7675.0, -7683.33, -7666.67, -7633.33, -7666.67, -7683.33, -7658.33, -7641.67, -7675.0, -7700.0, -7650.0, -7683.33, -7683.33, -7683.33, -7683.33, -7666.67, -7683.33, -7658.33, -7675.0, -7708.33, -7691.67, -7683.33, -7633.33, -7675.0, -7683.33, -7658.33, -7675.0, -7683.33, -7666.67, -7683.33, -7658.33, -7666.67, -7675.0, -7658.33, -7675.0, -7666.67, -7683.33, -7658.33, -7675.0, -7641.67, -7683.33, -7691.67, -7666.67, -7650.0, -7633.33, -7716.67, -7666.67, -7683.33, -7650.0, -7683.33, -7650.0, -7658.33, -7683.33, -7675.0, -7666.67, -7691.67, -7616.67, -7658.33, -7608.33, -7683.33, -7650.0, -7650.0, -7666.67, -7658.33, -7691.67, -7708.33, -7650.0, -7666.67, -7666.67, -7691.67, -7650.0, -7658.33, -7683.33, -7666.67, -7666.67, -7683.33, -7666.67, -7666.67, -7675.0, -7666.67, -7658.33, -7683.33, -7683.33, -7691.67, -7666.67, -7666.67, -7641.67, -7658.33, -7666.67, -7691.67, -7666.67, -7650.0, -7683.33, -7675.0, -7666.67, -7666.67, -7658.33, -7683.33, -7641.67, -7675.0, -7608.33, -7683.33, -7675.0, -7691.67, -7658.33, -7625.0], [-7633.33, -7675.0, -7666.67, -7691.67, -7666.67, -7666.67, -7658.33, -7691.67, -7666.67, -7675.0, -7650.0, -7666.67, -7658.33, -7683.33, -7666.67, -7650.0, -7650.0, -7675.0, -7675.0, -7658.33, -7683.33, -7666.67, -7658.33, -7666.67, -7666.67, -7666.67, -7675.0, -7691.67, -7683.33, -7650.0, -7658.33, -7691.67, -7650.0, -7683.33, -7658.33, -7666.67, -7691.67, -7633.33, -7708.33, -7666.67, -7583.33, -7691.67, -7666.67, -7650.0, -7675.0, -7641.67, -7658.33, -7633.33, -7658.33, -7658.33, -7683.33, -7675.0, -7633.33, -7650.0, -7641.67, -7666.67, -7650.0, -7666.67, -7633.33, -7666.67, -7658.33, -7675.0, -7641.67, -7650.0, -7683.33, -7641.67, -7683.33, -7658.33, -7666.67, -7650.0, -7666.67, -7658.33, -7658.33, -7658.33, -7650.0, -7666.67, -7658.33, -7658.33, -7650.0, -7658.33, -7650.0, -7658.33, -7658.33, -7650.0, -7658.33, -7633.33, -7691.67, -7633.33, -7633.33, -7683.33, -7650.0, -7658.33, -7658.33, -7650.0, -7658.33, -7658.33, -7608.33, -7675.0, -7625.0, -7658.33, -7541.67, -7758.33, -7591.67, -7683.33, -7650.0, -7716.67, -7666.67, -7616.67, -7666.67, -7641.67, -7666.67, -7683.33, -7641.67, -7633.33, -7641.67, -7666.67, -7666.67, -7608.33, -7741.67, -7508.33, -7641.67, -7733.33, -7608.33, -7750.0, -7575.0, -7666.67, -7625.0, -7666.67, -7633.33, -7666.67, -7708.33, -7625.0, -7658.33, -7658.33, -7633.33, -7625.0, -7708.33, -7633.33, -7666.67, -7650.0, -7650.0, -7625.0, -7650.0, -7641.67, -7650.0, -7691.67, -7675.0, -7608.33, -7691.67, -7683.33, -7608.33, -7666.67, -7641.67, -7658.33, -7608.33, -7708.33, -7658.33, -7633.33, -7683.33, -7675.0, -7658.33, -7658.33, -7666.67, -7641.67, -7675.0, -7675.0, -7641.67, -7641.67, -7650.0, -7683.33, -7641.67, -7683.33, -7658.33, -7658.33, -7625.0, -7658.33, -7650.0, -7641.67, -7641.67, -7616.67, -7641.67, -7625.0, -7650.0, -7641.67, -7633.33, -7666.67, -7641.67, -7658.33, -7633.33, -7658.33, -7641.67, -7641.67, -7641.67, -7641.67, -7650.0, -7641.67, -7650.0, -7641.67, -7650.0, -7641.67], [-7633.33, -7633.33, -7641.67, -7658.33, -7616.67, -7658.33, -7616.67, -7633.33, -7633.33, -7633.33, -7625.0, -7641.67, -7650.0, -7641.67, -7625.0, -7641.67, -7641.67, -7633.33, -7633.33, -7641.67, -7658.33, -7633.33, -7666.67, -7641.67, -7641.67, -7641.67, -7658.33, -7625.0, -7650.0, -7650.0, -7641.67, -7641.67, -7633.33, -7633.33, -7691.67, -7691.67, -7641.67, -7641.67, -7650.0, -7650.0, -7683.33, -7666.67, -7650.0, -7650.0, -7650.0, -7658.33, -7708.33, -7633.33, -7650.0, -7641.67, -7658.33, -7525.0, -7666.67, -7633.33, -7666.67, -7616.67, -7641.67, -7625.0, -7658.33, -7650.0, -7633.33, -7633.33, -7641.67, -7625.0, -7633.33, -7633.33, -7641.67, -7625.0, -7633.33, -7658.33, -7641.67, -7608.33, -7625.0, -7658.33, -7583.33, -7658.33, -7683.33, -7625.0, -7641.67, -7633.33, -7658.33, -7633.33, -7641.67, -7625.0, -7616.67, -7650.0, -7625.0, -7650.0, -7608.33, -7658.33, -7633.33, -7641.67, -7633.33, -7641.67, -7625.0, -7625.0, -7641.67, -7616.67, -7641.67, -7641.67, -7641.67, -7641.67, -7641.67, -7625.0, -7658.33, -7650.0, -7625.0, -7658.33, -7633.33, -7658.33, -7633.33, -7675.0, -7658.33, -7616.67, -7650.0, -7641.67, -7641.67, -7633.33, -7650.0, -7625.0, -7633.33, -7641.67, -7633.33, -7675.0, -7625.0, -7608.33, -7633.33, -7633.33, -7650.0, -7641.67, -7625.0, -7641.67, -7666.67, -7633.33, -7625.0, -7633.33, -7633.33, -7616.67, -7658.33, -7650.0, -7633.33, -7666.67, -7675.0, -7641.67, -7658.33, -7650.0, -7641.67, -7641.67, -7650.0, -7641.67, -7650.0, -7641.67, -7633.33, -7650.0, -7650.0, -7650.0, -7675.0, -7625.0, -7658.33, -7666.67, -7633.33, -7625.0, -7658.33, -7633.33, -7500.0, -7608.33, -7625.0, -7741.67, -7633.33, -7633.33, -7625.0, -7633.33, -7658.33, -7616.67, -7641.67, -7641.67, -7650.0, -7633.33, -7650.0, -7625.0, -7641.67, -7641.67, -7641.67, -7641.67, -7641.67, -7641.67, -7633.33, -7641.67, -7633.33, -7650.0, -7641.67, -7641.67, -7658.33, -7641.67, -7650.0, -7633.33, -7641.67, -7650.0, -7641.67, -7650.0], [-7633.33, -7633.33, -7625.0, -7625.0, -7641.67, -7641.67, -7616.67, -7641.67, -7633.33, -7625.0, -7641.67, -7633.33, -7625.0, -7633.33, -7666.67, -7625.0, -7650.0, -7650.0, -7641.67, -7641.67, -7633.33, -7658.33, -7658.33, -7650.0, -7616.67, -7641.67, -7658.33, -7658.33, -7641.67, -7625.0, -7666.67, -7608.33, -7650.0, -7625.0, -7666.67, -7641.67, -7641.67, -7650.0, -7641.67, -7641.67, -7641.67, -7658.33, -7625.0, -7633.33, -7633.33, -7641.67, -7641.67, -7641.67, -7641.67, -7600.0, -7633.33, -7633.33, -7633.33, -7641.67, -7641.67, -7625.0, -7625.0, -7633.33, -7633.33, -7650.0, -7633.33, -7641.67, -7616.67, -7633.33, -7633.33, -7650.0, -7633.33, -7625.0, -7641.67, -7633.33, -7633.33, -7641.67, -7616.67, -7650.0, -7675.0, -7625.0, -7641.67, -7633.33, -7608.33, -7625.0, -7633.33, -7633.33, -7633.33, -7625.0, -7633.33, -7633.33, -7633.33, -7625.0, -7633.33, -7633.33, -7625.0, -7616.67, -7641.67, -7633.33, -7633.33, -7625.0, -7633.33, -7633.33, -7641.67, -7633.33, -7625.0, -7650.0, -7625.0, -7658.33, -7641.67, -7650.0, -7616.67, -7641.67, -7641.67, -7633.33, -7650.0, -7633.33, -7633.33, -7633.33, -7633.33, -7600.0, -7650.0, -7633.33, -7650.0, -7633.33, -7633.33, -7633.33, -7633.33, -7625.0, -7650.0, -7616.67, -7650.0, -7633.33, -7616.67, -7641.67, -7641.67, -7641.67, -7633.33, -7625.0, -7641.67, -7625.0, -7641.67, -7641.67, -7633.33, -7641.67, -7633.33, -7641.67, -7641.67, -7641.67, -7633.33, -7633.33, -7633.33, -7633.33, -7625.0, -7633.33, -7641.67, -7625.0, -7641.67, -7633.33, -7650.0, -7616.67, -7625.0, -7641.67, -7633.33, -7641.67, -7625.0, -7641.67, -7625.0, -7650.0, -7625.0, -7650.0, -7633.33, -7633.33, -7633.33, -7633.33, -7625.0, -7633.33, -7625.0, -7658.33, -7633.33, -7633.33, -7633.33, -7641.67, -7641.67, -7641.67, -7633.33, -7641.67, -7625.0, -7616.67, -7633.33, -7641.67, -7625.0, -7625.0, -7633.33, -7650.0, -7650.0, -7625.0, -7625.0, -7650.0, -7633.33, -7625.0, -7616.67, -7625.0, -7641.67, -7633.33], [-7666.67, -7616.67, -7650.0, -7600.0, -7633.33, -7633.33, -7616.67, -7633.33, -7625.0, -7633.33, -7616.67, -7633.33, -7625.0, -7633.33, -7625.0, -7625.0, -7641.67, -7625.0, -7633.33, -7633.33, -7625.0, -7616.67, -7650.0, -7625.0, -7633.33, -7641.67, -7633.33, -7625.0, -7625.0, -7625.0, -7633.33, -7625.0, -7650.0, -7625.0, -7633.33, -7650.0, -7608.33, -7641.67, -7625.0, -7625.0, -7616.67, -7633.33, -7625.0, -7633.33, -7641.67, -7641.67, -7625.0, -7641.67, -7625.0, -7633.33, -7633.33, -7633.33, -7633.33, -7633.33, -7641.67, -7650.0, -7625.0, -7625.0, -7633.33, -7633.33, -7633.33, -7641.67, -7625.0, -7641.67, -7633.33, -7633.33, -7625.0, -7641.67, -7625.0, -7641.67, -7616.67, -7650.0, -7633.33, -7633.33, -7633.33, -7641.67, -7625.0, -7650.0, -7633.33, -7641.67, -7633.33, -7633.33, -7641.67, -7625.0, -7641.67, -7633.33, -7633.33, -7625.0, -7633.33, -7633.33, -7641.67, -7633.33, -7633.33, -7633.33, -7641.67, -7625.0, -7633.33, -7633.33, -7625.0, -7633.33, -7625.0, -7641.67, -7650.0, -7650.0, -7633.33, -7633.33, -7616.67, -7641.67, -7625.0, -7633.33, -7633.33, -7633.33, -7633.33, -7633.33, -7633.33, -7633.33, -7641.67, -7616.67, -7633.33, -7650.0, -7633.33, -7633.33, -7625.0, -7633.33, -7633.33, -7625.0, -7625.0, -7650.0, -7633.33, -7633.33, -7641.67, -7641.67, -7633.33, -7633.33, -7633.33, -7633.33, -7625.0, -7641.67, -7633.33, -7641.67, -7633.33, -7641.67, -7641.67, -7641.67, -7641.67, -7633.33, -7641.67, -7633.33, -7633.33, -7641.67, -7641.67, -7625.0, -7641.67, -7641.67, -7633.33, -7633.33, -7633.33, -7633.33, -7633.33, -7641.67, -7633.33, -7633.33, -7650.0, -7641.67, -7641.67, -7641.67, -7641.67, -7658.33, -7658.33, -7641.67, -7633.33, -7625.0, -7591.67, -7633.33, -7625.0, -7633.33, -7641.67, -7633.33, -7641.67, -7650.0, -7650.0, -7641.67, -7641.67, -7658.33, -7633.33, -7641.67, -7633.33, -7633.33, -7633.33, -7633.33, -7625.0, -7633.33, -7641.67, -7633.33, -7641.67, -7658.33, -7650.0, -7625.0, -7616.67, -7633.33], [-7633.33, -7633.33, -7616.67, -7633.33, -7641.67, -7633.33, -7625.0, -7650.0, -7616.67, -7641.67, -7633.33, -7625.0, -7650.0, -7641.67, -7641.67, -7641.67, -7616.67, -7650.0, -7641.67, -7641.67, -7641.67, -7641.67, -7625.0, -7633.33, -7633.33, -7650.0, -7641.67, -7650.0, -7641.67, -7625.0, -7633.33, -7641.67, -7633.33, -7641.67, -7633.33, -7641.67, -7625.0, -7633.33, -7625.0, -7625.0, -7650.0, -7633.33, -7633.33, -7625.0, -7625.0, -7633.33, -7641.67, -7650.0, -7675.0, -7633.33, -7616.67, -7641.67, -7616.67, -7650.0, -7633.33, -7650.0, -7641.67, -7641.67, -7625.0, -7641.67, -7633.33, -7641.67, -7625.0, -7641.67, -7641.67, -7641.67, -7625.0, -7633.33, -7633.33, -7641.67, -7633.33, -7633.33, -7633.33, -7633.33, -7633.33, -7625.0, -7641.67, -7641.67, -7625.0, -7633.33, -7650.0, -7633.33, -7666.67, -7633.33, -7633.33, -7633.33, -7641.67, -7650.0, -7633.33, -7633.33, -7625.0, -7641.67, -7633.33, -7625.0, -7625.0, -7641.67, -7650.0, -7641.67, -7641.67, -7633.33, -7650.0, -7616.67, -7625.0, -7650.0, -7625.0, -7641.67, -7616.67, -7625.0, -7633.33, -7641.67, -7641.67, -7616.67, -7608.33, -7650.0, -7625.0, -7633.33, -7658.33, -7641.67, -7625.0, -7633.33, -7641.67, -7641.67, -7633.33, -7641.67, -7666.67, -7633.33, -7608.33, -7633.33, -7633.33, -7633.33, -7633.33, -7608.33, -7641.67, -7600.0, -7641.67, -7633.33, -7633.33, -7650.0, -7633.33, -7608.33, -7650.0, -7625.0, -7625.0, -7641.67, -7625.0, -7625.0, -7625.0, -7633.33, -7650.0, -7616.67, -7616.67, -7666.67, -7616.67, -7616.67, -7666.67, -7625.0, -7641.67, -7616.67, -7633.33, -7633.33, -7641.67, -7641.67, -7641.67, -7625.0, -7658.33, -7625.0, -7641.67, -7658.33, -7633.33, -7633.33, -7641.67, -7650.0, -7625.0, -7641.67, -7633.33, -7641.67, -7625.0, -7633.33, -7633.33, -7641.67, -7633.33, -7633.33, -7633.33, -7641.67, -7633.33, -7633.33, -7650.0, -7641.67, -7633.33, -7633.33, -7633.33, -7633.33, -7641.67, -7625.0, -7641.67, -7625.0, -7616.67, -7633.33, -7641.67, -7625.0], [-7641.67, -7633.33, -7633.33, -7633.33, -7641.67, -7625.0, -7641.67, -7633.33, -7633.33, -7633.33, -7641.67, -7641.67, -7633.33, -7641.67, -7625.0, -7633.33, -7625.0, -7633.33, -7641.67, -7633.33, -7633.33, -7625.0, -7633.33, -7633.33, -7625.0, -7633.33, -7633.33, -7633.33, -7633.33, -7633.33, -7650.0, -7625.0, -7641.67, -7633.33, -7633.33, -7625.0, -7633.33, -7616.67, -7633.33, -7633.33, -7641.67, -7625.0, -7625.0, -7641.67, -7633.33, -7633.33, -7641.67, -7616.67, -7641.67, -7625.0, -7633.33, -7633.33, -7633.33, -7650.0, -7625.0, -7633.33, -7683.33, -7608.33, -7633.33, -7666.67, -7641.67, -7700.0, -7608.33, -7675.0, -7591.67, -7600.0, -7633.33, -7633.33, -7625.0, -7641.67, -7625.0, -7633.33, -7641.67, -7633.33, -7633.33, -7591.67, -7625.0, -7616.67, -7600.0, -7616.67, -7600.0, -7616.67, -7608.33, -7616.67, -7608.33, -7625.0, -7625.0, -7616.67, -7616.67, -7616.67, -7616.67, -7608.33, -7633.33, -7608.33, -7633.33, -7608.33, -7625.0, -7608.33, -7625.0, -7608.33, -7625.0, -7616.67, -7616.67, -7616.67, -7633.33, -7616.67, -7616.67, -7616.67, -7608.33, -7625.0, -7616.67, -7616.67, -7600.0, -7608.33, -7608.33, -7625.0, -7633.33, -7608.33, -7608.33, -7616.67, -7616.67, -7625.0, -7616.67, -7625.0, -7625.0, -7625.0, -7625.0, -7600.0, -7616.67, -7591.67, -7616.67, -7600.0, -7641.67, -7608.33, -7625.0, -7600.0, -7625.0, -7608.33, -7625.0, -7625.0, -7608.33, -7616.67, -7608.33, -7633.33, -7608.33, -7616.67, -7616.67, -7608.33, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7625.0, -7616.67, -7625.0, -7616.67, -7616.67, -7616.67, -7625.0, -7625.0, -7616.67, -7625.0, -7625.0, -7641.67, -7616.67, -7616.67, -7616.67, -7625.0, -7616.67, -7633.33, -7625.0, -7616.67, -7625.0, -7616.67, -7616.67, -7625.0, -7608.33, -7633.33, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7633.33, -7616.67, -7616.67, -7616.67, -7625.0, -7608.33, -7633.33, -7600.0, -7625.0, -7633.33, -7616.67, -7616.67, -7616.67, -7608.33, -7616.67, -7616.67], [-7625.0, -7625.0, -7625.0, -7616.67, -7625.0, -7616.67, -7625.0, -7616.67, -7625.0, -7625.0, -7625.0, -7608.33, -7625.0, -7616.67, -7625.0, -7616.67, -7625.0, -7608.33, -7616.67, -7616.67, -7625.0, -7625.0, -7625.0, -7600.0, -7625.0, -7625.0, -7608.33, -7616.67, -7625.0, -7625.0, -7608.33, -7625.0, -7616.67, -7616.67, -7616.67, -7625.0, -7625.0, -7616.67, -7608.33, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7625.0, -7616.67, -7625.0, -7608.33, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7625.0, -7616.67, -7625.0, -7625.0, -7616.67, -7600.0, -7625.0, -7616.67, -7625.0, -7608.33, -7600.0, -7608.33, -7625.0, -7616.67, -7608.33, -7633.33, -7600.0, -7616.67, -7616.67, -7608.33, -7608.33, -7608.33, -7608.33, -7616.67, -7616.67, -7616.67, -7608.33, -7608.33, -7616.67, -7608.33, -7608.33, -7616.67, -7616.67, -7608.33, -7608.33, -7616.67, -7608.33, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7625.0, -7616.67, -7616.67, -7616.67, -7625.0, -7616.67, -7616.67, -7616.67, -7616.67, -7600.0, -7600.0, -7616.67, -7608.33, -7616.67, -7616.67, -7608.33, -7608.33, -7608.33, -7616.67, -7625.0, -7616.67, -7625.0, -7625.0, -7616.67, -7625.0, -7616.67, -7616.67, -7616.67, -7608.33, -7616.67, -7616.67, -7616.67, -7625.0, -7608.33, -7600.0, -7608.33, -7608.33, -7625.0, -7608.33, -7616.67, -7625.0, -7616.67, -7616.67, -7583.33, -7633.33, -7608.33, -7600.0, -7608.33, -7616.67, -7616.67, -7600.0, -7616.67, -7575.0, -7625.0, -7583.33, -7600.0, -7608.33, -7591.67, -7600.0, -7591.67, -7591.67, -7600.0, -7600.0, -7591.67, -7616.67, -7600.0, -7591.67, -7616.67, -7591.67, -7591.67, -7608.33, -7600.0, -7600.0, -7608.33, -7600.0, -7608.33, -7591.67, -7591.67, -7591.67, -7583.33, -7591.67, -7608.33, -7608.33, -7600.0, -7591.67, -7600.0, -7608.33, -7591.67, -7600.0, -7591.67, -7616.67, -7591.67, -7583.33, -7600.0, -7583.33, -7583.33, -7591.67, -7591.67, -7625.0, -7608.33, -7608.33, -7600.0, -7591.67], [-7633.33, -7625.0, -7583.33, -7525.0, -7608.33, -7633.33, -7616.67, -7625.0, -7616.67, -7625.0, -7600.0, -7591.67, -7616.67, -7600.0, -7616.67, -7600.0, -7608.33, -7616.67, -7608.33, -7608.33, -7600.0, -7608.33, -7608.33, -7600.0, -7625.0, -7608.33, -7600.0, -7633.33, -7633.33, -7633.33, -7625.0, -7616.67, -7616.67, -7600.0, -7600.0, -7608.33, -7633.33, -7616.67, -7633.33, -7608.33, -7608.33, -7616.67, -7625.0, -7650.0, -7616.67, -7616.67, -7633.33, -7608.33, -7608.33, -7625.0, -7616.67, -7616.67, -7625.0, -7633.33, -7591.67, -7633.33, -7633.33, -7641.67, -7616.67, -7616.67, -7633.33, -7616.67, -7625.0, -7608.33, -7616.67, -7625.0, -7625.0, -7616.67, -7625.0, -7608.33, -7608.33, -7616.67, -7608.33, -7616.67, -7616.67, -7600.0, -7616.67, -7600.0, -7625.0, -7616.67, -7608.33, -7616.67, -7616.67, -7625.0, -7616.67, -7616.67, -7616.67, -7625.0, -7616.67, -7625.0, -7625.0, -7625.0, -7616.67, -7633.33, -7608.33, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7633.33, -7608.33, -7616.67, -7625.0, -7616.67, -7616.67, -7625.0, -7625.0, -7616.67, -7600.0, -7616.67, -7608.33, -7608.33, -7616.67, -7633.33, -7616.67, -7625.0, -7608.33, -7616.67, -7608.33, -7625.0, -7616.67, -7616.67, -7633.33, -7608.33, -7625.0, -7625.0, -7616.67, -7608.33, -7608.33, -7625.0, -7616.67, -7608.33, -7575.0, -7650.0, -7600.0, -7600.0, -7616.67, -7608.33, -7608.33, -7625.0, -7616.67, -7608.33, -7616.67, -7608.33, -7608.33, -7616.67, -7625.0, -7608.33, -7608.33, -7608.33, -7616.67, -7608.33, -7616.67, -7633.33, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7608.33, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7625.0, -7616.67, -7616.67, -7608.33, -7608.33, -7625.0, -7608.33, -7600.0, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7608.33, -7616.67, -7608.33, -7608.33, -7633.33, -7608.33, -7616.67, -7616.67, -7616.67, -7616.67, -7625.0, -7616.67, -7616.67], [-7608.33, -7616.67, -7616.67, -7608.33, -7616.67, -7608.33, -7625.0, -7616.67, -7608.33, -7608.33, -7625.0, -7616.67, -7600.0, -7625.0, -7608.33, -7616.67, -7616.67, -7616.67, -7616.67, -7608.33, -7600.0, -7633.33, -7625.0, -7616.67, -7625.0, -7608.33, -7616.67, -7616.67, -7616.67, -7616.67, -7625.0, -7625.0, -7616.67, -7616.67, -7608.33, -7608.33, -7600.0, -7625.0, -7616.67, -7616.67, -7608.33, -7616.67, -7608.33, -7600.0, -7616.67, -7608.33, -7608.33, -7616.67, -7625.0, -7608.33, -7616.67, -7625.0, -7600.0, -7633.33, -7608.33, -7616.67, -7608.33, -7625.0, -7616.67, -7608.33, -7616.67, -7625.0, -7608.33, -7608.33, -7616.67, -7616.67, -7616.67, -7625.0, -7625.0, -7608.33, -7616.67, -7616.67, -7616.67, -7608.33, -7608.33, -7608.33, -7616.67, -7616.67, -7608.33, -7608.33, -7608.33, -7616.67, -7625.0, -7608.33, -7608.33, -7625.0, -7616.67, -7625.0, -7625.0, -7608.33, -7608.33, -7616.67, -7608.33, -7608.33, -7608.33, -7616.67, -7600.0, -7633.33, -7616.67, -7616.67, -7616.67, -7608.33, -7608.33, -7608.33, -7616.67, -7608.33, -7625.0, -7616.67, -7616.67, -7616.67, -7625.0, -7625.0, -7608.33, -7625.0, -7616.67, -7608.33, -7625.0, -7616.67, -7616.67, -7608.33, -7625.0, -7616.67, -7616.67, -7616.67, -7608.33, -7616.67, -7625.0, -7608.33, -7625.0, -7616.67, -7625.0, -7608.33, -7608.33, -7608.33, -7600.0, -7608.33, -7600.0, -7625.0, -7600.0, -7608.33, -7625.0, -7625.0, -7591.67, -7616.67, -7633.33, -7608.33, -7608.33, -7600.0, -7608.33, -7616.67, -7625.0, -7625.0, -7625.0, -7608.33, -7625.0, -7600.0, -7608.33, -7608.33, -7608.33, -7633.33, -7616.67, -7625.0, -7616.67, -7633.33, -7616.67, -7608.33, -7616.67, -7608.33, -7616.67, -7616.67, -7616.67, -7600.0, -7608.33, -7625.0, -7616.67, -7608.33, -7616.67, -7616.67, -7616.67, -7625.0, -7608.33, -7625.0, -7616.67, -7616.67, -7600.0, -7616.67, -7616.67, -7608.33, -7633.33, -7616.67, -7616.67, -7625.0, -7616.67, -7616.67, -7608.33, -7616.67, -7616.67, -7616.67, -7616.67, -7600.0], [-7625.0, -7600.0, -7633.33, -7591.67, -7625.0, -7600.0, -7641.67, -7608.33, -7600.0, -7625.0, -7591.67, -7675.0, -7566.67, -7666.67, -7591.67, -7633.33, -7616.67, -7591.67, -7608.33, -7583.33, -7616.67, -7616.67, -7616.67, -7608.33, -7625.0, -7616.67, -7616.67, -7600.0, -7616.67, -7616.67, -7608.33, -7608.33, -7616.67, -7608.33, -7616.67, -7616.67, -7608.33, -7616.67, -7625.0, -7616.67, -7616.67, -7616.67, -7608.33, -7608.33, -7625.0, -7616.67, -7633.33, -7608.33, -7608.33, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7608.33, -7600.0, -7625.0, -7608.33, -7608.33, -7616.67, -7616.67, -7608.33, -7616.67, -7608.33, -7616.67, -7616.67, -7608.33, -7616.67, -7608.33, -7616.67, -7616.67, -7625.0, -7625.0, -7616.67, -7616.67, -7608.33, -7616.67, -7616.67, -7616.67, -7616.67, -7625.0, -7625.0, -7583.33, -7608.33, -7616.67, -7600.0, -7608.33, -7600.0, -7633.33, -7625.0, -7633.33, -7625.0, -7633.33, -7633.33, -7616.67, -7641.67, -7625.0, -7625.0, -7633.33, -7608.33, -7625.0, -7600.0, -7608.33, -7608.33, -7608.33, -7608.33, -7616.67, -7616.67, -7616.67, -7608.33, -7616.67, -7616.67, -7616.67, -7625.0, -7616.67, -7608.33, -7625.0, -7625.0, -7600.0, -7616.67, -7608.33, -7608.33, -7616.67, -7616.67, -7608.33, -7616.67, -7625.0, -7616.67, -7625.0, -7616.67, -7600.0, -7616.67, -7616.67, -7608.33, -7625.0, -7608.33, -7625.0, -7608.33, -7600.0, -7633.33, -7600.0, -7625.0, -7608.33, -7625.0, -7608.33, -7625.0, -7616.67, -7608.33, -7600.0, -7625.0, -7633.33, -7625.0, -7625.0, -7608.33, -7625.0, -7625.0, -7608.33, -7608.33, -7608.33, -7608.33, -7600.0, -7591.67, -7625.0, -7625.0, -7608.33, -7608.33, -7625.0, -7600.0, -7616.67, -7608.33, -7600.0, -7616.67, -7608.33, -7600.0, -7608.33, -7600.0, -7616.67, -7616.67, -7625.0, -7616.67, -7625.0, -7608.33, -7608.33, -7616.67, -7600.0, -7608.33, -7625.0, -7625.0, -7600.0, -7616.67, -7616.67, -7625.0, -7625.0, -7616.67, -7625.0, -7633.33, -7616.67, -7600.0, -7608.33], [-7616.67, -7591.67, -7608.33, -7616.67, -7616.67, -7616.67, -7616.67, -7608.33, -7616.67, -7616.67, -7616.67, -7616.67, -7633.33, -7600.0, -7591.67, -7608.33, -7616.67, -7608.33, -7600.0, -7616.67, -7616.67, -7625.0, -7608.33, -7633.33, -7633.33, -7616.67, -7641.67, -7625.0, -7625.0, -7616.67, -7600.0, -7625.0, -7600.0, -7616.67, -7625.0, -7625.0, -7616.67, -7625.0, -7608.33, -7625.0, -7616.67, -7625.0, -7625.0, -7600.0, -7616.67, -7616.67, -7608.33, -7608.33, -7616.67, -7625.0, -7608.33, -7625.0, -7608.33, -7608.33, -7616.67, -7608.33, -7633.33, -7616.67, -7625.0, -7616.67, -7608.33, -7608.33, -7616.67, -7625.0, -7608.33, -7608.33, -7616.67, -7625.0, -7616.67, -7616.67, -7608.33, -7608.33, -7608.33, -7608.33, -7608.33, -7616.67, -7600.0, -7616.67, -7616.67, -7616.67, -7625.0, -7616.67, -7616.67, -7608.33, -7625.0, -7608.33, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7608.33, -7616.67, -7633.33, -7616.67, -7591.67, -7616.67, -7616.67, -7616.67, -7625.0, -7616.67, -7633.33, -7608.33, -7608.33, -7625.0, -7616.67, -7616.67, -7625.0, -7616.67, -7600.0, -7608.33, -7600.0, -7608.33, -7616.67, -7625.0, -7616.67, -7608.33, -7633.33, -7608.33, -7625.0, -7608.33, -7608.33, -7608.33, -7608.33, -7608.33, -7616.67, -7616.67, -7625.0, -7616.67, -7625.0, -7616.67, -7608.33, -7625.0, -7616.67, -7616.67, -7608.33, -7616.67, -7600.0, -7608.33, -7625.0, -7608.33, -7608.33, -7625.0, -7608.33, -7600.0, -7616.67, -7616.67, -7583.33, -7591.67, -7616.67, -7616.67, -7608.33, -7616.67, -7550.0, -7541.67, -7608.33, -7616.67, -7625.0, -7616.67, -7616.67, -7608.33, -7616.67, -7608.33, -7616.67, -7616.67, -7616.67, -7600.0, -7608.33, -7600.0, -7625.0, -7608.33, -7600.0, -7616.67, -7608.33, -7608.33, -7600.0, -7625.0, -7616.67, -7608.33, -7591.67, -7608.33, -7616.67, -7608.33, -7616.67, -7608.33, -7625.0, -7608.33, -7616.67, -7616.67, -7625.0, -7608.33, -7608.33, -7608.33, -7616.67, -7608.33], [-7616.67, -7600.0, -7616.67, -7616.67, -7616.67, -7625.0, -7633.33, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7600.0, -7608.33, -7625.0, -7591.67, -7600.0, -7608.33, -7616.67, -7591.67, -7583.33, -7575.0, -7641.67, -7591.67, -7608.33, -7591.67, -7625.0, -7591.67, -7608.33, -7591.67, -7600.0, -7600.0, -7600.0, -7608.33, -7608.33, -7600.0, -7608.33, -7608.33, -7591.67, -7616.67, -7600.0, -7608.33, -7600.0, -7608.33, -7600.0, -7600.0, -7616.67, -7608.33, -7608.33, -7600.0, -7575.0, -7608.33, -7616.67, -7608.33, -7608.33, -7600.0, -7583.33, -7616.67, -7583.33, -7600.0, -7608.33, -7608.33, -7600.0, -7591.67, -7600.0, -7600.0, -7616.67, -7608.33, -7608.33, -7600.0, -7608.33, -7600.0, -7616.67, -7600.0, -7608.33, -7600.0, -7608.33, -7600.0, -7591.67, -7591.67, -7608.33, -7608.33, -7608.33, -7591.67, -7591.67, -7591.67, -7608.33, -7600.0, -7600.0, -7583.33, -7591.67, -7591.67, -7600.0, -7600.0, -7625.0, -7616.67, -7591.67, -7608.33, -7600.0, -7616.67, -7591.67, -7616.67, -7616.67, -7608.33, -7608.33, -7608.33, -7608.33, -7600.0, -7600.0, -7608.33, -7591.67, -7600.0, -7608.33, -7608.33, -7591.67, -7608.33, -7600.0, -7583.33, -7608.33, -7600.0, -7600.0, -7608.33, -7591.67, -7600.0, -7616.67, -7591.67, -7591.67, -7600.0, -7591.67, -7625.0, -7583.33, -7616.67, -7600.0, -7608.33, -7591.67, -7591.67, -7608.33, -7600.0, -7600.0, -7583.33, -7600.0, -7600.0, -7600.0, -7600.0, -7608.33, -7616.67, -7600.0, -7608.33, -7608.33, -7600.0, -7675.0, -7616.67, -7608.33, -7608.33, -7608.33, -7516.67, -7600.0, -7641.67, -7608.33, -7600.0, -7650.0, -7683.33, -7508.33, -7583.33, -7675.0, -7566.67, -7516.67, -7625.0, -7625.0, -7583.33, -7616.67, -7600.0, -7616.67, -7633.33, -7600.0, -7550.0, -7566.67, -7650.0, -7600.0, -7875.0, -7575.0, -7575.0, -7658.33, -7583.33, -7583.33, -7600.0, -7608.33, -7608.33, -7600.0, -7591.67, -7600.0, -7591.67, -7550.0, -7641.67, -7633.33, -7616.67, -7625.0, -7583.33, -7591.67, -7625.0], [-7600.0, -7608.33, -7616.67, -7625.0, -7608.33, -7616.67, -7625.0, -7616.67, -7600.0, -7591.67, -7616.67, -7608.33, -7608.33, -7616.67, -7616.67, -7600.0, -7641.67, -7591.67, -7608.33, -7616.67, -7608.33, -7625.0, -7658.33, -7641.67, -7633.33, -7625.0, -7558.33, -7641.67, -7591.67, -7633.33, -7616.67, -7625.0, -7608.33, -7641.67, -7616.67, -7633.33, -7675.0, -7566.67, -7641.67, -7608.33, -7616.67, -7625.0, -7600.0, -7600.0, -7625.0, -7616.67, -7600.0, -7600.0, -7583.33, -7600.0, -7608.33, -7591.67, -7658.33, -7591.67, -7658.33, -7608.33, -7650.0, -7600.0, -7625.0, -7600.0, -7625.0, -7625.0, -7600.0, -7650.0, -7633.33, -7591.67, -7625.0, -7600.0, -7641.67, -7650.0, -7625.0, -7600.0, -7600.0, -7616.67, -7608.33, -7608.33, -7583.33, -7591.67, -7650.0, -7575.0, -7600.0, -7625.0, -7608.33, -7600.0, -7608.33, -7616.67, -7600.0, -7600.0, -7608.33, -7600.0, -7600.0, -7583.33, -7600.0, -7591.67, -7600.0, -7575.0, -7591.67, -7591.67, -7583.33, -7600.0, -7575.0, -7600.0, -7583.33, -7600.0, -7600.0, -7608.33, -7583.33, -7600.0, -7591.67, -7583.33, -7591.67, -7608.33, -7591.67, -7583.33, -7600.0, -7600.0, -7608.33, -7600.0, -7600.0, -7591.67, -7608.33, -7600.0, -7591.67, -7608.33, -7600.0, -7608.33, -7600.0, -7600.0, -7600.0, -7608.33, -7600.0, -7600.0, -7600.0, -7600.0, -7608.33, -7600.0, -7591.67, -7608.33, -7608.33, -7600.0, -7608.33, -7591.67, -7608.33, -7600.0, -7608.33, -7583.33, -7608.33, -7600.0, -7608.33, -7616.67, -7600.0, -7608.33, -7600.0, -7608.33, -7591.67, -7608.33, -7591.67, -7600.0, -7600.0, -7608.33, -7600.0, -7600.0, -7608.33, -7600.0, -7600.0, -7600.0, -7608.33, -7591.67, -7608.33, -7600.0, -7600.0, -7600.0, -7600.0, -7608.33, -7600.0, -7608.33, -7608.33, -7600.0, -7600.0, -7583.33, -7583.33, -7591.67, -7616.67, -7600.0, -7600.0, -7608.33, -7608.33, -7616.67, -7600.0, -7616.67, -7608.33, -7600.0, -7608.33, -7600.0, -7600.0, -7608.33, -7600.0, -7600.0, -7591.67, -7600.0], [-7616.67, -7591.67, -7608.33, -7600.0, -7608.33, -7608.33, -7591.67, -7608.33, -7608.33, -7600.0, -7616.67, -7608.33, -7600.0, -7608.33, -7616.67, -7600.0, -7608.33, -7625.0, -7583.33, -7625.0, -7616.67, -7600.0, -7600.0, -7633.33, -7600.0, -7600.0, -7625.0, -7600.0, -7608.33, -7608.33, -7608.33, -7608.33, -7616.67, -7608.33, -7600.0, -7616.67, -7616.67, -7608.33, -7608.33, -7616.67, -7616.67, -7608.33, -7600.0, -7616.67, -7608.33, -7608.33, -7608.33, -7600.0, -7608.33, -7616.67, -7608.33, -7608.33, -7600.0, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7616.67, -7608.33, -7616.67, -7600.0, -7608.33, -7600.0, -7633.33, -7600.0, -7625.0, -7625.0, -7608.33, -7608.33, -7608.33, -7600.0, -7608.33, -7608.33, -7616.67, -7608.33, -7608.33, -7625.0, -7608.33, -7616.67, -7608.33, -7616.67, -7625.0, -7608.33, -7616.67, -7608.33, -7608.33, -7633.33, -7591.67, -7608.33, -7633.33, -7600.0, -7591.67, -7641.67, -7625.0, -7666.67, -7600.0, -7633.33, -7533.33, -7675.0, -7675.0, -7666.67, -7558.33, -7625.0, -7708.33, -7608.33, -7683.33, -7566.67, -7558.33, -7691.67, -7583.33, -7666.67, -7650.0, -7625.0, -7625.0, -7641.67, -7616.67, -7641.67, -7633.33, -7616.67, -7641.67, -7641.67, -7600.0, -7641.67, -7633.33, -7608.33, -7608.33, -7625.0, -7600.0, -7633.33, -7591.67, -7716.67, -7625.0, -7408.33, -7816.67, -7675.0, -7666.67, -7675.0, -7700.0, -7633.33, -7716.67, -7650.0, -7650.0, -7708.33, -7666.67, -7658.33, -7741.67, -7625.0, -7616.67, -7791.67, -7683.33, -7691.67, -7691.67, -7700.0, -7700.0, -7708.33, -7691.67, -7708.33, -7716.67, -7725.0, -7675.0, -7700.0, -7691.67, -7716.67, -7691.67, -7725.0, -7691.67, -7708.33, -7708.33, -7683.33, -7716.67, -7675.0, -7716.67, -7683.33, -7700.0, -7675.0, -7716.67, -7691.67, -7708.33, -7716.67, -7700.0, -7683.33, -7700.0, -7708.33, -7683.33, -7708.33, -7691.67, -7700.0, -7700.0, -7700.0, -7708.33, -7700.0, -7700.0, -7700.0, -7708.33, -7708.33, -7700.0, -7675.0, -7708.33, -7700.0], [-7700.0, -7708.33, -7716.67, -7700.0, -7716.67, -7708.33, -7725.0, -7675.0, -7716.67, -7691.67, -7766.67, -7650.0, -7758.33, -7691.67, -7658.33, -7741.67, -7716.67, -7666.67, -7716.67, -7683.33, -7708.33, -7675.0, -7716.67, -7675.0, -7733.33, -7683.33, -7716.67, -7700.0, -7725.0, -7700.0, -7716.67, -7700.0, -7708.33, -7700.0, -7708.33, -7691.67, -7708.33, -7716.67, -7708.33, -7691.67, -7716.67, -7708.33, -7691.67, -7691.67, -7716.67, -7683.33, -7700.0, -7708.33, -7700.0, -7700.0, -7708.33, -7708.33, -7683.33, -7716.67, -7700.0, -7716.67, -7725.0, -7700.0, -7700.0, -7708.33, -7700.0, -7708.33, -7708.33, -7700.0, -7716.67, -7700.0, -7700.0, -7691.67, -7700.0, -7700.0, -7691.67, -7708.33, -7691.67, -7700.0, -7683.33, -7700.0, -7691.67, -7708.33, -7708.33, -7700.0, -7700.0, -7691.67, -7708.33, -7708.33, -7708.33, -7700.0, -7700.0, -7716.67, -7691.67, -7700.0, -7683.33, -7700.0, -7683.33, -7700.0, -7716.67, -7700.0, -7683.33, -7716.67, -7708.33, -7691.67, -7708.33, -7691.67, -7700.0, -7700.0, -7725.0, -7708.33, -7700.0, -7691.67, -7700.0, -7766.67, -7700.0, -7691.67, -7691.67, -7691.67, -7691.67, -7733.33, -7700.0, -7733.33, -7658.33, -7766.67, -7650.0, -7758.33, -7641.67, -7733.33, -7716.67, -7675.0, -7741.67, -7666.67, -7758.33, -7666.67, -7741.67, -7658.33, -7708.33, -7700.0, -7683.33, -7675.0, -7716.67, -7650.0, -7783.33, -7658.33, -7783.33, -7675.0, -7808.33, -7666.67, -7800.0, -7675.0, -7758.33, -7641.67, -7691.67, -7675.0, -7683.33, -7733.33, -7683.33, -7783.33, -7666.67, -7800.0, -7658.33, -7783.33, -7666.67, -7750.0, -7666.67, -7691.67, -7675.0, -7691.67, -7725.0, -7675.0, -7775.0, -7666.67, -7783.33, -7666.67, -7716.67, -7675.0, -7683.33, -7683.33, -7708.33, -7741.67, -7683.33, -7783.33, -7666.67, -7766.67, -7666.67, -7700.0, -7666.67, -7708.33, -7700.0, -7691.67, -7750.0, -7683.33, -7775.0, -7666.67, -7716.67, -7650.0, -7683.33, -7675.0, -7708.33, -7750.0, -7675.0, -7783.33, -7675.0, -7725.0], [-7658.33, -7658.33, -7650.0, -7658.33, -7675.0, -7691.67, -7666.67, -7658.33, -7675.0, -7658.33, -7675.0, -7691.67, -7666.67, -7675.0, -7658.33, -7683.33, -7633.33, -7683.33, -7666.67, -7650.0, -7683.33, -7658.33, -7683.33, -7650.0, -7675.0, -7641.67, -7666.67, -7658.33, -7683.33, -7650.0, -7683.33, -7641.67, -7700.0, -7691.67, -7675.0, -7658.33, -7700.0, -7658.33, -7650.0, -7633.33, -7733.33, -7650.0, -7633.33, -7658.33, -7741.67, -7616.67, -7641.67, -7708.33, -7608.33, -7616.67, -7625.0, -7608.33, -7583.33, -7583.33, -7591.67, -7575.0, -7583.33, -7575.0, -7583.33, -7558.33, -7558.33, -7566.67, -7566.67, -7550.0, -7541.67, -7558.33, -7566.67, -7558.33, -7550.0, -7566.67, -7566.67, -7550.0, -7558.33, -7550.0, -7550.0, -7558.33, -7558.33, -7558.33, -7550.0, -7566.67, -7541.67, -7550.0, -7550.0, -7550.0, -7558.33, -7550.0, -7558.33, -7550.0, -7533.33, -7566.67, -7550.0, -7541.67, -7533.33, -7566.67, -7550.0, -7558.33, -7541.67, -7541.67, -7566.67, -7566.67, -7558.33, -7566.67, -7608.33, -7575.0, -7625.0, -7600.0, -7641.67, -7608.33, -7641.67, -7716.67, -7600.0, -7633.33, -7691.67, -7625.0, -7633.33, -7625.0, -7650.0, -7641.67, -7650.0, -7608.33, -7608.33, -7675.0, -7633.33, -7625.0, -7641.67, -7616.67, -7608.33, -7666.67, -7616.67, -7675.0, -7633.33, -7633.33, -7658.33, -7641.67, -7625.0, -7608.33, -7666.67, -7583.33, -7658.33, -7633.33, -7625.0, -7608.33, -7625.0, -7666.67, -7541.67, -7666.67, -7616.67, -7625.0, -7650.0, -7608.33, -7625.0, -7675.0, -7650.0, -7583.33, -7658.33, -7625.0, -7708.33, -7600.0, -7666.67, -7600.0, -7625.0, -7633.33, -7616.67, -7700.0, -7583.33, -7691.67, -7625.0, -7625.0, -7633.33, -7633.33, -7641.67, -7625.0, -7675.0, -7608.33, -7683.33, -7600.0, -7700.0, -7583.33, -7725.0, -7583.33, -7708.33, -7591.67, -7650.0, -7591.67, -7666.67, -7608.33, -7641.67, -7583.33, -7675.0, -7716.67, -7550.0, -7691.67, -7641.67, -7641.67, -7625.0, -7725.0, -7616.67, -7625.0, -7666.67, -7650.0], [-7650.0, -7625.0, -7641.67, -7658.33, -7675.0, -7633.33, -7650.0, -7675.0, -7625.0, -7675.0, -7641.67, -7625.0, -7658.33, -7666.67, -7616.67, -7625.0, -7625.0, -7658.33, -7616.67, -7641.67, -7625.0, -7625.0, -7650.0, -7683.33, -7575.0, -7600.0, -7591.67, -7650.0, -7641.67, -7633.33, -7608.33, -7666.67, -7525.0, -7633.33, -7366.67, -7558.33, -7575.0, -7591.67, -7533.33, -7550.0, -7608.33, -7566.67, -7533.33, -7541.67, -7558.33, -7583.33, -7550.0, -7525.0, -7591.67, -7541.67, -7575.0, -7566.67, -7550.0, -7550.0, -7583.33, -7558.33, -7533.33, -7550.0, -7525.0, -7558.33, -7541.67, -7566.67, -7541.67, -7566.67, -7575.0, -7533.33, -7533.33, -7566.67, -7566.67, -7558.33, -7550.0, -7550.0, -7558.33, -7558.33, -7533.33, -7558.33, -7533.33, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7541.67, -7541.67, -7541.67, -7533.33, -7558.33, -7550.0, -7541.67, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7541.67, -7533.33, -7550.0, -7541.67, -7550.0, -7566.67, -7550.0, -7525.0, -7558.33, -7541.67, -7541.67, -7558.33, -7558.33, -7533.33, -7525.0, -7550.0, -7558.33, -7541.67, -7550.0, -7533.33, -7550.0, -7525.0, -7583.33, -7550.0, -7550.0, -7566.67, -7516.67, -7541.67, -7566.67, -7550.0, -7533.33, -7533.33, -7550.0, -7550.0, -7550.0, -7541.67, -7525.0, -7550.0, -7533.33, -7533.33, -7533.33, -7533.33, -7541.67, -7525.0, -7566.67, -7550.0, -7550.0, -7550.0, -7533.33, -7550.0, -7550.0, -7491.67, -7583.33, -7525.0, -7550.0, -7558.33, -7541.67, -7533.33, -7566.67, -7533.33, -7508.33, -7550.0, -7541.67, -7566.67, -7516.67, -7641.67, -7616.67, -7633.33, -7433.33, -7508.33, -7608.33, -7533.33, -7608.33, -7575.0, -7600.0, -7600.0, -7608.33, -7550.0, -7583.33, -7575.0, -7575.0, -7566.67, -7550.0, -7541.67, -7575.0, -7566.67, -7558.33, -7550.0, -7550.0, -7558.33, -7550.0, -7558.33, -7575.0, -7566.67, -7558.33, -7575.0, -7558.33, -7566.67, -7575.0, -7550.0, -7575.0, -7558.33, -7566.67, -7558.33, -7558.33], [-7575.0, -7566.67, -7591.67, -7558.33, -7575.0, -7550.0, -7550.0, -7608.33, -7575.0, -7566.67, -7591.67, -7550.0, -7575.0, -7558.33, -7575.0, -7558.33, -7575.0, -7591.67, -7533.33, -7558.33, -7550.0, -7558.33, -7566.67, -7550.0, -7550.0, -7575.0, -7558.33, -7575.0, -7558.33, -7566.67, -7566.67, -7541.67, -7575.0, -7550.0, -7575.0, -7558.33, -7566.67, -7575.0, -7566.67, -7575.0, -7550.0, -7566.67, -7558.33, -7558.33, -7575.0, -7566.67, -7575.0, -7575.0, -7533.33, -7566.67, -7566.67, -7566.67, -7566.67, -7550.0, -7558.33, -7591.67, -7566.67, -7583.33, -7541.67, -7575.0, -7566.67, -7566.67, -7558.33, -7575.0, -7591.67, -7566.67, -7575.0, -7575.0, -7558.33, -7566.67, -7566.67, -7558.33, -7566.67, -7566.67, -7558.33, -7583.33, -7566.67, -7566.67, -7558.33, -7558.33, -7558.33, -7550.0, -7558.33, -7558.33, -7566.67, -7575.0, -7566.67, -7566.67, -7566.67, -7575.0, -7575.0, -7550.0, -7583.33, -7558.33, -7558.33, -7591.67, -7550.0, -7583.33, -7550.0, -7575.0, -7558.33, -7575.0, -7566.67, -7558.33, -7575.0, -7558.33, -7575.0, -7575.0, -7558.33, -7575.0, -7558.33, -7575.0, -7541.67, -7575.0, -7583.33, -7533.33, -7583.33, -7583.33, -7558.33, -7566.67, -7583.33, -7558.33, -7575.0, -7575.0, -7566.67, -7566.67, -7575.0, -7566.67, -7558.33, -7566.67, -7575.0, -7575.0, -7558.33, -7575.0, -7575.0, -7575.0, -7566.67, -7566.67, -7566.67, -7566.67, -7566.67, -7575.0, -7550.0, -7591.67, -7575.0, -7566.67, -7566.67, -7566.67, -7558.33, -7541.67, -7558.33, -7558.33, -7566.67, -7575.0, -7558.33, -7566.67, -7566.67, -7583.33, -7558.33, -7550.0, -7566.67, -7566.67, -7575.0, -7558.33, -7566.67, -7575.0, -7541.67, -7558.33, -7566.67, -7566.67, -7566.67, -7566.67, -7558.33, -7558.33, -7583.33, -7566.67, -7575.0, -7566.67, -7566.67, -7558.33, -7566.67, -7575.0, -7558.33, -7566.67, -7566.67, -7558.33, -7575.0, -7591.67, -7558.33, -7575.0, -7591.67, -7550.0, -7550.0, -7583.33, -7550.0, -7575.0, -7575.0, -7625.0, -7533.33, -7583.33], [-7533.33, -7591.67, -7558.33, -7583.33, -7525.0, -7550.0, -7541.67, -7550.0, -7541.67, -7558.33, -7525.0, -7575.0, -7558.33, -7550.0, -7558.33, -7558.33, -7591.67, -7558.33, -7575.0, -7558.33, -7566.67, -7558.33, -7558.33, -7558.33, -7541.67, -7575.0, -7558.33, -7558.33, -7566.67, -7566.67, -7583.33, -7558.33, -7541.67, -7558.33, -7550.0, -7566.67, -7483.33, -7591.67, -7583.33, -7550.0, -7550.0, -7575.0, -7566.67, -7550.0, -7558.33, -7575.0, -7566.67, -7558.33, -7558.33, -7558.33, -7566.67, -7566.67, -7558.33, -7566.67, -7575.0, -7575.0, -7550.0, -7575.0, -7566.67, -7566.67, -7558.33, -7566.67, -7566.67, -7566.67, -7566.67, -7583.33, -7566.67, -7558.33, -7558.33, -7566.67, -7558.33, -7558.33, -7558.33, -7558.33, -7558.33, -7550.0, -7566.67, -7566.67, -7558.33, -7558.33, -7566.67, -7566.67, -7566.67, -7566.67, -7558.33, -7575.0, -7575.0, -7583.33, -7550.0, -7558.33, -7575.0, -7566.67, -7558.33, -7558.33, -7558.33, -7566.67, -7558.33, -7558.33, -7575.0, -7566.67, -7558.33, -7566.67, -7558.33, -7566.67, -7575.0, -7558.33, -7566.67, -7566.67, -7575.0, -7566.67, -7575.0, -7566.67, -7566.67, -7558.33, -7575.0, -7566.67, -7566.67, -7558.33, -7558.33, -7566.67, -7558.33, -7566.67, -7558.33, -7558.33, -7558.33, -7566.67, -7558.33, -7575.0, -7566.67, -7558.33, -7575.0, -7558.33, -7566.67, -7550.0, -7558.33, -7566.67, -7575.0, -7575.0, -7558.33, -7558.33, -7566.67, -7591.67, -7575.0, -7566.67, -7566.67, -7558.33, -7566.67, -7566.67, -7575.0, -7550.0, -7566.67, -7575.0, -7558.33, -7566.67, -7566.67, -7575.0, -7575.0, -7566.67, -7558.33, -7566.67, -7558.33, -7550.0, -7550.0, -7575.0, -7575.0, -7566.67, -7566.67, -7575.0, -7566.67, -7558.33, -7575.0, -7566.67, -7558.33, -7575.0, -7566.67, -7566.67, -7566.67, -7566.67, -7566.67, -7558.33, -7566.67, -7558.33, -7575.0, -7558.33, -7566.67, -7575.0, -7558.33, -7566.67, -7566.67, -7558.33, -7566.67, -7558.33, -7575.0, -7566.67, -7566.67, -7558.33, -7566.67, -7566.67, -7575.0, -7566.67], [-7575.0, -7558.33, -7558.33, -7558.33, -7575.0, -7558.33, -7575.0, -7558.33, -7558.33, -7575.0, -7566.67, -7575.0, -7566.67, -7566.67, -7566.67, -7550.0, -7566.67, -7575.0, -7558.33, -7575.0, -7575.0, -7558.33, -7566.67, -7550.0, -7575.0, -7566.67, -7566.67, -7575.0, -7575.0, -7583.33, -7566.67, -7566.67, -7566.67, -7566.67, -7566.67, -7566.67, -7566.67, -7566.67, -7566.67, -7566.67, -7566.67, -7558.33, -7566.67, -7550.0, -7566.67, -7558.33, -7575.0, -7558.33, -7558.33, -7566.67, -7541.67, -7566.67, -7558.33, -7566.67, -7566.67, -7566.67, -7558.33, -7558.33, -7566.67, -7566.67, -7558.33, -7558.33, -7566.67, -7558.33, -7550.0, -7566.67, -7566.67, -7558.33, -7575.0, -7566.67, -7575.0, -7558.33, -7575.0, -7558.33, -7550.0, -7566.67, -7558.33, -7550.0, -7566.67, -7558.33, -7550.0, -7550.0, -7583.33, -7575.0, -7566.67, -7566.67, -7566.67, -7566.67, -7575.0, -7566.67, -7558.33, -7566.67, -7575.0, -7550.0, -7558.33, -7558.33, -7575.0, -7566.67, -7566.67, -7550.0, -7566.67, -7566.67, -7566.67, -7566.67, -7566.67, -7575.0, -7575.0, -7566.67, -7558.33, -7558.33, -7558.33, -7566.67, -7550.0, -7566.67, -7566.67, -7550.0, -7550.0, -7558.33, -7566.67, -7558.33, -7558.33, -7566.67, -7566.67, -7558.33, -7566.67, -7566.67, -7558.33, -7583.33, -7558.33, -7558.33, -7566.67, -7558.33, -7575.0, -7558.33, -7558.33, -7566.67, -7550.0, -7575.0, -7558.33, -7558.33, -7566.67, -7575.0, -7566.67, -7566.67, -7558.33, -7558.33, -7566.67, -7566.67, -7575.0, -7558.33, -7558.33, -7550.0, -7566.67, -7566.67, -7575.0, -7566.67, -7550.0, -7558.33, -7558.33, -7558.33, -7558.33, -7575.0, -7566.67, -7583.33, -7575.0, -7600.0, -7533.33, -7516.67, -7600.0, -7516.67, -7333.33, -7500.0, -7508.33, -7616.67, -7558.33, -7525.0, -7583.33, -7550.0, -7550.0, -7566.67, -7550.0, -7550.0, -7558.33, -7550.0, -7550.0, -7541.67, -7550.0, -7550.0, -7541.67, -7558.33, -7550.0, -7550.0, -7558.33, -7558.33, -7541.67, -7550.0, -7558.33, -7541.67, -7550.0, -7558.33], [-7516.67, -7550.0, -7550.0, -7533.33, -7558.33, -7550.0, -7533.33, -7541.67, -7550.0, -7550.0, -7550.0, -7550.0, -7525.0, -7550.0, -7541.67, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7533.33, -7550.0, -7541.67, -7550.0, -7550.0, -7541.67, -7541.67, -7558.33, -7550.0, -7550.0, -7541.67, -7550.0, -7541.67, -7550.0, -7541.67, -7550.0, -7541.67, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7533.33, -7550.0, -7541.67, -7558.33, -7541.67, -7541.67, -7541.67, -7558.33, -7541.67, -7541.67, -7566.67, -7550.0, -7541.67, -7541.67, -7550.0, -7550.0, -7541.67, -7550.0, -7541.67, -7541.67, -7558.33, -7541.67, -7541.67, -7541.67, -7558.33, -7566.67, -7550.0, -7541.67, -7533.33, -7566.67, -7558.33, -7541.67, -7541.67, -7550.0, -7558.33, -7550.0, -7533.33, -7550.0, -7550.0, -7550.0, -7558.33, -7541.67, -7541.67, -7558.33, -7550.0, -7566.67, -7558.33, -7558.33, -7541.67, -7550.0, -7541.67, -7550.0, -7541.67, -7566.67, -7550.0, -7550.0, -7558.33, -7558.33, -7541.67, -7550.0, -7558.33, -7550.0, -7550.0, -7558.33, -7550.0, -7541.67, -7550.0, -7550.0, -7550.0, -7558.33, -7550.0, -7558.33, -7541.67, -7541.67, -7541.67, -7558.33, -7550.0, -7550.0, -7550.0, -7558.33, -7550.0, -7541.67, -7541.67, -7541.67, -7550.0, -7533.33, -7558.33, -7550.0, -7550.0, -7583.33, -7583.33, -7550.0, -7558.33, -7550.0, -7558.33, -7541.67, -7541.67, -7541.67, -7550.0, -7566.67, -7550.0, -7550.0, -7558.33, -7550.0, -7550.0, -7558.33, -7550.0, -7541.67, -7558.33, -7550.0, -7541.67, -7558.33, -7550.0, -7550.0, -7558.33, -7541.67, -7550.0, -7541.67, -7550.0, -7550.0, -7558.33, -7558.33, -7541.67, -7550.0, -7558.33, -7541.67, -7533.33, -7550.0, -7550.0, -7558.33, -7550.0, -7566.67, -7566.67, -7550.0, -7575.0, -7541.67, -7533.33, -7508.33, -7525.0, -7466.67, -7616.67, -7516.67, -7550.0, -7558.33, -7516.67, -7508.33, -7583.33, -7575.0, -7558.33, -7566.67, -7541.67, -7550.0, -7558.33, -7541.67, -7550.0, -7566.67, -7558.33], [-7550.0, -7558.33, -7550.0, -7550.0, -7541.67, -7558.33, -7550.0, -7550.0, -7550.0, -7550.0, -7558.33, -7550.0, -7558.33, -7550.0, -7558.33, -7558.33, -7558.33, -7566.67, -7533.33, -7541.67, -7541.67, -7550.0, -7550.0, -7541.67, -7550.0, -7541.67, -7541.67, -7550.0, -7558.33, -7541.67, -7550.0, -7550.0, -7533.33, -7533.33, -7550.0, -7541.67, -7550.0, -7541.67, -7541.67, -7550.0, -7558.33, -7550.0, -7541.67, -7550.0, -7558.33, -7541.67, -7550.0, -7566.67, -7550.0, -7533.33, -7558.33, -7550.0, -7541.67, -7558.33, -7550.0, -7558.33, -7550.0, -7541.67, -7558.33, -7541.67, -7541.67, -7541.67, -7541.67, -7558.33, -7550.0, -7550.0, -7541.67, -7558.33, -7541.67, -7550.0, -7541.67, -7550.0, -7550.0, -7541.67, -7541.67, -7558.33, -7550.0, -7550.0, -7558.33, -7550.0, -7550.0, -7558.33, -7550.0, -7558.33, -7541.67, -7541.67, -7550.0, -7541.67, -7541.67, -7558.33, -7558.33, -7550.0, -7558.33, -7541.67, -7550.0, -7558.33, -7566.67, -7550.0, -7558.33, -7550.0, -7558.33, -7550.0, -7550.0, -7541.67, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7558.33, -7558.33, -7541.67, -7550.0, -7558.33, -7550.0, -7558.33, -7558.33, -7558.33, -7558.33, -7550.0, -7550.0, -7558.33, -7558.33, -7550.0, -7550.0, -7558.33, -7541.67, -7550.0, -7541.67, -7541.67, -7558.33, -7550.0, -7550.0, -7541.67, -7550.0, -7558.33, -7550.0, -7541.67, -7525.0, -7583.33, -7541.67, -7558.33, -7558.33, -7525.0, -7550.0, -7558.33, -7508.33, -7533.33, -7550.0, -7566.67, -7558.33, -7558.33, -7525.0, -7566.67, -7525.0, -7558.33, -7558.33, -7558.33, -7558.33, -7541.67, -7541.67, -7541.67, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7558.33, -7550.0, -7550.0, -7566.67, -7541.67, -7566.67, -7541.67, -7550.0, -7550.0, -7558.33, -7533.33, -7558.33, -7541.67, -7541.67, -7550.0, -7550.0, -7558.33, -7541.67, -7541.67, -7550.0, -7558.33, -7558.33, -7550.0, -7558.33, -7550.0], [-7558.33, -7550.0, -7541.67, -7550.0, -7558.33, -7550.0, -7541.67, -7558.33, -7541.67, -7558.33, -7550.0, -7550.0, -7550.0, -7550.0, -7558.33, -7550.0, -7550.0, -7558.33, -7558.33, -7550.0, -7550.0, -7550.0, -7558.33, -7541.67, -7541.67, -7558.33, -7541.67, -7566.67, -7558.33, -7541.67, -7575.0, -7558.33, -7558.33, -7550.0, -7558.33, -7550.0, -7558.33, -7558.33, -7550.0, -7550.0, -7550.0, -7566.67, -7566.67, -7558.33, -7541.67, -7558.33, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7558.33, -7566.67, -7558.33, -7550.0, -7550.0, -7558.33, -7550.0, -7541.67, -7558.33, -7558.33, -7558.33, -7558.33, -7558.33, -7550.0, -7558.33, -7550.0, -7558.33, -7558.33, -7566.67, -7558.33, -7558.33, -7558.33, -7558.33, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7550.0, -7566.67, -7558.33, -7558.33, -7558.33, -7550.0, -7541.67, -7558.33, -7566.67, -7550.0, -7558.33, -7550.0, -7550.0, -7558.33, -7558.33, -7550.0, -7550.0, -7550.0, -7558.33, -7541.67, -7558.33, -7550.0, -7550.0, -7550.0, -7558.33, -7558.33, -7550.0, -7558.33, -7550.0, -7550.0, -7558.33, -7550.0, -7558.33, -7558.33, -7558.33, -7558.33, -7558.33, -7566.67, -7558.33, -7575.0, -7558.33, -7558.33, -7558.33, -7558.33, -7550.0, -7550.0, -7541.67, -7575.0, -7566.67, -7558.33, -7541.67, -7550.0, -7550.0, -7541.67, -7550.0, -7566.67, -7541.67, -7550.0, -7566.67, -7541.67, -7500.0, -7533.33, -7591.67, -7550.0, -7500.0, -7525.0, -7516.67, -7508.33, -7500.0, -7500.0, -7508.33, -7550.0, -7558.33, -7541.67, -7475.0, -7525.0, -7466.67, -7416.67, -7508.33, -7566.67, -7366.67, -3683.33, -9600.0, -7291.67, -7658.33, -9683.33, -6691.67, -6041.67, -10708.33, -8000.0, -8050.0, -8733.33, -8600.0, -8500.0, -8725.0, -8641.67, -8641.67, -8675.0, -8566.67, -8650.0, -9308.33, -9050.0, -8725.0, -8683.33, -8483.33, -8441.67, -9533.33, -9175.0, -11908.33, -12683.33, -10033.33, -7858.33, -9675.0, -10408.33, -12341.67, -12766.67, -13991.67, -16400.0, -18566.67, -20308.33, -22216.67], [-11150.0, -11141.67, -11741.67, -12050.0, -12241.67, -12191.67, -11950.0, -11725.0, -10591.67, -10033.33, -9416.67, -8591.67, -9150.0, -9383.33, -9750.0, -9291.67, -9183.33, -9158.33, -7375.0, -8225.0, -8075.0, -7466.67, -7375.0, -8083.33, -7700.0, -7633.33, -7908.33, -8400.0, -7866.67, -8175.0, -8916.67, -8408.33, -8316.67, -7891.67, -8400.0, -8058.33, -7633.33, -7675.0, -7675.0, -7758.33, -7616.67, -7541.67, -7558.33, -7466.67, -7375.0, -7316.67, -7400.0, -7341.67, -7316.67, -7141.67, -7200.0, -7091.67, -7141.67, -7008.33, -6916.67, -7008.33, -6966.67, -7108.33, -7291.67, -7366.67, -7333.33, -7400.0, -7416.67, -7408.33, -7375.0, -7383.33, -7433.33, -7400.0, -7450.0, -7391.67, -7400.0, -7400.0, -7375.0, -7441.67, -7358.33, -7400.0, -7375.0, -7333.33, -7308.33, -7266.67, -7225.0, -7241.67, -7166.67, -7166.67, -7141.67, -7091.67, -7083.33, -7100.0, -7083.33, -7075.0, -7116.67, -7058.33, -7100.0, -7083.33, -7050.0, -7083.33, -7058.33, -7041.67, -7066.67, -6950.0, -6991.67, -7000.0, -6900.0, -6983.33, -6925.0, -6908.33, -6950.0, -6950.0, -6900.0, -6916.67, -6900.0, -6875.0, -6875.0, -6850.0, -6858.33, -6883.33, -6825.0, -6841.67, -6875.0, -6816.67, -6841.67, -6858.33, -6825.0, -6791.67, -6866.67, -6783.33, -6800.0, -6775.0, -6750.0, -6725.0, -6725.0, -6708.33, -6683.33, -6641.67, -6666.67, -6625.0, -6600.0, -6600.0, -6591.67, -6591.67, -6608.33, -6600.0, -6550.0, -6616.67, -6608.33, -6575.0, -6616.67, -6616.67, -6600.0, -6583.33, -6575.0, -6558.33, -6558.33, -6508.33, -6500.0, -6491.67, -6400.0, -6208.33, -5933.33, -4975.0, -5050.0, -5208.33, -5116.67, -5508.33, -5491.67, -5283.33, -4983.33, -4833.33, -4675.0, -4708.33, -4641.67, -4566.67, -4533.33, -4658.33, -4666.67, -5100.0, -5033.33, -4858.33, -5216.67, -4975.0, -4875.0, -5066.67, -4741.67, -4441.67, -4516.67, -4658.33, -4225.0, -4333.33, -4383.33, -4225.0, -4275.0, -4266.67, -4325.0, -4241.67, -4241.67, -4241.67, -4241.67, -4208.33, -4200.0, -4191.67], [-4175.0, -4183.33, -4141.67, -4558.33, -4350.0, -4308.33, -4391.67, -3991.67, -3866.67, -4150.0, -4550.0, -4683.33, -4491.67, -4691.67, -4883.33, -5066.67, -5075.0, -5058.33, -5433.33, -5250.0, -5366.67, -5125.0, -5991.67, -5483.33, -5341.67, -5583.33, -5266.67, -4958.33, -5008.33, -5083.33, -4950.0, -4800.0, -4525.0, -4325.0, -4208.33, -3766.67, -3808.33, -4083.33, -4375.0, -4425.0, -4433.33, -4658.33, -4525.0, -4525.0, -4700.0, -4616.67, -4483.33, -4525.0, -4508.33, -4600.0, -4258.33, -4675.0, -4325.0, -4650.0, -4425.0, -4466.67, -4366.67, -4533.33, -4433.33, -4450.0, -4508.33, -4425.0, -4483.33, -4366.67, -4525.0, -4391.67, -4458.33, -4350.0, -4266.67, -4366.67, -4391.67, -4350.0, -4275.0, -4308.33, -4333.33, -4300.0, -4316.67, -4358.33, -4308.33, -4433.33, -4408.33, -4216.67, -4325.0, -4316.67, -4258.33, -4266.67, -4241.67, -4216.67, -4300.0, -4291.67, -4116.67, -4208.33, -4208.33, -4150.0, -4200.0, -4183.33, -4216.67, -4183.33, -4250.0, -4233.33, -4041.67, -4391.67, -3925.0, -4250.0, -4841.67, -5308.33, -5025.0, -5000.0, -5025.0, -5050.0, -5025.0, -4983.33, -4766.67, -4808.33, -4850.0, -4716.67, -5516.67, -6458.33, -6391.67, -6033.33, -6633.33, -6191.67, -6083.33, -6100.0, -5775.0, -5850.0, -5883.33, -5691.67, -5658.33, -5833.33, -5550.0, -5308.33, -5133.33, -5375.0, -5258.33, -5066.67, -5100.0, -4958.33, -4925.0, -5058.33, -4858.33, -5166.67, -5175.0, -4841.67, -4925.0, -4958.33, -4700.0, -4841.67, -4750.0, -4616.67, -4658.33, -4141.67, -3841.67, -4041.67, -3600.0, -4150.0, -4333.33, -4166.67, -4133.33, -4216.67, -4225.0, -4116.67, -4216.67, -4150.0, -4050.0, -4233.33, -4141.67, -4116.67, -4083.33, -4116.67, -3841.67, -4250.0, -4216.67, -4108.33, -4350.0, -4516.67, -4833.33, -4841.67, -4916.67, -4783.33, -5033.33, -4850.0, -5016.67, -5075.0, -5133.33, -5133.33, -4850.0, -4150.0, -3816.67, -3766.67, -4083.33, -4233.33, -4358.33, -4091.67, -4025.0, -3991.67, -3983.33, -3808.33, -3533.33, -3791.67], [-3808.33, -3800.0, -4000.0, -4158.33, -3825.0, -3833.33, -3908.33, -3708.33, -3566.67, -3641.67, -3475.0, -3491.67, -3675.0, -3458.33, -3525.0, -3558.33, -4291.67, -4308.33, -4275.0, -4400.0, -4450.0, -4491.67, -4516.67, -4500.0, -4550.0, -4500.0, -4483.33, -4475.0, -4500.0, -4458.33, -4425.0, -4516.67, -4591.67, -4391.67, -4500.0, -4425.0, -4375.0, -4566.67, -4450.0, -4408.33, -4583.33, -4491.67, -4358.33, -4616.67, -4416.67, -4466.67, -4633.33, -4425.0, -4383.33, -4583.33, -4508.33, -4400.0, -4450.0, -4450.0, -4408.33, -4416.67, -4383.33, -4366.67, -4333.33, -4350.0, -4300.0, -4300.0, -4300.0, -4300.0, -4308.33, -4333.33, -4308.33, -4266.67, -4358.33, -4350.0, -4416.67, -4308.33, -4316.67, -4350.0, -4266.67, -4358.33, -4266.67, -3691.67, -3608.33, -4050.0, -4441.67, -4208.33, -4733.33, -4483.33, -4483.33, -4883.33, -4808.33, -4541.67, -4558.33, -4825.0, -4841.67, -4825.0, -4716.67, -4775.0, -4866.67, -4833.33, -4841.67, -4841.67, -4850.0, -4741.67, -4850.0, -5133.33, -4800.0, -4925.0, -4883.33, -4808.33, -4933.33, -5066.67, -4775.0, -4833.33, -5125.0, -5016.67, -4875.0, -4383.33, -4400.0, -4008.33, -4025.0, -4316.67, -3683.33, -5225.0, -4383.33, -4333.33, -4458.33, -3966.67, -4516.67, -3800.0, -3800.0, -4175.0, -3991.67, -3916.67, -3833.33, -3683.33, -3558.33, -3816.67, -4033.33, -3950.0, -3900.0, -3775.0, -4016.67, -3691.67, -3666.67, -3725.0, -3633.33, -3783.33, -3483.33, -3316.67, -3750.0, -3591.67, -3433.33, -3591.67, -3483.33, -3358.33, -3458.33, -3408.33, -2975.0, -2850.0, -3158.33, -2641.67, -2825.0, -2816.67, -2891.67, -2716.67, -3041.67, -2641.67, -3216.67, -3233.33, -3683.33, -3541.67, -3325.0, -3150.0, -3291.67, -3550.0, -3783.33, -3741.67, -3758.33, -3758.33, -3791.67, -3816.67, -2908.33, -3700.0, -3550.0, -3483.33, -3391.67, -3508.33, -3450.0, -3525.0, -3525.0, -3525.0, -3500.0, -3483.33, -3541.67, -3483.33, -3383.33, -3375.0, -3383.33, -3433.33, -3408.33, -3408.33, -3325.0, -3266.67], [-3358.33, -3350.0, -3333.33, -3408.33, -3366.67, -3341.67, -3300.0, -3475.0, -3325.0, -3275.0, -3141.67, -3291.67, -3391.67, -3108.33, -3133.33, -3325.0, -3441.67, -3283.33, -3283.33, -3366.67, -3325.0, -3258.33, -3333.33, -3300.0, -3225.0, -3308.33, -3266.67, -3175.0, -3175.0, -3316.67, -3083.33, -3025.0, -3308.33, -3300.0, -3125.0, -3175.0, -3225.0, -3191.67, -3133.33, -3341.67, -3291.67, -3166.67, -3250.0, -3241.67, -3208.33, -3233.33, -3225.0, -3241.67, -3208.33, -3225.0, -3241.67, -3216.67, -3325.0, -3283.33, -3208.33, -3300.0, -3225.0, -3233.33, -3250.0, -3250.0, -3258.33, -3266.67, -3258.33, -3275.0, -3258.33, -3258.33, -3283.33, -3241.67, -3283.33, -3258.33, -3250.0, -3258.33, -3241.67, -3241.67, -3225.0, -3208.33, -3225.0, -3200.0, -3200.0, -3191.67, -3191.67, -3191.67, -3200.0, -3175.0, -3175.0, -3183.33, -3150.0, -3150.0, -3166.67, -3158.33, -3166.67, -3158.33, -3125.0, -3158.33, -3158.33, -3141.67, -3158.33, -3158.33, -3150.0, -3175.0, -3166.67, -3150.0, -2891.67, -2433.33, -1941.67, -2041.67, -2800.0, -2966.67, -3100.0, -2916.67, -3433.33, -3166.67, -3100.0, -3141.67, -2858.33, -2616.67, -2758.33, -2858.33, -2858.33, -2983.33, -3066.67, -3325.0, -3216.67, -3258.33, -3333.33, -3275.0, -3458.33, -3691.67, -3533.33, -3525.0, -3291.67, -3291.67, -3383.33, -3250.0, -3225.0, -3258.33, -3191.67, -3241.67, -3216.67, -3208.33, -3233.33, -3200.0, -3100.0, -3241.67, -3191.67, -3125.0, -3150.0, -3191.67, -3208.33, -3191.67, -3200.0, -3191.67, -2708.33, -2691.67, -2683.33, -2666.67, -2683.33, -2691.67, -2733.33, -2783.33, -2800.0, -2816.67, -2816.67, -2808.33, -2758.33, -2825.0, -2825.0, -2816.67, -2708.33, -2708.33, -2633.33, -2716.67, -2633.33, -2683.33, -2600.0, -2175.0, -2191.67, -2150.0, -2258.33, -2241.67, -2100.0, -2108.33, -2075.0, -2241.67, -2366.67, -2141.67, -2091.67, -2341.67, -2116.67, -2250.0, -2058.33, -1991.67, -2125.0, -2183.33, -2075.0, -2400.0, -2283.33, -2258.33, -2350.0, -2358.33], [-2300.0, -2516.67, -2525.0, -2233.33, -2508.33, -2508.33, -2350.0, -2300.0, -2641.67, -2391.67, -2250.0, -2575.0, -2308.33, -2475.0, -2458.33, -2333.33, -2400.0, -2450.0, -2383.33, -2408.33, -2383.33, -2433.33, -2408.33, -2408.33, -2400.0, -2383.33, -2375.0, -2383.33, -2350.0, -2358.33, -2366.67, -2366.67, -2341.67, -2350.0, -2350.0, -2333.33, -2350.0, -2350.0, -2358.33, -2375.0, -2375.0, -2383.33, -2391.67, -2391.67, -2400.0, -2391.67, -2408.33, -2408.33, -2400.0, -2383.33, -2383.33, -2358.33, -2350.0, -2383.33, -2358.33, -2333.33, -2358.33, -2358.33, -2341.67, -2341.67, -2350.0, -2333.33, -2316.67, -2341.67, -2341.67, -2341.67, -2358.33, -2358.33, -2358.33, -2358.33, -2375.0, -2391.67, -2350.0, -2400.0, -2383.33, -2366.67, -2375.0, -2375.0, -2366.67, -2350.0, -2358.33, -2333.33, -2358.33, -2333.33, -2316.67, -2333.33, -2325.0, -2308.33, -2333.33, -2333.33, -2291.67, -2350.0, -2341.67, -2341.67, -2333.33, -2466.67, -2183.33, -2225.0, -2375.0, -2533.33, -2058.33, -2650.0, -2208.33, -2450.0, -2616.67, -2183.33, -2566.67, -2558.33, -2191.67, -2558.33, -2433.33, -2283.33, -2425.0, -2366.67, -2275.0, -2500.0, -2516.67, -2333.33, -2525.0, -2466.67, -2550.0, -2566.67, -2516.67, -2541.67, -2458.33, -2483.33, -2333.33, -2500.0, -2391.67, -2450.0, -2441.67, -2433.33, -2483.33, -2408.33, -2516.67, -2458.33, -2425.0, -2508.33, -2491.67, -2458.33, -2508.33, -2508.33, -2558.33, -2566.67, -2583.33, -2625.0, -2625.0, -2558.33, -2575.0, -2625.0, -2575.0, -2583.33, -2658.33, -2583.33, -2583.33, -2608.33, -2616.67, -2625.0, -2600.0, -2658.33, -2675.0, -2600.0, -2658.33, -2650.0, -2683.33, -2641.67, -2641.67, -2675.0, -2691.67, -2775.0, -2716.67, -2766.67, -2875.0, -2741.67, -2741.67, -2758.33, -2641.67, -2716.67, -2725.0, -2683.33, -2700.0, -2575.0, -2866.67, -2783.33, -2583.33, -2583.33, -2783.33, -2641.67, -2650.0, -2691.67, -2683.33, -2741.67, -2666.67, -2691.67, -2708.33, -2691.67, -2708.33, -2700.0, -2683.33, -2700.0], [-2491.67, -2516.67, -2608.33, -2583.33, -2608.33, -2608.33, -2508.33, -2558.33, -2600.0, -2533.33, -2566.67, -2833.33, -2808.33, -2658.33, -2608.33, -2600.0, -2616.67, -2558.33, -2783.33, -2700.0, -2691.67, -2533.33, -2775.0, -2483.33, -2483.33, -2583.33, -2550.0, -2458.33, -2233.33, -2308.33, -2250.0, -2233.33, -2325.0, -2275.0, -2291.67, -2183.33, -2375.0, -2433.33, -2225.0, -2366.67, -2291.67, -2258.33, -2333.33, -2366.67, -2316.67, -2358.33, -2258.33, -2366.67, -2316.67, -2308.33, -2458.33, -2458.33, -2508.33, -2550.0, -2491.67, -2366.67, -2458.33, -2433.33, -2391.67, -2108.33, -2475.0, -2358.33, -2400.0, -2383.33, -2425.0, -2391.67, -2383.33, -2375.0, -2358.33, -2366.67, -2433.33, -2383.33, -2408.33, -2508.33, -2400.0, -2366.67, -2433.33, -2533.33, -2416.67, -2408.33, -2333.33, -2233.33, -2116.67, -2091.67, -2083.33, -2091.67, -2075.0, -2083.33, -2091.67, -2066.67, -2066.67, -2058.33, -2108.33, -2083.33, -2166.67, -2141.67, -2191.67, -2300.0, -2300.0, -2316.67, -2350.0, -2241.67, -2308.33, -2383.33, -2341.67, -2383.33, -2500.0, -2491.67, -2425.0, -2391.67, -2425.0, -2466.67, -2358.33, -2391.67, -2383.33, -2416.67, -2566.67, -2700.0, -2516.67, -2358.33, -2533.33, -2566.67, -2483.33, -2591.67, -2508.33, -2516.67, -2566.67, -2550.0, -2550.0, -2491.67, -2533.33, -2641.67, -2533.33, -2516.67, -2516.67, -2466.67, -2391.67, -2700.0, -2616.67, -2566.67, -2508.33, -2533.33, -2525.0, -2541.67, -2400.0, -2758.33, -2591.67, -2300.0, -2683.33, -2725.0, -2308.33, -2716.67, -2475.0, -2383.33, -2800.0, -2325.0, -2483.33, -2775.0, -2300.0, -2691.67, -2733.33, -2250.0, -2433.33, -2816.67, -2441.67, -2550.0, -2516.67, -2391.67, -2716.67, -2333.33, -2475.0, -2700.0, -2266.67, -2675.0, -2658.33, -2233.33, -2675.0, -2691.67, -2208.33, -2708.33, -2641.67, -2141.67, -2683.33, -2383.33, -2116.67, -2700.0, -2200.0, -2366.67, -2608.33, -2075.0, -2666.67, -2400.0, -2091.67, -2758.33, -2325.0, -2025.0, -2383.33, -1816.67, -1741.67, -1991.67], [-2308.33, -2316.67, -2300.0, -2283.33, -2275.0, -2300.0, -2291.67, -2258.33, -2283.33, -2283.33, -2266.67, -2275.0, -2258.33, -2233.33, -2291.67, -2258.33, -2241.67, -2258.33, -2291.67, -2325.0, -2341.67, -2408.33, -2425.0, -2333.33, -2450.0, -2250.0, -2125.0, -2133.33, -1991.67, -1975.0, -1958.33, -2033.33, -1941.67, -1925.0, -1966.67, -2033.33, -2000.0, -2083.33, -2191.67, -2175.0, -2125.0, -2091.67, -2308.33, -2183.33, -2133.33, -1633.33, -1716.67, -1750.0, -2000.0, -2000.0, -1483.33, -2225.0, -2075.0, -1875.0, -1750.0, -2050.0, -1950.0, -2108.33, -1900.0, -1933.33, -1966.67, -1958.33, -1991.67, -1933.33, -1925.0, -1900.0, -1983.33, -2025.0, -1908.33, -1916.67, -2033.33, -1950.0, -1975.0, -1950.0, -1966.67, -1983.33, -1991.67, -1950.0, -1966.67, -2150.0, -2041.67, -2000.0, -2000.0, -2000.0, -1916.67, -1633.33, -2041.67, -3308.33, -4141.67, -6366.67, -7516.67, -7900.0, -8516.67, -8983.33, -9666.67, -10058.33, -11033.33, -4491.67, -408.33, 1783.33, 4091.67, 4358.33, 4291.67, 4508.33, 4283.33, 3358.33, 3408.33, 3258.33, 3158.33, 3216.67, 3208.33, 3158.33, 3000.0, 3050.0, 2775.0, 2883.33, 2750.0, 2766.67, 2616.67, 2833.33, 2841.67, 2858.33, 2641.67, 2775.0, 3025.0, 1633.33, -875.0, -1766.67, -2791.67, -3666.67, -4775.0, -4691.67, -4808.33, -6416.67, -6941.67, -7325.0, -8000.0, -8525.0, -8308.33, -9566.67, -9650.0, -10550.0, -10033.33, -10158.33, -10425.0, -10883.33, -11783.33, -11316.67, -11125.0, -11075.0, -11258.33, -11158.33, -11316.67, -11208.33, -11133.33, -11366.67, -11425.0, -11250.0, -11416.67, -12108.33, -11900.0, -11233.33, -11583.33, -11300.0, -11700.0, -12800.0, -11891.67, -12633.33, -12258.33, -12300.0, -11916.67, -10933.33, -10483.33, -8950.0, -7141.67, -6250.0, -5491.67, -4741.67, -4258.33, -3866.67, -3675.0, -4125.0, -4150.0, -4000.0, -2833.33, -341.67, 1225.0, 866.67, 1550.0, 550.0, 2000.0, 1900.0, 1533.33, 3116.67, 1158.33, 150.0, -675.0, 175.0, -150.0, -2666.67], [-8800.0, -8700.0, -8625.0, -8666.67, -8525.0, -8500.0, -8416.67, -8216.67, -8191.67, -8133.33, -7966.67, -7958.33, -7800.0, -7658.33, -7433.33, -7041.67, -6808.33, -6508.33, -6008.33, -5458.33, -5533.33, -5325.0, -5066.67, -4691.67, -4725.0, -4533.33, -4583.33, -3891.67, -2475.0, 3650.0, 5275.0, 10291.67, 15258.33, 17333.33, 16633.33, 14175.0, 10341.67, 14141.67, 10575.0, 10566.67, 11025.0, 10166.67, 9325.0, 8908.33, 6900.0, 4550.0, 4575.0, 4583.33, 4658.33, 4675.0, 3650.0, 5441.67, 4658.33, 3941.67, 2850.0, 4358.33, 3300.0, 2366.67, 2891.67, 2883.33, 2583.33, 2775.0, 2233.33, 1783.33, 2491.67, 2125.0, 1866.67, 1933.33, 1558.33, 1475.0, 1508.33, 1508.33, 1550.0, 1683.33, 1658.33, 1625.0, 1833.33, 1716.67, 1541.67, 1558.33, 1483.33, 1000.0, 1325.0, 1366.67, 1025.0, 1116.67, 1208.33, 1100.0, 1166.67, 1258.33, 1200.0, 1050.0, 1141.67, 1183.33, 1175.0, 1208.33, 1150.0, 1233.33, 1125.0, 1108.33, 1033.33, 1325.0, 1425.0, 1416.67, 1391.67, 1375.0, 1033.33, 1016.67, 1108.33, 1016.67, 950.0, 1033.33, 1058.33, 983.33, 966.67, 975.0, 983.33, 983.33, 958.33, 933.33, 975.0, 975.0, 941.67, 975.0, 950.0, 950.0, 991.67, 966.67, 925.0, 883.33, 941.67, 866.67, 883.33, 925.0, 958.33, 916.67, 975.0, 916.67, 966.67, 941.67, 875.0, 841.67, 825.0, 958.33, 550.0, 775.0, 716.67, 666.67, 758.33, 716.67, 683.33, 658.33, 658.33, 608.33, 608.33, 608.33, 633.33, 666.67, 691.67, 700.0, 591.67, 741.67, 658.33, 716.67, 675.0, 650.0, 666.67, 658.33, 666.67, 666.67, 666.67, 675.0, 683.33, 658.33, 675.0, 708.33, 708.33, 725.0, 741.67, 741.67, 750.0, 791.67, 716.67, 716.67, 800.0, 258.33, 433.33, -133.33, 191.67, 8.33, -350.0, 50.0, 1291.67, 1341.67, 1225.0, 983.33, 625.0, 616.67, 933.33, 1116.67], [908.33, 783.33, 908.33, 858.33, 833.33, 758.33, 808.33, 758.33, 708.33, 616.67, 433.33, 841.67, 458.33, 158.33, 275.0, 358.33, 600.0, 475.0, 683.33, 458.33, 408.33, 641.67, 475.0, 391.67, 466.67, 500.0, 83.33, 225.0, 2600.0, 5500.0, 8158.33, 7566.67, 6150.0, 3858.33, 4225.0, 5966.67, 7358.33, 7200.0, 7216.67, 5783.33, 4566.67, 3550.0, 4616.67, 7716.67, 8308.33, 8800.0, 8266.67, 8008.33, 7025.0, 8566.67, 8325.0, 6616.67, 4483.33, 3750.0, 3358.33, 2950.0, 2766.67, 2875.0, 2500.0, 2350.0, 2375.0, 2275.0, 2400.0, 2300.0, 2075.0, 2075.0, 2250.0, 1941.67, 1991.67, 1983.33, 1925.0, 1950.0, 1816.67, 1766.67, 1708.33, 1700.0, 1650.0, 1608.33, 1516.67, 1600.0, 1625.0, 1525.0, 1525.0, 1483.33, 1433.33, 1441.67, 1441.67, 1416.67, 1475.0, 1408.33, 1566.67, 1600.0, 1575.0, 1508.33, 1458.33, 1400.0, 1325.0, 1258.33, 1175.0, 1083.33, 1025.0, 983.33, 991.67, 975.0, 983.33, 1033.33, 1033.33, 1025.0, 1083.33, 1000.0, 1075.0, 1216.67, 1041.67, 1191.67, 1241.67, 1350.0, 1491.67, 1308.33, 1583.33, 1558.33, 1750.0, 1816.67, 1825.0, 1766.67, 1875.0, 1825.0, 2241.67, 1891.67, 1783.33, 1883.33, 1608.33, 1425.0, 1483.33, 1458.33, 1391.67, 1383.33, 1366.67, 1366.67, 1233.33, 1208.33, 1183.33, 1208.33, 1208.33, 1200.0, 1233.33, 1283.33, 1266.67, 1250.0, 1258.33, 1250.0, 1258.33, 1216.67, 1183.33, 1183.33, 1158.33, 1125.0, 1091.67, 1050.0, 1033.33, 1000.0, 1016.67, 991.67, 975.0, 1000.0, 958.33, 975.0, 1000.0, 933.33, 1075.0, 1050.0, 1025.0, 1041.67, 1066.67, 1050.0, 1125.0, 1050.0, 1066.67, 1083.33, 1075.0, 1175.0, 1183.33, 900.0, 741.67, 333.33, 208.33, -316.67, -166.67, 325.0, 33.33, -416.67, -408.33, -358.33, -858.33, -933.33, -1258.33, -1500.0, -1625.0, -1508.33, -1475.0, -1458.33], [-6025.0, -5958.33, -6116.67, -5416.67, -5425.0, -5583.33, -5550.0, -5458.33, -5508.33, -5475.0, -5525.0, -5350.0, -5316.67, -5208.33, -5150.0, -5183.33, -5133.33, -5175.0, -5141.67, -5366.67, -5416.67, -5250.0, -5241.67, -5341.67, -5433.33, -5675.0, -5916.67, -5975.0, -5833.33, -5900.0, -5991.67, -5966.67, -5850.0, -5891.67, -5966.67, -6008.33, -6100.0, -6100.0, -6200.0, -6216.67, -6158.33, -6300.0, -6275.0, -6225.0, -6300.0, -6275.0, -6291.67, -6266.67, -6308.33, -6241.67, -6275.0, -6300.0, -6166.67, -6291.67, -6300.0, -6141.67, -6150.0, -6108.33, -6058.33, -6083.33, -6016.67, -6033.33, -6083.33, -5975.0, -5966.67, -5925.0, -5975.0, -5933.33, -5916.67, -5866.67, -5875.0, -5800.0, -5775.0, -5800.0, -5883.33, -5858.33, -5833.33, -5941.67, -5925.0, -5908.33, -5941.67, -6000.0, -6033.33, -5908.33, -6225.0, -6208.33, -6141.67, -6225.0, -6108.33, -6058.33, -6058.33, -5933.33, -5958.33, -6033.33, -5958.33, -6225.0, -6966.67, -7966.67, -7983.33, -8233.33, -8650.0, -8691.67, -8566.67, -8575.0, -8383.33, -8383.33, -8316.67, -8183.33, -8200.0, -8100.0, -8058.33, -7958.33, -8008.33, -7975.0, -7925.0, -7975.0, -7983.33, -7900.0, -7900.0, -7875.0, -7616.67, -7858.33, -7933.33, -7833.33, -7866.67, -7966.67, -8041.67, -8050.0, -8058.33, -8066.67, -8016.67, -8016.67, -8025.0, -8000.0, -7991.67, -8033.33, -8033.33, -8041.67, -8016.67, -8058.33, -8000.0, -7983.33, -7975.0, -7966.67, -7916.67, -8008.33, -8016.67, -7958.33, -7966.67, -8025.0, -7900.0, -7941.67, -7966.67, -7941.67, -7966.67, -7975.0, -8008.33, -7941.67, -7975.0, -7883.33, -7783.33, -7816.67, -7741.67, -7666.67, -7925.0, -7675.0, -7625.0, -7675.0, -7650.0, -7650.0, -7700.0, -7675.0, -7658.33, -7708.33, -7691.67, -7716.67, -7683.33, -7791.67, -7733.33, -7783.33, -7783.33, -7758.33, -7775.0, -7758.33, -7891.67, -7725.0, -7791.67, -7783.33, -7666.67, -7666.67, -7683.33, -7633.33, -7683.33, -7658.33, -7666.67, -7650.0, -7691.67, -7675.0, -7608.33, -7700.0], [-7833.33, -7850.0, -7841.67, -7900.0, -7850.0, -7950.0, -7925.0, -7925.0, -7941.67, -7916.67, -7908.33, -7941.67, -7875.0, -7866.67, -7850.0, -7808.33, -7808.33, -7791.67, -7800.0, -7758.33, -7816.67, -7791.67, -7858.33, -7916.67, -7933.33, -8000.0, -7991.67, -8016.67, -8041.67, -8025.0, -8025.0, -8058.33, -8025.0, -8033.33, -8000.0, -8016.67, -8008.33, -7950.0, -7950.0, -7950.0, -7933.33, -7950.0, -7966.67, -7950.0, -7925.0, -7950.0, -7908.33, -7950.0, -7966.67, -8008.33, -8083.33, -8108.33, -8166.67, -8166.67, -8266.67, -8300.0, -8341.67, -8341.67, -8333.33, -8341.67, -8341.67, -8333.33, -8391.67, -8266.67, -8483.33, -8458.33, -8358.33, -8416.67, -8358.33, -8408.33, -8475.0, -8400.0, -8441.67, -8441.67, -8391.67, -8416.67, -8416.67, -8408.33, -8441.67, -8433.33, -8491.67, -8475.0, -8741.67, -8825.0, -8791.67, -8808.33, -8916.67, -9066.67, -9275.0, -9200.0, -9208.33, -9208.33, -9150.0, -9183.33, -9100.0, -9125.0, -9108.33, -9083.33, -9091.67, -9075.0, -9075.0, -9058.33, -9108.33, -9100.0, -9141.67, -9175.0, -9200.0, -9200.0, -9300.0, -9341.67, -9350.0, -9400.0, -9400.0, -9391.67, -9391.67, -9391.67, -9416.67, -9383.33, -9375.0, -9366.67, -9333.33, -9341.67, -9325.0, -9333.33, -9316.67, -9308.33, -9325.0, -9375.0, -9275.0, -9250.0, -9175.0, -9108.33, -9083.33, -9041.67, -9058.33, -8858.33, -8991.67, -8916.67, -8891.67, -8925.0, -8891.67, -8891.67, -9075.0, -9233.33, -9358.33, -9233.33, -9241.67, -9233.33, -9225.0, -9200.0, -9141.67, -9175.0, -9141.67, -9175.0, -9150.0, -9058.33, -9208.33, -9125.0, -9016.67, -9200.0, -9150.0, -9141.67, -9100.0, -9091.67, -9141.67, -9016.67, -9083.33, -9016.67, -8950.0, -9033.33, -9033.33, -8908.33, -9033.33, -9050.0, -8983.33, -8875.0, -8933.33, -8958.33, -8958.33, -8875.0, -8958.33, -8941.67, -8916.67, -8800.0, -8975.0, -8958.33, -8941.67, -8833.33, -8941.67, -8908.33, -8891.67, -8783.33, -8866.67, -8841.67, -8825.0, -8783.33, -8741.67, -8825.0, -8808.33, -8775.0], [-9041.67, -8933.33, -8875.0, -8991.67, -8858.33, -8933.33, -8941.67, -8750.0, -8950.0, -8916.67, -8825.0, -9008.33, -8841.67, -8950.0, -8966.67, -8916.67, -9000.0, -8900.0, -8866.67, -8983.33, -8825.0, -9116.67, -9091.67, -9133.33, -9150.0, -8983.33, -9183.33, -8958.33, -8858.33, -9050.0, -8808.33, -8925.0, -8983.33, -8850.0, -8975.0, -8958.33, -8900.0, -9008.33, -8966.67, -8950.0, -8983.33, -8933.33, -9016.67, -8925.0, -8941.67, -9000.0, -8808.33, -8908.33, -9066.67, -8908.33, -9008.33, -9008.33, -9116.67, -9075.0, -9008.33, -9083.33, -9000.0, -9041.67, -9125.0, -9075.0, -9133.33, -9091.67, -9091.67, -9091.67, -9058.33, -9050.0, -9033.33, -9016.67, -9016.67, -9008.33, -8991.67, -9008.33, -9000.0, -9000.0, -8991.67, -9008.33, -8966.67, -9050.0, -9025.0, -9050.0, -9025.0, -9116.67, -8991.67, -9050.0, -9008.33, -9066.67, -9000.0, -8883.33, -8958.33, -8966.67, -8825.0, -8958.33, -8816.67, -8783.33, -8858.33, -8741.67, -8741.67, -8725.0, -8725.0, -8608.33, -8591.67, -8591.67, -8583.33, -8558.33, -8550.0, -8516.67, -8450.0, -8525.0, -8575.0, -8575.0, -8566.67, -8600.0, -8625.0, -8600.0, -8625.0, -8558.33, -8550.0, -8558.33, -8508.33, -8516.67, -8533.33, -8508.33, -8483.33, -8483.33, -8475.0, -8433.33, -8458.33, -8466.67, -8466.67, -8433.33, -8416.67, -8500.0, -8483.33, -8508.33, -8541.67, -8491.67, -8516.67, -8475.0, -8483.33, -8491.67, -8466.67, -8466.67, -8500.0, -8416.67, -8416.67, -8450.0, -8425.0, -8408.33, -8441.67, -8441.67, -8441.67, -8400.0, -8441.67, -8458.33, -8433.33, -8516.67, -8458.33, -8466.67, -8533.33, -8550.0, -8516.67, -8566.67, -8525.0, -8491.67, -8483.33, -8508.33, -8491.67, -8483.33, -8458.33, -8508.33, -8475.0, -8450.0, -8516.67, -8458.33, -8408.33, -8450.0, -8458.33, -8408.33, -8458.33, -8700.0, -8866.67, -9066.67, -9325.0, -9316.67, -9466.67, -9758.33, -10116.67, -10558.33, -10633.33, -10125.0, -9800.0, -9700.0, -9708.33, -9541.67, -9466.67, -9458.33, -9408.33, -9416.67, -9383.33, -9366.67], [-9183.33, -9166.67, -9166.67, -9191.67, -9133.33, -9200.0, -9266.67, -9275.0, -9283.33, -9225.0, -9225.0, -9250.0, -9241.67, -9275.0, -9316.67, -9400.0, -9416.67, -9500.0, -9450.0, -9516.67, -9491.67, -9508.33, -9458.33, -9491.67, -9441.67, -9466.67, -9441.67, -9450.0, -9450.0, -9500.0, -9533.33, -9525.0, -9508.33, -9550.0, -9541.67, -9566.67, -9533.33, -9525.0, -9533.33, -9525.0, -9608.33, -9533.33, -9575.0, -9533.33, -9516.67, -9516.67, -9500.0, -9500.0, -9516.67, -9483.33, -9558.33, -9550.0, -9558.33, -9558.33, -9541.67, -9516.67, -9508.33, -9500.0, -9458.33, -9491.67, -9508.33, -9508.33, -9525.0, -9550.0, -9533.33, -9541.67, -9550.0, -9525.0, -9558.33, -9541.67, -9533.33, -9525.0, -9558.33, -9558.33, -9558.33, -9600.0, -9575.0, -9616.67, -9583.33, -9558.33, -9566.67, -9583.33, -9550.0, -9575.0, -9616.67, -9600.0, -9658.33, -9683.33, -9775.0, -9725.0, -9758.33, -9733.33, -9700.0, -9766.67, -9733.33, -9816.67, -9850.0, -9883.33, -9916.67, -9900.0, -9941.67, -9958.33, -9933.33, -9958.33, -9950.0, -9916.67, -9908.33, -9908.33, -9916.67, -9933.33, -9866.67, -9916.67, -9891.67, -9891.67, -9875.0, -9891.67, -9908.33, -9908.33, -10075.0, -9875.0, -9675.0, -9350.0, -9025.0, -8541.67, -8291.67, -7808.33, -7725.0, -6566.67, -6225.0, -5675.0, -5375.0, -5433.33, -5241.67, -5075.0, -5000.0, -4483.33, -4625.0, -4466.67, -4250.0, -3908.33, -3758.33, -3550.0, -3483.33, -3500.0, -3333.33, -3316.67, -3191.67, -3016.67, -2891.67, -2575.0, -2691.67, -2300.0, -2266.67, -2116.67, -1850.0, -1700.0, -900.0, -1158.33, -941.67, -958.33, -508.33, -475.0, -883.33, -558.33, -150.0, -58.33, 175.0, 383.33, 358.33, 441.67, 516.67, 741.67, 650.0, 858.33, 900.0, 958.33, 1191.67, 1175.0, 1316.67, 1416.67, 1700.0, 1333.33, 1583.33, 1641.67, 1566.67, 1733.33, 1766.67, 1783.33, 1858.33, 1908.33, 1925.0, 1925.0, 1966.67, 2033.33, 2158.33, 2091.67, 2141.67, 2150.0, 2216.67, 2183.33], [2783.33, 2791.67, 2841.67, 2833.33, 2841.67, 2858.33, 2950.0, 2850.0, 2916.67, 3025.0, 2975.0, 3008.33, 3066.67, 2991.67, 3075.0, 3108.33, 3066.67, 3133.33, 3166.67, 3091.67, 3000.0, 3133.33, 3233.33, 3175.0, 2933.33, 3133.33, 3183.33, 3041.67, 2933.33, 3241.67, 3075.0, 2875.0, 3116.67, 3208.33, 3016.67, 3025.0, 3191.67, 3183.33, 3033.33, 3216.67, 3191.67, 3208.33, 3225.0, 3216.67, 3233.33, 3283.33, 3266.67, 3266.67, 3275.0, 3291.67, 3300.0, 3316.67, 3325.0, 3341.67, 3375.0, 3383.33, 3416.67, 3375.0, 4008.33, 5391.67, 6425.0, 6516.67, 5775.0, 3125.0, -1925.0, -2458.33, -3816.67, -4250.0, -4558.33, -4975.0, -4550.0, -4058.33, -3908.33, -3541.67, -3616.67, -3333.33, -2608.33, -2816.67, -2933.33, -2866.67, -2841.67, -2941.67, -2825.0, -2958.33, -2900.0, -3091.67, -2908.33, -2875.0, -2941.67, -2933.33, -3150.0, -3016.67, -3150.0, -3108.33, -3091.67, -3116.67, -3225.0, -3116.67, -3141.67, -3216.67, -3258.33, -3241.67, -3466.67, -2983.33, -3275.0, -3208.33, -3333.33, -3350.0, -3533.33, -3508.33, -3541.67, -3525.0, -3441.67, -3516.67, -3625.0, -3558.33, -3500.0, -3483.33, -3508.33, -3408.33, -3508.33, -3525.0, -3516.67, -3533.33, -3575.0, -3591.67, -3575.0, -3600.0, -3725.0, -3625.0, -3608.33, -3600.0, -3608.33, -3566.67, -3725.0, -3650.0, -3641.67, -3666.67, -3683.33, -3608.33, -3725.0, -3666.67, -3666.67, -3716.67, -3725.0, -3816.67, -3700.0, -3716.67, -3708.33, -3691.67, -3716.67, -3750.0, -3741.67, -3750.0, -3858.33, -3766.67, -3775.0, -3741.67, -3758.33, -3741.67, -3816.67, -3775.0, -3808.33, -3950.0, -3925.0, -4133.33, -4350.0, -4816.67, -5266.67, -8033.33, -13683.33, -2850.0, -650.0, 566.67, 2216.67, 2850.0, 2250.0, 2866.67, 3275.0, 3850.0, 4291.67, 5700.0, 4266.67, 3075.0, 2316.67, 2000.0, 2133.33, 1883.33, 2200.0, 2600.0, 3591.67, 3741.67, 3433.33, 4050.0, 4116.67, 4575.0, 4791.67, 5116.67, 5616.67, 5733.33], [6591.67, 6733.33, 6941.67, 7016.67, 7016.67, 7133.33, 7158.33, 7050.0, 7066.67, 7016.67, 7191.67, 7100.0, 7083.33, 7166.67, 7225.0, 7258.33, 7225.0, 7291.67, 7183.33, 7408.33, 7425.0, 7316.67, 7300.0, 7375.0, 7300.0, 7325.0, 7433.33, 7441.67, 7433.33, 7416.67, 7450.0, 7491.67, 7450.0, 7408.33, 7475.0, 7516.67, 7483.33, 7525.0, 7525.0, 7516.67, 7566.67, 7575.0, 7591.67, 7591.67, 7608.33, 7650.0, 7641.67, 7666.67, 7683.33, 7675.0, 7675.0, 7691.67, 7716.67, 7691.67, 7716.67, 7750.0, 7741.67, 7750.0, 7758.33, 7783.33, 7775.0, 7791.67, 7800.0, 7800.0, 7808.33, 7825.0, 7841.67, 7816.67, 7958.33, 8033.33, 8058.33, 8691.67, 7458.33, 6641.67, 4908.33, 2950.0, 2000.0, 950.0, -75.0, -16.67, -566.67, -858.33, -983.33, -1200.0, -1458.33, -1525.0, -1741.67, -1800.0, -1875.0, -1925.0, -2066.67, -2116.67, -2241.67, -2333.33, -2458.33, -2508.33, -2541.67, -2758.33, -2583.33, -2808.33, -2966.67, -2916.67, -2925.0, -3025.0, -3183.33, -3050.0, -3108.33, -3033.33, -3358.33, -3158.33, -3191.67, -3216.67, -3175.0, -3291.67, -3300.0, -3291.67, -3075.0, -3358.33, -3383.33, -3391.67, -3400.0, -3441.67, -3458.33, -3416.67, -3391.67, -3500.0, -3500.0, -3533.33, -3525.0, -3491.67, -3416.67, -3516.67, -3483.33, -3558.33, -3475.0, -3483.33, -3483.33, -3475.0, -3341.67, -3508.33, -3508.33, -3508.33, -3491.67, -3500.0, -3491.67, -3491.67, -3500.0, -3491.67, -3350.0, -3516.67, -3525.0, -3525.0, -3525.0, -3525.0, -3525.0, -3533.33, -3550.0, -3541.67, -3458.33, -3658.33, -3566.67, -3558.33, -3575.0, -3591.67, -3566.67, -3550.0, -3583.33, -3575.0, -3583.33, -3583.33, -3575.0, -3541.67, -3650.0, -3608.33, -3583.33, -3600.0, -3583.33, -3608.33, -3600.0, -3591.67, -3600.0, -3608.33, -3591.67, -3600.0, -3600.0, -3591.67, -3608.33, -3450.0, -3725.0, -3616.67, -3633.33, -3625.0, -3633.33, -3633.33, -3625.0, -3633.33, -3633.33, -3641.67, -3641.67, -3633.33], [-3666.67, -3658.33, -3666.67, -3666.67, -3658.33, -3691.67, -3658.33, -3650.0, -3666.67, -3658.33, -3658.33, -3633.33, -3650.0, -3650.0, -3633.33, -3641.67, -3658.33, -3641.67, -3658.33, -3658.33, -3650.0, -3641.67, -3658.33, -3658.33, -3666.67, -3666.67, -3641.67, -3666.67, -3675.0, -3650.0, -3675.0, -3683.33, -3666.67, -3691.67, -3666.67, -3683.33, -3683.33, -3675.0, -3691.67, -3675.0, -3683.33, -3691.67, -3533.33, -3708.33, -3708.33, -3700.0, -3683.33, -3691.67, -3716.67, -3708.33, -3700.0, -3700.0, -3716.67, -3691.67, -3683.33, -3725.0, -3700.0, -3666.67, -3733.33, -3700.0, -3725.0, -3691.67, -3700.0, -3683.33, -3700.0, -3700.0, -3708.33, -3658.33, -3700.0, -3708.33, -3691.67, -3608.33, -3691.67, -3716.67, -3708.33, -3683.33, -3725.0, -3725.0, -3725.0, -3741.67, -3691.67, -3725.0, -3741.67, -3758.33, -3741.67, -3741.67, -3758.33, -3750.0, -3758.33, -3758.33, -3758.33, -3766.67, -3741.67, -3758.33, -3758.33, -3741.67, -3741.67, -3733.33, -3741.67, -3741.67, -3741.67, -3733.33, -3725.0, -3775.0, -3750.0, -3766.67, -3725.0, -3741.67, -3750.0, -3750.0, -3750.0, -3758.33, -3758.33, -3750.0, -3750.0, -3750.0, -3750.0, -3766.67, -3750.0, -3741.67, -3741.67, -3758.33, -3758.33, -3758.33, -3758.33, -3750.0, -3758.33, -3758.33, -3750.0, -3741.67, -3741.67, -3750.0, -3750.0, -3750.0, -3758.33, -3783.33, -3741.67, -3766.67, -3750.0, -3650.0, -3775.0, -3791.67, -3775.0, -3800.0, -3783.33, -3783.33, -3791.67, -3808.33, -3775.0, -3825.0, -3775.0, -3808.33, -3808.33, -3783.33, -3791.67, -3791.67, -3791.67, -3800.0, -3783.33, -3791.67, -3783.33, -3783.33, -3783.33, -3783.33, -3800.0, -3783.33, -3791.67, -3791.67, -3791.67, -3783.33, -3783.33, -3791.67, -3791.67, -3800.0, -3791.67, -3783.33, -3816.67, -3800.0, -3783.33, -3808.33, -3791.67, -3800.0, -3783.33, -3800.0, -3791.67, -3808.33, -3791.67, -3800.0, -3783.33, -3808.33, -3800.0, -3800.0, -3800.0, -3791.67, -3791.67, -3800.0, -3825.0, -3800.0, -3791.67, -3808.33], [-3808.33, -3808.33, -3800.0, -3816.67, -3800.0, -3808.33, -3808.33, -3808.33, -3808.33, -3800.0, -3808.33, -3800.0, -3808.33, -3825.0, -3825.0, -3808.33, -3800.0, -3825.0, -3800.0, -3791.67, -3816.67, -3841.67, -3808.33, -3808.33, -3775.0, -3750.0, -3616.67, -3808.33, -3808.33, -3791.67, -3825.0, -3800.0, -3800.0, -3816.67, -3775.0, -3775.0, -3791.67, -3783.33, -3783.33, -3808.33, -3816.67, -3783.33, -3800.0, -3825.0, -3750.0, -3825.0, -3808.33, -3800.0, -3808.33, -3825.0, -3783.33, -3808.33, -3800.0, -3808.33, -3800.0, -3791.67, -3791.67, -3816.67, -3816.67, -3808.33, -3800.0, -3791.67, -3800.0, -3808.33, -3791.67, -3791.67, -3808.33, -3791.67, -3791.67, -3800.0, -3783.33, -3783.33, -3791.67, -3800.0, -3800.0, -3800.0, -3783.33, -3800.0, -3783.33, -3800.0, -3800.0, -3775.0, -3808.33, -3791.67, -3791.67, -3800.0, -3783.33, -3800.0, -3791.67, -3800.0, -3800.0, -3783.33, -3783.33, -3791.67, -3791.67, -3791.67, -3800.0, -3750.0, -3825.0, -3791.67, -3741.67, -3825.0, -3775.0, -3791.67, -3775.0, -3791.67, -3791.67, -3791.67, -3800.0, -3775.0, -3791.67, -3800.0, -3800.0, -3800.0, -3791.67, -3800.0, -3800.0, -3775.0, -3800.0, -3791.67, -3808.33, -3800.0, -3816.67, -3800.0, -3816.67, -3791.67, -3791.67, -3741.67, -3800.0, -3791.67, -3800.0, -3808.33, -3783.33, -3800.0, -3791.67, -3800.0, -3775.0, -3791.67, -3783.33, -3816.67, -3808.33, -3825.0, -3758.33, -3816.67, -3841.67, -3783.33, -3808.33, -3825.0, -3775.0, -3816.67, -3791.67, -3825.0, -3758.33, -3800.0, -3775.0, -3775.0, -3766.67, -3775.0, -3750.0, -3758.33, -3791.67, -3775.0, -3758.33, -3791.67, -3791.67, -3741.67, -3783.33, -3791.67, -3725.0, -3775.0, -3791.67, -3766.67, -3758.33, -3791.67, -3750.0, -3766.67, -3725.0, -3791.67, -3766.67, -3766.67, -3791.67, -3908.33, -3775.0, -3816.67, -3766.67, -3750.0, -3841.67, -3766.67, -3816.67, -3758.33, -3808.33, -3775.0, -3800.0, -3741.67, -3750.0, -3791.67, -3758.33, -3783.33, -3783.33, -3791.67]]

In [9]:
#testing database
#need another code to upload these heartrates into the database
arr = []
fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)
#maybe init
my_db = my_sql.connect(
        host="localhost",
        user="Arduino",
        passwd="ArduinoGetStarted.com",
        database="vital_signs")
my_cursor = my_db.cursor()
#global mult #testing to stop after 5
mult =0
table = "acceleration"

def get_bed_status(table): #select acceleration data from desired table - WORKS
    sql = "SELECT status FROM "+table+" ORDER BY id DESC LIMIT 3"
    my_cursor.execute(sql)
    bed_array = my_cursor.fetchall()
    reps = bed_array.count(0)
    if reps ==3: #found 3 in a row with no one there
        bed_status = 0
    else:
        bed_status = 1
    return bed_status

def select_accel(table): #select acceleration data from desired table - WORKS
    sql = "SELECT * FROM "+table
    my_cursor.execute(sql)
    data = my_cursor.fetchall()
    return data

#function 0: get data from desired table and set to variable called data
# data = select_accel("acceleration") 
# print(data)

def minute_data(data):
    xar = []
    for i in range(len(data)-100,len(data)): #get last 3 rows of data
    #for i in range(0,len(data)):
        yar = []
        xar.append(data[i][0]) #id
        #print(data[i][0],":\n")
        yar.append(data[i][5]) #array; row 4 = breathing rate
        yar = (data[i][5].split(","))
        for i in range(0,len(yar)-1):
            yar[i] = float(yar[i]) #convert string to float
        yar.pop() #removes the extra value
        #print(yar)
        #print("/n")
        arr.append(yar)
    graphData = arr #rename
    return (graphData)

def CalculateSingleHR(smoothrate,bandpassArr, samplingfreq, peakdis, peakprom, data, numberofperiods = 1):
    
    if isinstance(data, np.ndarray):
        print("Good to continue")    
    else:
        data = np.array(data)

    length = np.array(data).size
    #plt.plot(range(0,length), data, color = 'red')
    
    newArr = np.ones(length)
    
    sos = signal.butter(10, bandpassArr, 'band', fs=samplingfreq, output='sos')
    #sos = signal.butter(10, [20, 200], 'band', fs=450, output='sos')
    filterdata = signal.sosfilt(sos, data)
    #plt.plot(filterdata, color = "blue")

    for j in range(0, length-smoothrate):
        RMS = 0
        for k in range(0,smoothrate):
            RMS += (filterdata[j+k]) ** 2
        RMS = math.sqrt(RMS/(smoothrate))
        newArr[j] = RMS
    #plt.plot(range(0,length-smoothrate), newArr[:length-smoothrate])
    #plt.show()
    HR = 0

    peaks, _ = find_peaks(newArr, distance=peakdis, prominence = peakprom)
    #peaks, _ = find_peaks(newArr, distance=3, prominence = 1.5)


    HR = HR + len(peaks)*numberofperiods #6 for 10s
    #print(HR)
    
    #plt.plot(range(0, length- smoothrate),newArr[:length-smoothrate])
    #plt.plot(peaks, newArr[peaks], "x")
    #plt.show()
    predicted = HR
        
    return predicted

def insert_HR(HR, bed_status, table):
    sql = "INSERT INTO "+table+" (heart_rate, bed_status) VALUES (%s,%s)" #need 2 values or it won't work for me
    val = (HR, bed_status)
    my_cursor.execute(sql, val)
    my_db.commit() #need this after every execute!!
    #print("1 record inserted, ID:", my_cursor.lastrowid) #proof that it works

    #check length of database
    my_cursor.execute("SELECT COUNT(*) FROM "+table)
    data = my_cursor.fetchall()
    print("table size:" +str(data[0][0]))
    count = data[0][0]
    if count>100: #or can do it count>100
        my_cursor.execute("DELETE FROM "+table+" LIMIT 50") #can't use delete top w/ mysql - this will delete from row and limit it to the top 50 rows
        my_db.commit()
    #print(count[0][0]) #it's an array list so need to call as this to get the single count value
    #end of function 4 - maybe return size
    

bed_status = get_bed_status("acceleration")
if bed_status == 0:
    HR = 0
else:
#function 0: get data from desired table and set to variable called data
    data = select_accel("acceleration") 
    #print(data)
    print("\n")
    
    graphData = minute_data(data)
    print("Graph Data")
    print("\n")
    print(graphData)
    data = graphData[0]
    print(len(graphData))
    
    smoothrate = 10
    bandpass = [20,200]
    samplingfreq = 450
    peakdis = 3
    peakprom = 1.5
    HR=0
    
    for i in range(0,len(graphData)): #should be 3
            PredictedHR = CalculateSingleHR(smoothrate, bandpass, samplingfreq, peakdis, peakprom, graphData[i])
            # tempArr = np.array(graphData[i])
            # sr=len(tempArr)
            # tempArr = signal.sosfilt(sos, tempArr)
            # peaks, _ = find_peaks(tempArr, distance=3, prominence = 75)
            HR = HR + PredictedHR #6 for 10s
#print(HR)
#insert_HR(HR, bed_status, "test")
#PredictedHR = CalculateSingleHR(smoothrate, bandpass, samplingfreq, peakdis, peakprom, data)



Graph Data


[[-13608.33, -13650.0, -13625.0, -13633.33, -13625.0, -13625.0, -13575.0, -13608.33, -13625.0, -13616.67, -13666.67, -13658.33, -13625.0, -13633.33, -13675.0, -13600.0, -13608.33, -13625.0, -13625.0, -13616.67, -13625.0, -13583.33, -13583.33, -13575.0, -13583.33, -13600.0, -13716.67, -13541.67, -13616.67, -13600.0, -13583.33, -13575.0, -13583.33, -13575.0, -13625.0, -13583.33, -13575.0, -13608.33, -13608.33, -13591.67, -13625.0, -13633.33, -13600.0, -13633.33, -13633.33, -13600.0, -13608.33, -13625.0, -13608.33, -13575.0, -13608.33, -13525.0, -13583.33, -13525.0, -13558.33, -13608.33, -13491.67, -13550.0, -13575.0, -13566.67, -13558.33, -13583.33, -13608.33, -13566.67, -13583.33, -13583.33, -13583.33, -13591.67, -13583.33, -13600.0, -13616.67, -13575.0, -13658.33, -13633.33, -13733.33, -13675.0, -13650.0, -13683.33, -13633.33, -13591.67, -13658.33, -13675.0, -13633.33, -13666.67, -13633.33, -13600.0, -13600.0, -13575.0, -13608.33, -13591.67, -13616.67, -13616.67, -13625.

In [ ]:
[[6591.67, 6733.33, 6941.67, 7016.67, 7016.67, 7133.33, 7158.33, 7050.0, 7066.67, 7016.67, 7191.67, 7100.0, 7083.33, 7166.67, 7225.0, 7258.33, 7225.0, 7291.67, 7183.33, 7408.33, 7425.0, 7316.67, 7300.0, 7375.0, 7300.0, 7325.0, 7433.33, 7441.67, 7433.33, 7416.67, 7450.0, 7491.67, 7450.0, 7408.33, 7475.0, 7516.67, 7483.33, 7525.0, 7525.0, 7516.67, 7566.67, 7575.0, 7591.67, 7591.67, 7608.33, 7650.0, 7641.67, 7666.67, 7683.33, 7675.0, 7675.0, 7691.67, 7716.67, 7691.67, 7716.67, 7750.0, 7741.67, 7750.0, 7758.33, 7783.33, 7775.0, 7791.67, 7800.0, 7800.0, 7808.33, 7825.0, 7841.67, 7816.67, 7958.33, 8033.33, 8058.33, 8691.67, 7458.33, 6641.67, 4908.33, 2950.0, 2000.0, 950.0, -75.0, -16.67, -566.67, -858.33, -983.33, -1200.0, -1458.33, -1525.0, -1741.67, -1800.0, -1875.0, -1925.0, -2066.67, -2116.67, -2241.67, -2333.33, -2458.33, -2508.33, -2541.67, -2758.33, -2583.33, -2808.33, -2966.67, -2916.67, -2925.0, -3025.0, -3183.33, -3050.0, -3108.33, -3033.33, -3358.33, -3158.33, -3191.67, -3216.67, -3175.0, -3291.67, -3300.0, -3291.67, -3075.0, -3358.33, -3383.33, -3391.67, -3400.0, -3441.67, -3458.33, -3416.67, -3391.67, -3500.0, -3500.0, -3533.33, -3525.0, -3491.67, -3416.67, -3516.67, -3483.33, -3558.33, -3475.0, -3483.33, -3483.33, -3475.0, -3341.67, -3508.33, -3508.33, -3508.33, -3491.67, -3500.0, -3491.67, -3491.67, -3500.0, -3491.67, -3350.0, -3516.67, -3525.0, -3525.0, -3525.0, -3525.0, -3525.0, -3533.33, -3550.0, -3541.67, -3458.33, -3658.33, -3566.67, -3558.33, -3575.0, -3591.67, -3566.67, -3550.0, -3583.33, -3575.0, -3583.33, -3583.33, -3575.0, -3541.67, -3650.0, -3608.33, -3583.33, -3600.0, -3583.33, -3608.33, -3600.0, -3591.67, -3600.0, -3608.33, -3591.67, -3600.0, -3600.0, -3591.67, -3608.33, -3450.0, -3725.0, -3616.67, -3633.33, -3625.0, -3633.33, -3633.33, -3625.0, -3633.33, -3633.33, -3641.67, -3641.67, -3633.33], [-3666.67, -3658.33, -3666.67, -3666.67, -3658.33, -3691.67, -3658.33, -3650.0, -3666.67, -3658.33, -3658.33, -3633.33, -3650.0, -3650.0, -3633.33, -3641.67, -3658.33, -3641.67, -3658.33, -3658.33, -3650.0, -3641.67, -3658.33, -3658.33, -3666.67, -3666.67, -3641.67, -3666.67, -3675.0, -3650.0, -3675.0, -3683.33, -3666.67, -3691.67, -3666.67, -3683.33, -3683.33, -3675.0, -3691.67, -3675.0, -3683.33, -3691.67, -3533.33, -3708.33, -3708.33, -3700.0, -3683.33, -3691.67, -3716.67, -3708.33, -3700.0, -3700.0, -3716.67, -3691.67, -3683.33, -3725.0, -3700.0, -3666.67, -3733.33, -3700.0, -3725.0, -3691.67, -3700.0, -3683.33, -3700.0, -3700.0, -3708.33, -3658.33, -3700.0, -3708.33, -3691.67, -3608.33, -3691.67, -3716.67, -3708.33, -3683.33, -3725.0, -3725.0, -3725.0, -3741.67, -3691.67, -3725.0, -3741.67, -3758.33, -3741.67, -3741.67, -3758.33, -3750.0, -3758.33, -3758.33, -3758.33, -3766.67, -3741.67, -3758.33, -3758.33, -3741.67, -3741.67, -3733.33, -3741.67, -3741.67, -3741.67, -3733.33, -3725.0, -3775.0, -3750.0, -3766.67, -3725.0, -3741.67, -3750.0, -3750.0, -3750.0, -3758.33, -3758.33, -3750.0, -3750.0, -3750.0, -3750.0, -3766.67, -3750.0, -3741.67, -3741.67, -3758.33, -3758.33, -3758.33, -3758.33, -3750.0, -3758.33, -3758.33, -3750.0, -3741.67, -3741.67, -3750.0, -3750.0, -3750.0, -3758.33, -3783.33, -3741.67, -3766.67, -3750.0, -3650.0, -3775.0, -3791.67, -3775.0, -3800.0, -3783.33, -3783.33, -3791.67, -3808.33, -3775.0, -3825.0, -3775.0, -3808.33, -3808.33, -3783.33, -3791.67, -3791.67, -3791.67, -3800.0, -3783.33, -3791.67, -3783.33, -3783.33, -3783.33, -3783.33, -3800.0, -3783.33, -3791.67, -3791.67, -3791.67, -3783.33, -3783.33, -3791.67, -3791.67, -3800.0, -3791.67, -3783.33, -3816.67, -3800.0, -3783.33, -3808.33, -3791.67, -3800.0, -3783.33, -3800.0, -3791.67, -3808.33, -3791.67, -3800.0, -3783.33, -3808.33, -3800.0, -3800.0, -3800.0, -3791.67, -3791.67, -3800.0, -3825.0, -3800.0, -3791.67, -3808.33], [-3808.33, -3808.33, -3800.0, -3816.67, -3800.0, -3808.33, -3808.33, -3808.33, -3808.33, -3800.0, -3808.33, -3800.0, -3808.33, -3825.0, -3825.0, -3808.33, -3800.0, -3825.0, -3800.0, -3791.67, -3816.67, -3841.67, -3808.33, -3808.33, -3775.0, -3750.0, -3616.67, -3808.33, -3808.33, -3791.67, -3825.0, -3800.0, -3800.0, -3816.67, -3775.0, -3775.0, -3791.67, -3783.33, -3783.33, -3808.33, -3816.67, -3783.33, -3800.0, -3825.0, -3750.0, -3825.0, -3808.33, -3800.0, -3808.33, -3825.0, -3783.33, -3808.33, -3800.0, -3808.33, -3800.0, -3791.67, -3791.67, -3816.67, -3816.67, -3808.33, -3800.0, -3791.67, -3800.0, -3808.33, -3791.67, -3791.67, -3808.33, -3791.67, -3791.67, -3800.0, -3783.33, -3783.33, -3791.67, -3800.0, -3800.0, -3800.0, -3783.33, -3800.0, -3783.33, -3800.0, -3800.0, -3775.0, -3808.33, -3791.67, -3791.67, -3800.0, -3783.33, -3800.0, -3791.67, -3800.0, -3800.0, -3783.33, -3783.33, -3791.67, -3791.67, -3791.67, -3800.0, -3750.0, -3825.0, -3791.67, -3741.67, -3825.0, -3775.0, -3791.67, -3775.0, -3791.67, -3791.67, -3791.67, -3800.0, -3775.0, -3791.67, -3800.0, -3800.0, -3800.0, -3791.67, -3800.0, -3800.0, -3775.0, -3800.0, -3791.67, -3808.33, -3800.0, -3816.67, -3800.0, -3816.67, -3791.67, -3791.67, -3741.67, -3800.0, -3791.67, -3800.0, -3808.33, -3783.33, -3800.0, -3791.67, -3800.0, -3775.0, -3791.67, -3783.33, -3816.67, -3808.33, -3825.0, -3758.33, -3816.67, -3841.67, -3783.33, -3808.33, -3825.0, -3775.0, -3816.67, -3791.67, -3825.0, -3758.33, -3800.0, -3775.0, -3775.0, -3766.67, -3775.0, -3750.0, -3758.33, -3791.67, -3775.0, -3758.33, -3791.67, -3791.67, -3741.67, -3783.33, -3791.67, -3725.0, -3775.0, -3791.67, -3766.67, -3758.33, -3791.67, -3750.0, -3766.67, -3725.0, -3791.67, -3766.67, -3766.67, -3791.67, -3908.33, -3775.0, -3816.67, -3766.67, -3750.0, -3841.67, -3766.67, -3816.67, -3758.33, -3808.33, -3775.0, -3800.0, -3741.67, -3750.0, -3791.67, -3758.33, -3783.33, -3783.33, -3791.67]]